In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import os
import time

get_url_final function

In [2]:
#this function just gets the html code that we need to scrape for data, updated version that is not specific to salah 
def get_url_final(code, year_range, category, player):
    base_url = 'https://fbref.com/en/players/{}/matchlogs/{}/c9/{}/{}-Match-Logs'
    url = base_url.format(code, year_range, category, player)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    html_filtered = soup.find('tbody')
    return(html_filtered)

get_dat, get_date and get_matchweek functions

In [3]:
def get_dat(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('td', {'data-stat': col}).text)
    return (coldat)   

def get_date(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('th', {'data-stat': col}).text)
    return (coldat)   

def get_matchweek(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('td', {'data-stat': col}).text)
    coldat = [coldat.replace('Matchweek ', '') for coldat in coldat]
    return(coldat)

get_data function

In [4]:
#now, we want to modify the function above, to allow us to get any players data that we want, not just salah

def get_data(code, year_range, player, fpl_name):
    data_summary = get_url_final(code, year_range, '', player)
    
    #this if statement is checking whether or not this player actually has data for the particular year. In the case that they do not actually have
    #data for this year, just return an empty dataframe 
    if not data_summary:
        empty = pd.DataFrame()
        return(empty)
    
    data_passing = get_url_final(code, year_range, 'passing', player)
    data_passingtype = get_url_final(code, year_range, 'passing_types', player)
    data_gca = get_url_final(code, year_range, 'gca', player)
    data_defense = get_url_final(code, year_range, 'defense', player)
    data_possession = get_url_final(code, year_range, 'possession', player)
    
    ################################################################
    #getting the data for the necessary columns
    
    ##### summary data #####
    #misc data 
    date = get_date(data_summary, 'date')
    day = get_dat(data_summary,'dayofweek')
    matchweek = get_matchweek(data_summary, 'round')
    venue = get_dat(data_summary, 'venue')
    result = get_dat(data_summary, 'result')
    team = get_dat(data_summary, 'team')
    opponent = get_dat(data_summary, 'opponent')
    start = get_dat(data_summary, 'game_started')
    position = get_dat(data_summary, 'position')
    mins = get_dat(data_summary, 'minutes')

    #performance data
    goals = get_dat(data_summary, 'goals')
    assist = get_dat(data_summary, 'assists')
    pen_goals = get_dat(data_summary, 'pens_made')
    pen_attempted = get_dat(data_summary, 'pens_att')
    shots = get_dat(data_summary, 'shots')
    sot = get_dat(data_summary, 'shots_on_target')
    yellow = get_dat(data_summary, 'cards_yellow')
    red = get_dat(data_summary, 'cards_red')
    touches = get_dat(data_summary, 'touches')
    tackles = get_dat(data_summary, 'tackles')
    interceptions = get_dat(data_summary, 'interceptions')
    blocks = get_dat(data_summary, 'blocks')

    #expected performance metrics
    xg = get_dat(data_summary, 'xg')
    npxg = get_dat(data_summary, 'npxg')
    xag = get_dat(data_summary, 'xg_assist')

    #shot creating actions
    sca = get_dat(data_summary, 'sca')
    gca = get_dat(data_summary, 'gca')

    #passes
    passes_completed = get_dat(data_summary, 'passes_completed')
    passes_attempted = get_dat(data_summary, 'passes')
    prg_passes = get_dat(data_summary, 'progressive_passes')

    #carries 
    carries = get_dat(data_summary, 'carries')
    prg_carries = get_dat(data_summary, 'progressive_carries')

    #take-ons
    takeon_att = get_dat(data_summary, 'take_ons')
    takeon_suc = get_dat(data_summary, 'take_ons_won')
    
    ##### passing data #####
    #total
    pass_distance = get_dat(data_passing, 'passes_total_distance')
    prg_pass_distance = get_dat(data_passing, 'passes_progressive_distance')

    #short
    short_pass_completed = get_dat(data_passing, 'passes_completed_short')
    short_pass_attempted = get_dat(data_passing, 'passes_short')

    #medium 
    med_pass_completed = get_dat(data_passing, 'passes_completed_medium')
    med_pass_attempted = get_dat(data_passing, 'passes_medium')

    #long
    long_pass_completed = get_dat(data_passing, 'passes_completed_long')
    long_pass_attempted = get_dat(data_passing, 'passes_long')

    #misc
    xa = get_dat(data_passing, 'pass_xa')
    keypass = get_dat(data_passing, 'assisted_shots')
    finalthird_pass = get_dat(data_passing, 'passes_into_final_third')
    penaltyarea_pass = get_dat(data_passing, 'passes_into_penalty_area')
    penaltyarea_cross = get_dat(data_passing, 'crosses_into_penalty_area')
    
    
    ##### passing types data #####
    live_pass = get_dat(data_passingtype, 'passes_live')
    dead_pass = get_dat(data_passingtype, 'passes_dead')
    freekick_pass = get_dat(data_passingtype, 'passes_free_kicks')
    through_balls = get_dat(data_passingtype, 'through_balls')
    switches = get_dat(data_passingtype, 'passes_switches')
    crosses = get_dat(data_passingtype, 'crosses')
    throwin = get_dat(data_passingtype, 'throw_ins')
    corners = get_dat(data_passingtype, 'corner_kicks')
    offside_pass = get_dat(data_passingtype, 'passes_offsides')
    
    
    ##### gca data #####
    #sca types 
    live_sca = get_dat(data_gca, 'sca_passes_live')
    deadball_sca = get_dat(data_gca, 'sca_passes_dead')
    takeons_sca = get_dat(data_gca, 'sca_take_ons')
    shots_sca = get_dat(data_gca, 'sca_shots')
    fouls_sca = get_dat(data_gca, 'sca_fouled')
    defense_sca = get_dat(data_gca, 'sca_defense')

    #gca types 
    live_gca = get_dat(data_gca, 'gca_passes_live')
    deadball_gca = get_dat(data_gca, 'gca_passes_dead')
    takeons_gca = get_dat(data_gca, 'gca_take_ons')
    shots_gca = get_dat(data_gca, 'gca_shots')
    fouls_gca = get_dat(data_gca, 'gca_fouled')
    defense_gca = get_dat(data_gca, 'gca_defense')
    
    
    ##### defensive data ######
    #tackles
    tackles_won = get_dat(data_defense,'tackles_won')
    tackles_def = get_dat(data_defense, 'tackles_def_3rd')
    tackles_mid = get_dat(data_defense, 'tackles_mid_3rd')
    tackles_att = get_dat(data_defense, 'tackles_att_3rd')

    #challenges 
    dribblers_tackled = get_dat(data_defense, 'challenge_tackles')
    dribbles_challenged = get_dat(data_defense, 'challenges')
    challenges_lost = get_dat(data_defense, 'challenges_lost')

    #blocks 
    shots_blocked = get_dat(data_defense, 'blocked_shots')
    pass_blocked = get_dat(data_defense, 'blocked_passes')
    clearances = get_dat(data_defense, 'clearances')
    def_errors = get_dat(data_defense, 'errors')
    
    
    ##### possession data #####
    #touches 
    def_penarea_touches = get_dat(data_possession, 'touches_def_pen_area')
    def_third_touches = get_dat(data_possession, 'touches_def_3rd')
    mid_third_touches = get_dat(data_possession, 'touches_mid_3rd')
    att_third_touches = get_dat(data_possession, 'touches_att_3rd')
    pen_area_touches = get_dat(data_possession, 'touches_att_pen_area')

    #carries
    carry_distance = get_dat(data_possession, 'carries_distance')
    prg_carry_distance = get_dat(data_possession, 'carries_progressive_distance')
    final_third_carries = get_dat(data_possession, 'carries_into_final_third')
    pen_area_carries = get_dat(data_possession, 'carries_into_penalty_area')
    miscontrols = get_dat(data_possession, 'miscontrols')
    dispossesed = get_dat(data_possession, 'dispossessed')

    #receiving 
    pass_received = get_dat(data_possession, 'passes_received')
    prg_pass_received = get_dat(data_possession, 'progressive_passes_received')


    ###########################################################
    #creating dataframe
    df = pd.DataFrame({
        'Date': date,
        'Day': day,
        'Matchweek': matchweek,
        'Venue': venue,
        'Result': result,
        'Team': team,
        'Opponent': opponent, 
        'Start': start, 
        'Position': position,
        'Minutes Played': mins, 
        'Goals': goals, 
        'Assists': assist, 
        'Penalties Scored': pen_goals, 
        'Penalties Attempted': pen_attempted, 
        'Shots': shots, 
        'Shots on Target': sot, 
        'Yellow Cards': yellow, 
        'Red Cards': red, 
        'Touches': touches, 
        'Tackles': tackles,
        'Interceptions': interceptions, 
        'Blocks': blocks, 
        'xG': xg,
        'npxG': npxg,
        'xAG': xag, 
        'Shot Creating Actions': sca, 
        'Goal Creating Actions': gca, 
        'Passes Completed': passes_completed, 
        'Passes Attempted': passes_attempted, 
        'Progressive Passes': prg_passes,
        'Carries': carries, 
        'Progressive Carries': prg_carries, 
        'Take-ons Attempted': takeon_att, 
        'Successful Take-ons': takeon_suc,
        
        'Passing Distance': pass_distance, 
        'Progressive Passing Distance': prg_pass_distance,
        'Short Passes Completed' : short_pass_completed,
        'Short Passes Attempted': short_pass_attempted,
        'Medium Passes Completed': med_pass_completed,
        'Medium Passes Attempted': med_pass_attempted,
        'Long Passes Completed': long_pass_completed,
        'Long Passes Attempted': long_pass_attempted,
        'Expected Assists': xa,
        'Key Passes': keypass,
        'Passes into Final Third': finalthird_pass,
        'Passes into Penalty Area': penaltyarea_pass,
        'Crosses into Penalty Area': penaltyarea_cross,
    
        'Live Pass': live_pass,
        'Dead Pass': dead_pass,
        'Free Kick Pass': freekick_pass,
        'Through Balls': through_balls,
        'Switches': switches,
        'Crosses': crosses,
        'Throw Ins Taken': throwin,
        'Corners Taken': corners,
        'Passes Offside': offside_pass,
    
        'Live SCA': live_sca,
        'Deadball SCA': deadball_sca,
        'Take-on SCA': takeons_sca,
        'Shot SCA': shots_sca,
        'Foul SCA': fouls_sca,
        'Defense SCA': defense_sca,
    
        'Live GCA': live_gca,
        'Deadball GCA': deadball_gca,
        'Take-on GCA': takeons_gca,
        'Shot GCA': shots_gca,
        'Foul GCA': fouls_gca,
        'Defense GCA': defense_gca,
    
        'Tackles Won': tackles_won,
        'Defensive Third Tackles': tackles_def,
        'Middle Third Tackles': tackles_mid,
        'Attacking Third Tackles': tackles_att,
        'Dribblers Tackled': dribblers_tackled,
        'Dribblers Tackled Attempts': dribbles_challenged,
        'Challenges Lost': challenges_lost,
        'Shots Blocked': shots_blocked,
        'Passes Blocked': pass_blocked,
        'Clearances': clearances,
        'Defensive Errors': def_errors,
    
        'Defensive Penalty Area Touches': def_penarea_touches,
        'Defensive Third Touches': def_third_touches,
        'Middle Third Touches': mid_third_touches,
        'Attacking Third Touches': att_third_touches,
        'Penalty Area Touches': pen_area_touches,
        'Carry Distance': carry_distance,
        'Progressive Carry Distance': prg_carry_distance,
        'Final Third Carries': final_third_carries,
        'Carries into Penalty Area': pen_area_carries,
        'Miscontrols': miscontrols,
        'Dispossessed': dispossesed,
        'Passes Received': pass_received,
        'Progressive Passes Received': prg_pass_received
    })
    
    #modifying data types
    df = df.astype({
        'Date': 'datetime64[ns]',
        'Day': 'object',
        'Matchweek': 'int64',
        'Venue': 'object',
        'Result': 'object',
        'Team': 'object',
        'Opponent': 'object', 
        'Start': 'object', 
        'Position': 'object',
        'Minutes Played': 'int64', 
        'Goals': 'int64', 
        'Assists': 'int64', 
        'Penalties Scored': 'int64', 
        'Penalties Attempted': 'int64', 
        'Shots': 'int64', 
        'Shots on Target': 'int64', 
        'Yellow Cards': 'int64', 
        'Red Cards': 'int64', 
        'Touches': 'int64', 
        'Tackles': 'int64',
        'Interceptions': 'int64', 
        'Blocks': 'int64', 
        'xG': 'float64',
        'npxG': 'float64',
        'xAG': 'float64', 
        'Shot Creating Actions': 'int64', 
        'Goal Creating Actions': 'int64', 
        'Passes Completed': 'int64', 
        'Passes Attempted': 'int64', 
        'Progressive Passes': 'int64',
        'Carries': 'int64', 
        'Progressive Carries': 'int64', 
        'Take-ons Attempted': 'int64', 
        'Successful Take-ons': 'int64',
        
        'Short Passes Completed' : 'int64',
        'Short Passes Attempted': 'int64',
        'Medium Passes Completed': 'int64',
        'Medium Passes Attempted': 'int64',
        'Long Passes Completed': 'int64',
        'Long Passes Attempted': 'int64',
        'Expected Assists': 'float64',
        'Key Passes': 'int64',
        'Passes into Final Third': 'int64',
        'Passes into Penalty Area': 'int64',
        'Crosses into Penalty Area': 'int64',
    
        'Live Pass': 'int64',
        'Dead Pass': 'int64',
        'Free Kick Pass': 'int64',
        'Through Balls': 'int64',
        'Switches': 'int64',
        'Crosses': 'int64',
        'Throw Ins Taken': 'int64',
        'Corners Taken': 'int64',
        'Passes Offside': 'int64',
    
        'Live SCA': 'int64',
        'Deadball SCA': 'int64',
        'Take-on SCA': 'int64',
        'Shot SCA': 'int64',
        'Foul SCA': 'int64',
        'Defense SCA': 'int64',
    
        'Live GCA': 'int64',
        'Deadball GCA': 'int64',
        'Take-on GCA': 'int64',
        'Shot GCA': 'int64',
        'Foul GCA': 'int64',
        'Defense GCA': 'int64',
    
        'Tackles Won': 'int64',
        'Defensive Third Tackles': 'int64',
        'Middle Third Tackles': 'int64',
        'Attacking Third Tackles': 'int64',
        'Dribblers Tackled': 'int64',
        'Dribblers Tackled Attempts': 'int64',
        'Challenges Lost': 'int64',
        'Shots Blocked': 'int64',
        'Passes Blocked': 'int64',
        'Clearances': 'int64',
        'Defensive Errors': 'int64',
    
        'Defensive Penalty Area Touches': 'int64',
        'Defensive Third Touches': 'int64',
        'Middle Third Touches': 'int64',
        'Attacking Third Touches': 'int64',
        'Penalty Area Touches': 'int64',
        'Carry Distance': 'float64',
        'Progressive Carry Distance': 'float64',
        'Final Third Carries': 'int64',
        'Carries into Penalty Area': 'int64',
        'Miscontrols': 'int64',
        'Dispossessed': 'int64',
        'Passes Received': 'int64',
        'Progressive Passes Received': 'int64'
    })
    
    #changing date column data type 
    df['Date'] = df['Date'].dt.date
    
    ###############################################################################
    
    #now we want to get the fpl data 
    start_year = year_range[:4] #getting the whole of the first year 
    end_year = year_range[-2:]
    format_year = f'{start_year}-{end_year}'
    directory = f'Fantasy-Premier-League/data/{format_year}/players'
    files = os.listdir(directory)
    csv_file = [f for f in files if fpl_name in f]
    fpldf = pd.read_csv(os.path.join(directory, csv_file[0]) + '/gw.csv')

    #some of the columns are duplicated (from the webscraped data), so we want to remove these 
    fpldf = fpldf.drop(['assists', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'fixture',
                                          'goals_conceded', 'goals_scored', 'penalties_missed', 'penalties_saved', 'red_cards'
                                          ,'team_a_score', 'team_h_score', 'was_home', 'yellow_cards', 'element', 'opponent_team', 'starts'
                                          ,'expected_goals_conceded'], axis=1, 
                       errors = 'ignore')

    #we now want to change the 'kickoff time' feature into a datetime object 
    fpldf['kickoff_time'] = pd.to_datetime(fpldf['kickoff_time'])

    #creating new column, which is just the kickoff date 
    fpldf['kickoff_date'] = fpldf['kickoff_time'].dt.date
    
    finaldf = pd.merge(df, fpldf, left_on='Date', right_on='kickoff_date', how='inner')
    
    return(finaldf)

In [5]:
#this is just to use the get_data function to create a representative dataframe, with the purpose being just to get the column names. This is why 
#we have to specify another get_data function below (get_data_final), which implements functionality that deals with players who have empty pages in 
#Fbref
test = get_data('66b76d44', '2022-2023', 'Emi-Buendia', 'Emiliano_Buen')
column_names = test.columns

In [6]:
#this is the final version of get_data, which includes additional functionality to deal with players who hav eempty pages in Fbref
def get_data_final(code, year_range, player, fpl_name):
    data_summary = get_url_final(code, year_range, '', player)
    
    #this if statement is checking whether or not this player actually has data for the particular year. In the case that they do not actually have
    #data for this year, just return an empty dataframe 
    if not data_summary:
        empty = pd.DataFrame(columns = column_names)
        return(empty)
    
    data_passing = get_url_final(code, year_range, 'passing', player)
    data_passingtype = get_url_final(code, year_range, 'passing_types', player)
    data_gca = get_url_final(code, year_range, 'gca', player)
    data_defense = get_url_final(code, year_range, 'defense', player)
    data_possession = get_url_final(code, year_range, 'possession', player)
    
    ################################################################
    #getting the data for the necessary columns
    
    ##### summary data #####
    #misc data 
    date = get_date(data_summary, 'date')
    day = get_dat(data_summary,'dayofweek')
    matchweek = get_matchweek(data_summary, 'round')
    venue = get_dat(data_summary, 'venue')
    result = get_dat(data_summary, 'result')
    team = get_dat(data_summary, 'team')
    opponent = get_dat(data_summary, 'opponent')
    start = get_dat(data_summary, 'game_started')
    position = get_dat(data_summary, 'position')
    mins = get_dat(data_summary, 'minutes')

    #performance data
    goals = get_dat(data_summary, 'goals')
    assist = get_dat(data_summary, 'assists')
    pen_goals = get_dat(data_summary, 'pens_made')
    pen_attempted = get_dat(data_summary, 'pens_att')
    shots = get_dat(data_summary, 'shots')
    sot = get_dat(data_summary, 'shots_on_target')
    yellow = get_dat(data_summary, 'cards_yellow')
    red = get_dat(data_summary, 'cards_red')
    touches = get_dat(data_summary, 'touches')
    tackles = get_dat(data_summary, 'tackles')
    interceptions = get_dat(data_summary, 'interceptions')
    blocks = get_dat(data_summary, 'blocks')

    #expected performance metrics
    xg = get_dat(data_summary, 'xg')
    npxg = get_dat(data_summary, 'npxg')
    xag = get_dat(data_summary, 'xg_assist')

    #shot creating actions
    sca = get_dat(data_summary, 'sca')
    gca = get_dat(data_summary, 'gca')

    #passes
    passes_completed = get_dat(data_summary, 'passes_completed')
    passes_attempted = get_dat(data_summary, 'passes')
    prg_passes = get_dat(data_summary, 'progressive_passes')

    #carries 
    carries = get_dat(data_summary, 'carries')
    prg_carries = get_dat(data_summary, 'progressive_carries')

    #take-ons
    takeon_att = get_dat(data_summary, 'take_ons')
    takeon_suc = get_dat(data_summary, 'take_ons_won')
    
    ##### passing data #####
    #total
    pass_distance = get_dat(data_passing, 'passes_total_distance')
    prg_pass_distance = get_dat(data_passing, 'passes_progressive_distance')

    #short
    short_pass_completed = get_dat(data_passing, 'passes_completed_short')
    short_pass_attempted = get_dat(data_passing, 'passes_short')

    #medium 
    med_pass_completed = get_dat(data_passing, 'passes_completed_medium')
    med_pass_attempted = get_dat(data_passing, 'passes_medium')

    #long
    long_pass_completed = get_dat(data_passing, 'passes_completed_long')
    long_pass_attempted = get_dat(data_passing, 'passes_long')

    #misc
    xa = get_dat(data_passing, 'pass_xa')
    keypass = get_dat(data_passing, 'assisted_shots')
    finalthird_pass = get_dat(data_passing, 'passes_into_final_third')
    penaltyarea_pass = get_dat(data_passing, 'passes_into_penalty_area')
    penaltyarea_cross = get_dat(data_passing, 'crosses_into_penalty_area')
    
    
    ##### passing types data #####
    live_pass = get_dat(data_passingtype, 'passes_live')
    dead_pass = get_dat(data_passingtype, 'passes_dead')
    freekick_pass = get_dat(data_passingtype, 'passes_free_kicks')
    through_balls = get_dat(data_passingtype, 'through_balls')
    switches = get_dat(data_passingtype, 'passes_switches')
    crosses = get_dat(data_passingtype, 'crosses')
    throwin = get_dat(data_passingtype, 'throw_ins')
    corners = get_dat(data_passingtype, 'corner_kicks')
    offside_pass = get_dat(data_passingtype, 'passes_offsides')
    
    
    ##### gca data #####
    #sca types 
    live_sca = get_dat(data_gca, 'sca_passes_live')
    deadball_sca = get_dat(data_gca, 'sca_passes_dead')
    takeons_sca = get_dat(data_gca, 'sca_take_ons')
    shots_sca = get_dat(data_gca, 'sca_shots')
    fouls_sca = get_dat(data_gca, 'sca_fouled')
    defense_sca = get_dat(data_gca, 'sca_defense')

    #gca types 
    live_gca = get_dat(data_gca, 'gca_passes_live')
    deadball_gca = get_dat(data_gca, 'gca_passes_dead')
    takeons_gca = get_dat(data_gca, 'gca_take_ons')
    shots_gca = get_dat(data_gca, 'gca_shots')
    fouls_gca = get_dat(data_gca, 'gca_fouled')
    defense_gca = get_dat(data_gca, 'gca_defense')
    
    
    ##### defensive data ######
    #tackles
    tackles_won = get_dat(data_defense,'tackles_won')
    tackles_def = get_dat(data_defense, 'tackles_def_3rd')
    tackles_mid = get_dat(data_defense, 'tackles_mid_3rd')
    tackles_att = get_dat(data_defense, 'tackles_att_3rd')

    #challenges 
    dribblers_tackled = get_dat(data_defense, 'challenge_tackles')
    dribbles_challenged = get_dat(data_defense, 'challenges')
    challenges_lost = get_dat(data_defense, 'challenges_lost')

    #blocks 
    shots_blocked = get_dat(data_defense, 'blocked_shots')
    pass_blocked = get_dat(data_defense, 'blocked_passes')
    clearances = get_dat(data_defense, 'clearances')
    def_errors = get_dat(data_defense, 'errors')
    
    
    ##### possession data #####
    #touches 
    def_penarea_touches = get_dat(data_possession, 'touches_def_pen_area')
    def_third_touches = get_dat(data_possession, 'touches_def_3rd')
    mid_third_touches = get_dat(data_possession, 'touches_mid_3rd')
    att_third_touches = get_dat(data_possession, 'touches_att_3rd')
    pen_area_touches = get_dat(data_possession, 'touches_att_pen_area')

    #carries
    carry_distance = get_dat(data_possession, 'carries_distance')
    prg_carry_distance = get_dat(data_possession, 'carries_progressive_distance')
    final_third_carries = get_dat(data_possession, 'carries_into_final_third')
    pen_area_carries = get_dat(data_possession, 'carries_into_penalty_area')
    miscontrols = get_dat(data_possession, 'miscontrols')
    dispossesed = get_dat(data_possession, 'dispossessed')

    #receiving 
    pass_received = get_dat(data_possession, 'passes_received')
    prg_pass_received = get_dat(data_possession, 'progressive_passes_received')


    ###########################################################
    #creating dataframe
    df = pd.DataFrame({
        'Date': date,
        'Day': day,
        'Matchweek': matchweek,
        'Venue': venue,
        'Result': result,
        'Team': team,
        'Opponent': opponent, 
        'Start': start, 
        'Position': position,
        'Minutes Played': mins, 
        'Goals': goals, 
        'Assists': assist, 
        'Penalties Scored': pen_goals, 
        'Penalties Attempted': pen_attempted, 
        'Shots': shots, 
        'Shots on Target': sot, 
        'Yellow Cards': yellow, 
        'Red Cards': red, 
        'Touches': touches, 
        'Tackles': tackles,
        'Interceptions': interceptions, 
        'Blocks': blocks, 
        'xG': xg,
        'npxG': npxg,
        'xAG': xag, 
        'Shot Creating Actions': sca, 
        'Goal Creating Actions': gca, 
        'Passes Completed': passes_completed, 
        'Passes Attempted': passes_attempted, 
        'Progressive Passes': prg_passes,
        'Carries': carries, 
        'Progressive Carries': prg_carries, 
        'Take-ons Attempted': takeon_att, 
        'Successful Take-ons': takeon_suc,
        
        'Passing Distance': pass_distance, 
        'Progressive Passing Distance': prg_pass_distance,
        'Short Passes Completed' : short_pass_completed,
        'Short Passes Attempted': short_pass_attempted,
        'Medium Passes Completed': med_pass_completed,
        'Medium Passes Attempted': med_pass_attempted,
        'Long Passes Completed': long_pass_completed,
        'Long Passes Attempted': long_pass_attempted,
        'Expected Assists': xa,
        'Key Passes': keypass,
        'Passes into Final Third': finalthird_pass,
        'Passes into Penalty Area': penaltyarea_pass,
        'Crosses into Penalty Area': penaltyarea_cross,
    
        'Live Pass': live_pass,
        'Dead Pass': dead_pass,
        'Free Kick Pass': freekick_pass,
        'Through Balls': through_balls,
        'Switches': switches,
        'Crosses': crosses,
        'Throw Ins Taken': throwin,
        'Corners Taken': corners,
        'Passes Offside': offside_pass,
    
        'Live SCA': live_sca,
        'Deadball SCA': deadball_sca,
        'Take-on SCA': takeons_sca,
        'Shot SCA': shots_sca,
        'Foul SCA': fouls_sca,
        'Defense SCA': defense_sca,
    
        'Live GCA': live_gca,
        'Deadball GCA': deadball_gca,
        'Take-on GCA': takeons_gca,
        'Shot GCA': shots_gca,
        'Foul GCA': fouls_gca,
        'Defense GCA': defense_gca,
    
        'Tackles Won': tackles_won,
        'Defensive Third Tackles': tackles_def,
        'Middle Third Tackles': tackles_mid,
        'Attacking Third Tackles': tackles_att,
        'Dribblers Tackled': dribblers_tackled,
        'Dribblers Tackled Attempts': dribbles_challenged,
        'Challenges Lost': challenges_lost,
        'Shots Blocked': shots_blocked,
        'Passes Blocked': pass_blocked,
        'Clearances': clearances,
        'Defensive Errors': def_errors,
    
        'Defensive Penalty Area Touches': def_penarea_touches,
        'Defensive Third Touches': def_third_touches,
        'Middle Third Touches': mid_third_touches,
        'Attacking Third Touches': att_third_touches,
        'Penalty Area Touches': pen_area_touches,
        'Carry Distance': carry_distance,
        'Progressive Carry Distance': prg_carry_distance,
        'Final Third Carries': final_third_carries,
        'Carries into Penalty Area': pen_area_carries,
        'Miscontrols': miscontrols,
        'Dispossessed': dispossesed,
        'Passes Received': pass_received,
        'Progressive Passes Received': prg_pass_received
    })
    
    for column in df.columns:
        df[column] = df[column].replace('',0) #replacing empty strings with zero 
    
    #modifying data types
    df = df.astype({
        'Date': 'datetime64[ns]',
        'Day': 'object',
        'Matchweek': 'int64',
        'Venue': 'object',
        'Result': 'object',
        'Team': 'object',
        'Opponent': 'object', 
        'Start': 'object', 
        'Position': 'object',
        'Minutes Played': 'int64', 
        'Goals': 'int64', 
        'Assists': 'int64', 
        'Penalties Scored': 'int64', 
        'Penalties Attempted': 'int64', 
        'Shots': 'int64', 
        'Shots on Target': 'int64', 
        'Yellow Cards': 'int64', 
        'Red Cards': 'int64', 
        'Touches': 'int64', 
        'Tackles': 'int64',
        'Interceptions': 'int64', 
        'Blocks': 'int64', 
        'xG': 'float64',
        'npxG': 'float64',
        'xAG': 'float64', 
        'Shot Creating Actions': 'int64', 
        'Goal Creating Actions': 'int64', 
        'Passes Completed': 'int64', 
        'Passes Attempted': 'int64', 
        'Progressive Passes': 'int64',
        'Carries': 'int64', 
        'Progressive Carries': 'int64', 
        'Take-ons Attempted': 'int64', 
        'Successful Take-ons': 'int64',
        
        'Short Passes Completed' : 'int64',
        'Short Passes Attempted': 'int64',
        'Medium Passes Completed': 'int64',
        'Medium Passes Attempted': 'int64',
        'Long Passes Completed': 'int64',
        'Long Passes Attempted': 'int64',
        'Expected Assists': 'float64',
        'Key Passes': 'int64',
        'Passes into Final Third': 'int64',
        'Passes into Penalty Area': 'int64',
        'Crosses into Penalty Area': 'int64',
    
        'Live Pass': 'int64',
        'Dead Pass': 'int64',
        'Free Kick Pass': 'int64',
        'Through Balls': 'int64',
        'Switches': 'int64',
        'Crosses': 'int64',
        'Throw Ins Taken': 'int64',
        'Corners Taken': 'int64',
        'Passes Offside': 'int64',
    
        'Live SCA': 'int64',
        'Deadball SCA': 'int64',
        'Take-on SCA': 'int64',
        'Shot SCA': 'int64',
        'Foul SCA': 'int64',
        'Defense SCA': 'int64',
    
        'Live GCA': 'int64',
        'Deadball GCA': 'int64',
        'Take-on GCA': 'int64',
        'Shot GCA': 'int64',
        'Foul GCA': 'int64',
        'Defense GCA': 'int64',
    
        'Tackles Won': 'int64',
        'Defensive Third Tackles': 'int64',
        'Middle Third Tackles': 'int64',
        'Attacking Third Tackles': 'int64',
        'Dribblers Tackled': 'int64',
        'Dribblers Tackled Attempts': 'int64',
        'Challenges Lost': 'int64',
        'Shots Blocked': 'int64',
        'Passes Blocked': 'int64',
        'Clearances': 'int64',
        'Defensive Errors': 'int64',
    
        'Defensive Penalty Area Touches': 'int64',
        'Defensive Third Touches': 'int64',
        'Middle Third Touches': 'int64',
        'Attacking Third Touches': 'int64',
        'Penalty Area Touches': 'int64',
        'Carry Distance': 'float64',
        'Progressive Carry Distance': 'float64',
        'Final Third Carries': 'int64',
        'Carries into Penalty Area': 'int64',
        'Miscontrols': 'int64',
        'Dispossessed': 'int64',
        'Passes Received': 'int64',
        'Progressive Passes Received': 'int64'
    })
    
    #changing date column data type 
    df['Date'] = df['Date'].dt.date
    
    ###############################################################################
    
    #now we want to get the fpl data 
    start_year = year_range[:4] #getting the whole of the first year 
    end_year = year_range[-2:]
    format_year = f'{start_year}-{end_year}'
    directory = f'Fantasy-Premier-League/data/{format_year}/players'
    files = os.listdir(directory)
    csv_file = [f for f in files if fpl_name in f]
    fpldf = pd.read_csv(os.path.join(directory, csv_file[0]) + '/gw.csv')

    #some of the columns are duplicated (from the webscraped data), so we want to remove these 
    fpldf = fpldf.drop(['assists', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'fixture',
                                          'goals_conceded', 'goals_scored', 'penalties_missed', 'penalties_saved', 'red_cards'
                                          ,'team_a_score', 'team_h_score', 'was_home', 'yellow_cards', 'element', 'opponent_team', 'starts'
                                          ,'expected_goals_conceded'], axis=1, 
                       errors = 'ignore')

    #we now want to change the 'kickoff time' feature into a datetime object 
    fpldf['kickoff_time'] = pd.to_datetime(fpldf['kickoff_time'])

    #creating new column, which is just the kickoff date 
    fpldf['kickoff_date'] = fpldf['kickoff_time'].dt.date
    
    finaldf = pd.merge(df, fpldf, left_on='Date', right_on='kickoff_date', how='inner')
    
    return(finaldf)

get_premgames function

In [7]:
def get_premgames (code, player):
    base_url = f'https://fbref.com/en/players/{code}/{player}'
    html = requests.get(base_url).text
    soup = BeautifulSoup(html, 'lxml')
    summary = soup.find('table', class_='stats_table sortable min_width')
    table = summary.find('tbody')
    
    comp = table.find_all('td', attrs={'data-stat': 'comp_level'})
    comp_text = [cell.get_text() for cell in comp]
    games = table.find_all('td', attrs={'data-stat': 'games'})
    games_text = [cell.get_text() for cell in games]
    season = table.find_all('th', attrs={'data-stat': 'year_id'})
    season_text  = [cell.get_text() for cell in season]
    
    country = table.find_all('td', attrs={'data-stat': 'country'})
    country_text = [cell.get_text() for cell in country]
    
    df = pd.DataFrame({
        'Season': season_text,
        'Competition': comp_text, 
        'Games Played': games_text,
        'Country': country_text
    })

    season_list = ('2023-2024', '2022-2023', '2021-2022', '2020-2021', '2019-2020', '2018-2019', '2017-2018')
    total_games = df[(df['Competition'] == '1. Premier League') & (df['Season'].isin(season_list)) & (df['Country'] == 'eng ENG')]['Games Played']
    total_games = pd.to_numeric(total_games)
    prem_games = total_games.sum()
    return(prem_games)

This code specifies the location to place the .csv file into

In [8]:
nest_folder = ("Player_Data")
os.makedirs(nest_folder, exist_ok = True)

nest_folder_def = os.path.join("Player_Data", "Defenders")

## Now, we actually use the functions above to get the necessary data 

### Diogo Jota

In [35]:
jota_2324_finaldf = get_data('178ae8f8', '2023-2024', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [9]:
jota_2223_finaldf = get_data('178ae8f8', '2022-2023', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [10]:
jota_2122_finaldf = get_data('178ae8f8', '2021-2022', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [12]:
jota_2021_finaldf = get_data('178ae8f8', '2020-2021', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [6]:
jota_1920_finaldf = get_data('178ae8f8', '2019-2020', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [7]:
jota_1819_finaldf = get_data('178ae8f8', '2018-2019', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [26]:
jota_finaldat = pd.concat([jota_1819_finaldf, jota_1920_finaldf, jota_2021_finaldf, jota_2122_finaldf
                            , jota_2223_finaldf, jota_2324_finaldf], join = 'inner', ignore_index = True)
jota_finaldat.to_csv(os.path.join(nest_folder, "jota_finaldat.csv"))

### Luis Diaz

In [27]:
diaz_2324_finaldf = get_data('4a1a9578', '2023-2024', 'Luis-Diaz',  "Luis_Díaz")

In [28]:
diaz_2223_finaldf = get_data('4a1a9578', '2022-2023', 'Luis-Diaz',  "Luis_Díaz")

In [29]:
diaz_2122_finaldf = get_data('4a1a9578', '2021-2022', 'Luis-Diaz',  "Luis_Díaz")

In [30]:
diaz_finaldat = pd.concat([diaz_2122_finaldf, diaz_2223_finaldf, diaz_2324_finaldf], join = 'inner', ignore_index = True)
diaz_finaldat.to_csv(os.path.join(nest_folder, "diaz_finaldat.csv"))

### Cody Gakpo

In [31]:
gakpo_2324_finaldf = get_data('1971591f', '2023-2024', 'Cody-Gakpo',  "Cody_Gakpo")

In [32]:
gakpo_2223_finaldf = get_data('1971591f', '2022-2023', 'Cody-Gakpo',  "Cody_Gakpo")

In [33]:
gakpo_finaldat = pd.concat([gakpo_2223_finaldf, gakpo_2324_finaldf], join = 'inner', ignore_index = True)
gakpo_finaldat.to_csv(os.path.join(nest_folder, "gakpo_finaldat.csv"))

### Darwin Nunez

In [36]:
darwin_2324_finaldf = get_data('4d77b365', '2023-2024', 'Darwin-Nunez',  "Darwin_Núñez Ribeiro")

In [38]:
darwin_2223_finaldf = get_data('4d77b365', '2022-2023', 'Darwin-Nunez',  "Darwin")

In [39]:
darwin_finaldat = pd.concat([darwin_2223_finaldf, darwin_2324_finaldf], join = 'inner', ignore_index = True)
darwin_finaldat.to_csv(os.path.join(nest_folder, "darwin_finaldat.csv"))

# Updated Function
This new function compile_dat allows us to get all of the data for a single player in one function, instead of having to do it one by one as can be seen above. 

In [9]:
def compile_dat (code, player, fpl_name, checkgames):
    #these are the seasons for which we have fpl data 
    year_list = ('2023-24', '2022-23', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18')
    
    #active_years is a list that will store the seasons that a given player was active. The for loop will check to see which years a player was active
    #in the prem, and will append this year to the list
    active_years = []
    for year in year_list: 
        directory = f'Fantasy-Premier-League/data/{year}/players'
        files = os.listdir(directory)
        csv_file = [f for f in files if fpl_name in f]
        if csv_file:
            update_year = year[:5] + '20' + year[5:]
            active_years.append(update_year)
    
    #create an empty dictionary to store all of the dataframes 
    dataframes = {}
    
    #for each season a player was active, the get_data function from above is used to add the data for that season to the dataframes dictionary
    for year in active_years: 
        dataframes[year] = get_data_final(code, year, player, fpl_name)
        #1 minute pause in between each iteration, to ensure that we don't get banned from FBref
        time.sleep(60)
    
    #concatenate all of the dataframes, and return one final concatenated dataframe
    finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)
    
    if checkgames:
        games_played = get_premgames(code, player)
        if games_played == finaldf.shape[0]:
            return(finaldf)
        else:
            return(print('The number of Premier League games played by ' + player + '(' + str(games_played) +  ') does not match the number of rows in the dataframe ('+ str(finaldf.shape[0]) + ')'))
    else: 
        return(finaldf)

#note that the to_csv code is not included in this function, because there are certain players whose names were changed in between seasons 
#(e.g. Diogo Teixeira da Silva to Diogo Jota). In these cases, we need to use the compile function twice to get all of the data, before concatenating
#and then finally exporting to csv 

## Arsenal

In [43]:
saka_finaldat = compile_dat('bc7dc64d', 'Bukayo-Saka', 'Bukayo')
saka_finaldat.to_csv(os.path.join(nest_folder, "saka_finaldat.csv"))

In [45]:
havertz_finaldat = compile_dat('fed7cb61', 'Kai-Havertz', 'Kai_Havertz')
havertz_finaldat.to_csv(os.path.join(nest_folder, "havertz_finaldat.csv"))

In [50]:
jesus_finaldat = compile_dat('b66315ae', 'Gabriel-Jesus', 'de Jesus')
jesus_finaldat.to_csv(os.path.join(nest_folder, "jesus_finaldat.csv"))

In [47]:
trossard_finaldat = compile_dat('38ceb24a', 'Leandro-Trossard', 'Leandro_Trossard')
trossard_finaldat.to_csv(os.path.join(nest_folder, "trossard_finaldat.csv"))

In [51]:
martinelli_finaldat = compile_dat('48a5a5d6', 'Gabriel-Martinelli', 'Martinelli')
martinelli_finaldat.to_csv(os.path.join(nest_folder, "martinelli_finaldat.csv"))

In [52]:
odegaard_finaldat = compile_dat('79300479', 'Martin-Odegaard', 'Ødegaard')
odegaard_finaldat.to_csv(os.path.join(nest_folder, "odegaard_finaldat.csv"))

In [53]:
sterling_finaldat = compile_dat('b400bde0', 'Raheem-Sterling', 'Raheem_Sterling')
sterling_finaldat.to_csv(os.path.join(nest_folder, "sterling_finaldat.csv"))

In [88]:
jorginho_finaldat = compile_dat('45db685d', 'Jorginho', 'Jorge Luiz_Frello', checkgames = True)
if isinstance(jorginho_finaldat, pd.DataFrame):
    jorginho_finaldat.to_csv(os.path.join(nest_folder, "jorginho_finaldat.csv"))

In [100]:
rice_finaldat = compile_dat('1c7012b8', 'Declan-Rice', 'Declan_Rice', checkgames = True)
if isinstance(rice_finaldat, pd.DataFrame):
    rice_finaldat.to_csv(os.path.join(nest_folder, "rice_finaldat.csv"))

In [90]:
partey_finaldat = compile_dat('529f49ab', 'Thomas-Partey', 'Thomas_Partey', checkgames = True)
if isinstance(partey_finaldat, pd.DataFrame):
    partey_finaldat.to_csv(os.path.join(nest_folder, "partey_finaldat.csv"))

Arsenal Defenders

In [11]:
gabriel_finaldat = compile_dat('67ac5bb8', 'Gabriel-Magalhaes', 'Magalh', checkgames = True)
if isinstance(gabriel_finaldat, pd.DataFrame):
    gabriel_finaldat.to_csv(os.path.join(nest_folder_def, "gabriel_finaldat.csv"))

In [12]:
saliba_finaldat = compile_dat('972aeb2a', 'William-Saliba', 'William_Saliba', checkgames = True)
if isinstance(saliba_finaldat, pd.DataFrame):
    saliba_finaldat.to_csv(os.path.join(nest_folder_def, "saliba_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [13]:
white_finaldat = compile_dat('35e413f1', 'Ben-White', 'Ben_White', checkgames = True)
if isinstance(white_finaldat, pd.DataFrame):
    white_finaldat.to_csv(os.path.join(nest_folder_def, "white_finaldat.csv"))

In [14]:
zinchenko_finaldat = compile_dat('51cf8561', 'Oleksandr-Zinchenko', 'Oleksandr_Zinchenko', checkgames = True)
if isinstance(zinchenko_finaldat, pd.DataFrame):
    zinchenko_finaldat.to_csv(os.path.join(nest_folder_def, "zinchenko_finaldat.csv"))

In [15]:
kiwior_finaldat = compile_dat('dc3e663e', 'Jakub-Kiwior', 'Jakub_Kiwior', checkgames = True)
if isinstance(kiwior_finaldat, pd.DataFrame):
    kiwior_finaldat.to_csv(os.path.join(nest_folder_def, "kiwior_finaldat.csv"))

In [16]:
tierney_finaldat = compile_dat('fce2302c', 'Kieran-Tierney', 'Kieran_Tierney', checkgames = True)
if isinstance(tierney_finaldat, pd.DataFrame):
    tierney_finaldat.to_csv(os.path.join(nest_folder_def, "tierney_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [17]:
tomiyasu_finaldat = compile_dat('b3af9be1', 'Takehiro-Tomiyasu', 'Takehiro_Tomiyasu', checkgames = True)
if isinstance(tomiyasu_finaldat, pd.DataFrame):
    tomiyasu_finaldat.to_csv(os.path.join(nest_folder_def, "tomiyasu_finaldat.csv"))

## Aston Villa

In [61]:
duran_finaldat = compile_dat('414184f7', 'Jhon-Duran', 'Jhon', checkgames=True)
duran_finaldat.to_csv(os.path.join(nest_folder, "duran_finaldat.csv"))

In [63]:
watkins_finaldat = compile_dat('aed3a70f', 'Ollie-Watkins', 'Ollie_Watkins', checkgames=True)
if isinstance(watkins_finaldat, pd.DataFrame):
    watkins_finaldat.to_csv(os.path.join(nest_folder, "watkins_finaldat.csv"))

In [64]:
ramsey_finaldat = compile_dat('1544f145', 'Jacob-Ramsey', 'Jacob_Ramsey', checkgames = True)
if isinstance(ramsey_finaldat, pd.DataFrame):
    ramsey_finaldat.to_csv(os.path.join(nest_folder, "ramsey_finaldat.csv"))

In [68]:
mcginn_finaldat = compile_dat('90f91999', 'John-McGinn', 'John_McGinn', checkgames = False)
if isinstance(mcginn_finaldat, pd.DataFrame):
    mcginn_finaldat.to_csv(os.path.join(nest_folder, "mcginn_finaldat.csv"))

In [66]:
bailey_finaldat = compile_dat('3a233281', 'Leon-Bailey', 'Leon_Bailey', checkgames = True)
if isinstance(bailey_finaldat, pd.DataFrame):
    bailey_finaldat.to_csv(os.path.join(nest_folder, "bailey_finaldat.csv"))

In [84]:
buendia_finaldat = compile_dat('66b76d44', 'Emi-Buendia', 'Emiliano_Buen', checkgames = True)
if isinstance(buendia_finaldat, pd.DataFrame):
    buendia_finaldat.to_csv(os.path.join(nest_folder, "buendia_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/2405533476.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [69]:
tielemans_finaldat = compile_dat('56f7a928', 'Youri-Tielemans', 'Youri_Tielemans', checkgames = True)
if isinstance(tielemans_finaldat, pd.DataFrame):
    tielemans_finaldat.to_csv(os.path.join(nest_folder, "tielemans_finaldat.csv"))

In [92]:
barkley_finaldat = compile_dat('3a24769f', 'Ross-Barkley', 'Ross_Barkley', checkgames = False)
if isinstance(barkley_finaldat, pd.DataFrame):
    barkley_finaldat.to_csv(os.path.join(nest_folder, "barkley_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/2561086557.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [86]:
onana_finaldat = compile_dat('828657ff', 'Amadou-Onana', 'Amadou_Onana', checkgames = True)
if isinstance(onana_finaldat, pd.DataFrame):
    onana_finaldat.to_csv(os.path.join(nest_folder, "onana_finaldat.csv"))

In [87]:
rogers_finaldat = compile_dat('2e5915f1', 'Morgan-Rogers', 'Morgan_Rogers', checkgames = True)
if isinstance(rogers_finaldat, pd.DataFrame):
    rogers_finaldat.to_csv(os.path.join(nest_folder, "rogers_finaldat.csv"))

Aston Villa Defenders

In [18]:
digne_finaldat = compile_dat('1b84dbe1', 'Lucas-Digne', 'Lucas_Digne', checkgames = True)
if isinstance(digne_finaldat, pd.DataFrame):
    digne_finaldat.to_csv(os.path.join(nest_folder_def, "digne_finaldat.csv"))

In [19]:
konsa_finaldat = compile_dat('0313a347', 'Ezri-Konsa', 'Ezri_Konsa', checkgames = True)
if isinstance(konsa_finaldat, pd.DataFrame):
    konsa_finaldat.to_csv(os.path.join(nest_folder_def, "konsa_finaldat.csv"))

In [20]:
pau_finaldat = compile_dat('532e1e4f', 'Pau-Torres', 'Pau_Torres', checkgames = True)
if isinstance(pau_finaldat, pd.DataFrame):
    pau_finaldat.to_csv(os.path.join(nest_folder_def, "pau_finaldat.csv"))

In [21]:
cash_finaldat = compile_dat('2389cdc2', 'Matty-Cash', 'Cash', checkgames = True)
if isinstance(cash_finaldat, pd.DataFrame):
    cash_finaldat.to_csv(os.path.join(nest_folder_def, "cash_finaldat.csv"))

In [22]:
dcarlos_finaldat = compile_dat('b4a014b1', 'Diego-Carlos', 'Diego Carlos', checkgames = True)
if isinstance(dcarlos_finaldat, pd.DataFrame):
    dcarlos_finaldat.to_csv(os.path.join(nest_folder_def, "dcarlos_finaldat.csv"))

In [23]:
hause_finaldat = compile_dat('bf8fad51', 'Kortney-Hause', 'Kortney_Hause', checkgames = True)
if isinstance(hause_finaldat, pd.DataFrame):
    hause_finaldat.to_csv(os.path.join(nest_folder_def, "hause_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [24]:
mings_finaldat = compile_dat('8397a50c', 'Tyrone-Mings', 'Tyrone_Mings', checkgames = True)
if isinstance(mings_finaldat, pd.DataFrame):
    mings_finaldat.to_csv(os.path.join(nest_folder_def, "mings_finaldat.csv"))

## Bournemouth

In [101]:
unal_finaldat = compile_dat('f8eca1b6', 'Enes-Unal', 'Enes', checkgames = True)
if isinstance(unal_finaldat, pd.DataFrame):
    unal_finaldat.to_csv(os.path.join(nest_folder, "unal_finaldat.csv"))

In [102]:
semenyo_finaldat = compile_dat('efd2ec23', 'Antoine-Semenyo', 'Antoine_Semenyo', checkgames = True)
if isinstance(semenyo_finaldat, pd.DataFrame):
    semenyo_finaldat.to_csv(os.path.join(nest_folder, "semenyo_finaldat.csv"))

In [103]:
sinisterra_finaldat = compile_dat('8e16dd48', 'Luis-Sinisterra', 'Luis_Sinisterra', checkgames = True)
if isinstance(sinisterra_finaldat, pd.DataFrame):
    sinisterra_finaldat.to_csv(os.path.join(nest_folder, "sinisterra_finaldat.csv"))

In [104]:
tavernier_finaldat = compile_dat('5c0da4a4', 'Marcus-Tavernier', 'Marcus_Tavernier', checkgames = True)
if isinstance(tavernier_finaldat, pd.DataFrame):
    tavernier_finaldat.to_csv(os.path.join(nest_folder, "tavernier_finaldat.csv"))

In [105]:
cook_finaldat = compile_dat('2afc7272', 'Lewis-Cook', 'Lewis_Cook', checkgames = True)
if isinstance(cook_finaldat, pd.DataFrame):
    cook_finaldat.to_csv(os.path.join(nest_folder, "cook_finaldat.csv"))

In [106]:
kluivert_finaldat = compile_dat('4c3a6744', 'Justin-Kluivert', 'Justin_Kluivert', checkgames = True)
if isinstance(kluivert_finaldat, pd.DataFrame):
    kluivert_finaldat.to_csv(os.path.join(nest_folder, "kluivert_finaldat.csv"))

In [107]:
dango_finaldat = compile_dat('2f9e4435', 'Dango-Ouattara', 'Dango_Ouattara', checkgames = True)
if isinstance(dango_finaldat, pd.DataFrame):
    dango_finaldat.to_csv(os.path.join(nest_folder, "dango_finaldat.csv"))

In [108]:
christie_finaldat = compile_dat('26ce2263', 'Ryan-Christie', 'Ryan_Christie', checkgames = True)
if isinstance(christie_finaldat, pd.DataFrame):
    christie_finaldat.to_csv(os.path.join(nest_folder, "christie_finaldat.csv"))

In [109]:
scott_finaldat = compile_dat('104d0bb8', 'Alex-Scott', 'Alex_Scott', checkgames = True)
if isinstance(scott_finaldat, pd.DataFrame):
    scott_finaldat.to_csv(os.path.join(nest_folder, "scott_finaldat.csv"))

In [110]:
billing_finaldat = compile_dat('d328a254', 'Philip-Billing', 'Philip_Billing', checkgames = True)
if isinstance(billing_finaldat, pd.DataFrame):
    billing_finaldat.to_csv(os.path.join(nest_folder, "billing_finaldat.csv"))

In [111]:
adams_finaldat = compile_dat('2b09d998', 'Tyler-Adams', 'Tyler_Adams', checkgames = True)
if isinstance(adams_finaldat, pd.DataFrame):
    adams_finaldat.to_csv(os.path.join(nest_folder, "adams_finaldat.csv"))

In [113]:
brooks_finaldat = compile_dat('dc4cae05', 'David-Brooks', 'David_Brooks', checkgames = True)
if isinstance(brooks_finaldat, pd.DataFrame):
    brooks_finaldat.to_csv(os.path.join(nest_folder, "brooks_finaldat.csv"))

Bournemouth Defenders

In [25]:
zabarnyi_finaldat = compile_dat('88968486', 'Illia_Zabarnyi', 'Illia_Zabarnyi', checkgames = False)
if isinstance(zabarnyi_finaldat, pd.DataFrame):
    zabarnyi_finaldat.to_csv(os.path.join(nest_folder_def, "zabarnyi_finaldat.csv"))

The number of Premier League games played by Illia_Zabarnyi(92) does not match the number of rows in the dataframe (42)


In [26]:
kerkez_finaldat = compile_dat('0ad53bdc', 'Milos-Kerkez', 'Milos_Kerkez', checkgames = True)
if isinstance(kerkez_finaldat, pd.DataFrame):
    kerkez_finaldat.to_csv(os.path.join(nest_folder_def, "kerkez_finaldat.csv"))

In [27]:
asmith_finaldat = compile_dat('20b104bc', 'Adam-Smith', 'Adam_Smith', checkgames = True)
if isinstance(asmith_finaldat, pd.DataFrame):
    asmith_finaldat.to_csv(os.path.join(nest_folder_def, "asmith_finaldat.csv"))

In [28]:
senesi_finaldat = compile_dat('35141f4c', 'Marcos-Senesi', 'Senesi', checkgames = True)
if isinstance(senesi_finaldat, pd.DataFrame):
    senesi_finaldat.to_csv(os.path.join(nest_folder_def, "senesi_finaldat.csv"))

In [31]:
aarons_finaldat = compile_dat('774cf58b', 'Max-Aarons', 'Max_Aarons', checkgames = True)
if isinstance(aarons_finaldat, pd.DataFrame):
    aarons_finaldat.to_csv(os.path.join(nest_folder_def, "aarons_finaldat.csv"))

## Brentford

In [114]:
toney_finaldat = compile_dat('e09f279b', 'Ivan-Toney', 'Ivan_Toney', checkgames = True)
if isinstance(toney_finaldat, pd.DataFrame):
    toney_finaldat.to_csv(os.path.join(nest_folder, "toney_finaldat.csv"))

In [115]:
wissa_finaldat = compile_dat('2500cef9', 'Yoane-Wissa', 'Yoane_Wissa', checkgames = True)
if isinstance(wissa_finaldat, pd.DataFrame):
    wissa_finaldat.to_csv(os.path.join(nest_folder, "wissa_finaldat.csv"))

In [116]:
mbeumo_finaldat = compile_dat('6afaebf2', 'Bryan-Mbeumo', 'Bryan_Mbeumo', checkgames = True)
if isinstance(mbeumo_finaldat, pd.DataFrame):
    mbeumo_finaldat.to_csv(os.path.join(nest_folder, "mbeumo_finaldat.csv"))

In [117]:
jensen_finaldat = compile_dat('0f134faf', 'Mathias-Jensen', 'Mathias_Jensen', checkgames = True)
if isinstance(jensen_finaldat, pd.DataFrame):
    jensen_finaldat.to_csv(os.path.join(nest_folder, "jensen_finaldat.csv"))

In [118]:
schade_finaldat = compile_dat('52afb588', 'Kevin-Schade', 'Kevin_Schade', checkgames = True)
if isinstance(schade_finaldat, pd.DataFrame):
    schade_finaldat.to_csv(os.path.join(nest_folder, "schade_finaldat.csv"))

In [119]:
janelt_finaldat = compile_dat('8449d35e', 'Vitaly-Janelt', 'Vitaly_Janelt', checkgames = True)
if isinstance(janelt_finaldat, pd.DataFrame):
    janelt_finaldat.to_csv(os.path.join(nest_folder, "janelt_finaldat.csv"))

In [120]:
damsgaard_finaldat = compile_dat('215f3907', 'Mikkel-Damsgaard', 'Mikkel_Damsgaard', checkgames = True)
if isinstance(damsgaard_finaldat, pd.DataFrame):
    damsgaard_finaldat.to_csv(os.path.join(nest_folder, "damsgaard_finaldat.csv"))

In [121]:
norgaard_finaldat = compile_dat('df0a4c90', 'Christian-Norgaard', 'Christian_Nø', checkgames = True)
if isinstance(norgaard_finaldat, pd.DataFrame):
    norgaard_finaldat.to_csv(os.path.join(nest_folder, "norgaard_finaldat.csv"))

In [122]:
lewispotter_finaldat = compile_dat('41f08ac8', 'Keane-Lewis-Potter', 'Keane_Lewis', checkgames = True)
if isinstance(lewispotter_finaldat, pd.DataFrame):
    lewispotter_finaldat.to_csv(os.path.join(nest_folder, "lewispotter_finaldat.csv"))

Brentford Defenders

In [32]:
collins_finaldat = compile_dat('a8c19eb8', 'Nathan-Collins', 'Nathan_Collins', checkgames = True)
if isinstance(collins_finaldat, pd.DataFrame):
    collins_finaldat.to_csv(os.path.join(nest_folder_def, "collins_finaldat.csv"))

In [33]:
ajer_finaldat = compile_dat('a8c0acb7', 'Kristoffer-Ajer', 'Kristoffer_Ajer', checkgames = True)
if isinstance(ajer_finaldat, pd.DataFrame):
    ajer_finaldat.to_csv(os.path.join(nest_folder_def, "ajer_finaldat.csv"))

In [34]:
roerslev_finaldat = compile_dat('57c94db2', 'Mads-Roerslev', 'Mads_Roerslev', checkgames = True)
if isinstance(roerslev_finaldat, pd.DataFrame):
    roerslev_finaldat.to_csv(os.path.join(nest_folder_def, "roerslev_finaldat.csv"))

In [35]:
pinnock_finaldat = compile_dat('e541326e', 'Ethan-Pinnock', 'Ethan_Pinnock', checkgames = True)
if isinstance(pinnock_finaldat, pd.DataFrame):
    pinnock_finaldat.to_csv(os.path.join(nest_folder_def, "pinnock_finaldat.csv"))

In [36]:
mee_finaldat = compile_dat('8df7a2fb', 'Ben-Mee', 'Ben_Mee', checkgames = True)
if isinstance(mee_finaldat, pd.DataFrame):
    mee_finaldat.to_csv(os.path.join(nest_folder_def, "mee_finaldat.csv"))

In [37]:
rhenry_finaldat = compile_dat('8a3c1dc7', 'Rico-Henry', 'Rico_Henry', checkgames = True)
if isinstance(rhenry_finaldat, pd.DataFrame):
    rhenry_finaldat.to_csv(os.path.join(nest_folder_def, "rhenry_finaldat.csv"))

In [38]:
hickey_finaldat = compile_dat('1780bb4a', 'Aaron-Hickey', 'Aaron_Hickey', checkgames = True)
if isinstance(hickey_finaldat, pd.DataFrame):
    hickey_finaldat.to_csv(os.path.join(nest_folder_def, "hickey_finaldat.csv"))

## Brighton

In [123]:
welbeck_finaldat = compile_dat('ce5143da', 'Danny-Welbeck', 'Danny_Welbeck', checkgames = True)
if isinstance(welbeck_finaldat, pd.DataFrame):
    welbeck_finaldat.to_csv(os.path.join(nest_folder, "welbeck_finaldat.csv"))

In [124]:
jpedro_finaldat = compile_dat('e8832875', 'Joao-Pedro', 'Junqueira', checkgames = True)
if isinstance(jpedro_finaldat, pd.DataFrame):
    jpedro_finaldat.to_csv(os.path.join(nest_folder, "jpedro_finaldat.csv"))

In [125]:
ferguson_finaldat = compile_dat('4596da74', 'Evan-Ferguson', 'Evan_Ferguson', checkgames = True)
if isinstance(ferguson_finaldat, pd.DataFrame):
    ferguson_finaldat.to_csv(os.path.join(nest_folder, "ferguson_finaldat.csv"))

In [126]:
mitoma_finaldat = compile_dat('74618572', 'Kaoru-Mitoma', 'Kaoru_Mitoma', checkgames = True)
if isinstance(mitoma_finaldat, pd.DataFrame):
    mitoma_finaldat.to_csv(os.path.join(nest_folder, "mitoma_finaldat.csv"))

In [127]:
adingra_finaldat = compile_dat('4dcec659', 'Simon-Adingra', 'Simon_Adingra', checkgames = True)
if isinstance(adingra_finaldat, pd.DataFrame):
    adingra_finaldat.to_csv(os.path.join(nest_folder, "adingra_finaldat.csv"))

In [128]:
hinshelwood_finaldat = compile_dat('e98211e7', 'Jack-Hinshelwood', 'Jack_Hinshelwood', checkgames = True)
if isinstance(hinshelwood_finaldat, pd.DataFrame):
    hinshelwood_finaldat.to_csv(os.path.join(nest_folder, "hinshelwood_finaldat.csv"))

In [129]:
milner_finaldat = compile_dat('2f90f6b8', 'James-Milner', 'James_Milner', checkgames = True)
if isinstance(milner_finaldat, pd.DataFrame):
    milner_finaldat.to_csv(os.path.join(nest_folder, "milner_finaldat.csv"))

In [130]:
enciso_finaldat = compile_dat('9cfbad36', 'Julio-Enciso', 'Julio_Enciso', checkgames = True)
if isinstance(enciso_finaldat, pd.DataFrame):
    enciso_finaldat.to_csv(os.path.join(nest_folder, "enciso_finaldat.csv"))

In [140]:
march_latedat = compile_dat('bb5fbd2b', 'Solly-March', 'Solly_March', checkgames = False)
march_earlydat = compile_dat('bb5fbd2b', 'Solly-March', 'Solomon_March', checkgames = False)
march_finaldat = pd.concat([march_latedat, march_earlydat], join = "inner", ignore_index = True)
if isinstance(march_finaldat, pd.DataFrame):
    march_finaldat.to_csv(os.path.join(nest_folder, "march_finaldat.csv"))

Brighton Defenders

In [98]:
veltman_finaldat = compile_dat('dad4b285', 'Joel-Veltman', 'Veltman', checkgames = True)
if isinstance(veltman_finaldat, pd.DataFrame):
    veltman_finaldat.to_csv(os.path.join(nest_folder_def, "veltman_finaldat.csv"))

In [40]:
dunk_finaldat = compile_dat('282f75f3', 'Lewis-Dunk', 'Lewis_Dunk', checkgames = True)
if isinstance(dunk_finaldat, pd.DataFrame):
    dunk_finaldat.to_csv(os.path.join(nest_folder_def, "dunk_finaldat.csv"))

In [41]:
vanhecke_finaldat = compile_dat('4fd08daa', 'Jan-Paul-van-Hecke', 'van Hecke', checkgames = True)
if isinstance(vanhecke_finaldat, pd.DataFrame):
    vanhecke_finaldat.to_csv(os.path.join(nest_folder_def, "vanhecke_finaldat.csv"))

In [42]:
estupinan_finaldat = compile_dat('d38fdf53', 'Pervis-Estupinan', 'Pervis_Estupi', checkgames = True)
if isinstance(estupinan_finaldat, pd.DataFrame):
    estupinan_finaldat.to_csv(os.path.join(nest_folder_def, "estupinan_finaldat.csv"))

In [43]:
webster_finaldat = compile_dat('c40b6180', 'Adam-Webster', 'Adam_Webster', checkgames = True)
if isinstance(webster_finaldat, pd.DataFrame):
    webster_finaldat.to_csv(os.path.join(nest_folder_def, "webster_finaldat.csv"))

In [44]:
igor_finaldat = compile_dat('a4e85758', 'Igor', 'Igor', checkgames = True)
if isinstance(igor_finaldat, pd.DataFrame):
    igor_finaldat.to_csv(os.path.join(nest_folder_def, "igor_finaldat.csv"))

In [45]:
lamptey_finaldat = compile_dat('f4e433d4', 'Tariq-Lamptey', 'Tariq_Lamptey', checkgames = True)
if isinstance(lamptey_finaldat, pd.DataFrame):
    lamptey_finaldat.to_csv(os.path.join(nest_folder_def, "lamptey_finaldat.csv"))

## Chelsea

In [142]:
jackson_finaldat = compile_dat('9c36ed83', 'Nicolas-Jackson', 'Nicolas_Jackson', checkgames = True)
if isinstance(jackson_finaldat, pd.DataFrame):
    jackson_finaldat.to_csv(os.path.join(nest_folder, "jackson_finaldat.csv"))

In [143]:
palmer_finaldat = compile_dat('dc7f8a28', 'Cole-Palmer', 'Cole_Palmer', checkgames = True)
if isinstance(palmer_finaldat, pd.DataFrame):
    palmer_finaldat.to_csv(os.path.join(nest_folder, "palmer_finaldat.csv"))

In [138]:
madueke_finaldat = compile_dat('bf34eebd', 'Noni-Madueke', 'Noni_Madueke', checkgames = True)
if isinstance(madueke_finaldat, pd.DataFrame):
    madueke_finaldat.to_csv(os.path.join(nest_folder, "madueke_finaldat.csv"))

In [139]:
felix_finaldat = compile_dat('8aafd64f', 'Joao-Felix', 'Sequeira', checkgames = True)
if isinstance(felix_finaldat, pd.DataFrame):
    felix_finaldat.to_csv(os.path.join(nest_folder, "felix_finaldat.csv"))

In [141]:
enzo_finaldat = compile_dat('5ff4ab71', 'Enzo-Fernandez', 'Enzo_Fern', checkgames = True)
if isinstance(enzo_finaldat, pd.DataFrame):
    enzo_finaldat.to_csv(os.path.join(nest_folder, "enzo_finaldat.csv"))

In [156]:
caicedo_finaldat = compile_dat('16264a81', 'Moises-Caicedo', 'Caicedo', checkgames = True)
if isinstance(caicedo_finaldat, pd.DataFrame):
    caicedo_finaldat.to_csv(os.path.join(nest_folder, "caicedo_finaldat.csv"))

In [145]:
neto_finaldat = compile_dat('7ba2eaa9', 'Pedro-Neto', 'Pedro_Lomba', checkgames = True)
if isinstance(neto_finaldat, pd.DataFrame):
    neto_finaldat.to_csv(os.path.join(nest_folder, "neto_finaldat.csv"))

In [146]:
dewsburyhall_finaldat = compile_dat('5c74c0f5', 'Kiernan-Dewsbury-Hall', 'Kiernan_Dewsbury', checkgames = True)
if isinstance(dewsburyhall_finaldat, pd.DataFrame):
    dewsburyhall_finaldat.to_csv(os.path.join(nest_folder, "dewsburyhall_finaldat.csv"))

In [162]:
mudryk_finaldat = compile_dat('049a888d', 'Mykhailo-Mudryk', 'Mykhailo_Mudryk', checkgames = False)
if isinstance(mudryk_finaldat, pd.DataFrame):
    mudryk_finaldat.to_csv(os.path.join(nest_folder, "mudryk_finaldat.csv"))

Chelsea Defenders

In [46]:
colwill_finaldat = compile_dat('700783e7', 'Levi-Colwill', 'Levi_Colwill', checkgames = True)
if isinstance(colwill_finaldat, pd.DataFrame):
    colwill_finaldat.to_csv(os.path.join(nest_folder_def, "colwill_finaldat.csv"))

In [47]:
cucurella_finaldat = compile_dat('1daec722', 'Marc-Cucurella', 'Marc_Cucurella', checkgames = True)
if isinstance(cucurella_finaldat, pd.DataFrame):
    cucurella_finaldat.to_csv(os.path.join(nest_folder_def, "cucurella_finaldat.csv"))

In [48]:
gusto_finaldat = compile_dat('d56b9520', 'Malo-Gusto', 'Malo_Gusto', checkgames = True)
if isinstance(gusto_finaldat, pd.DataFrame):
    gusto_finaldat.to_csv(os.path.join(nest_folder_def, "gusto_finaldat.csv"))

In [49]:
wfofana_finaldat = compile_dat('132a82f1', 'Wesley-Fofana', 'Wesley_Fofana', checkgames = True)
if isinstance(wfofana_finaldat, pd.DataFrame):
    wfofana_finaldat.to_csv(os.path.join(nest_folder_def, "wfofana_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [50]:
badiashile_finaldat = compile_dat('06df8256', 'Benoit-Badiashile', 'Badiashile', checkgames = True)
if isinstance(badiashile_finaldat, pd.DataFrame):
    badiashile_finaldat.to_csv(os.path.join(nest_folder_def, "badiashile_finaldat.csv"))

In [86]:
chilwell_finaldat = compile_dat('d2424d1b', 'Ben-Chilwell', 'Chilwell', checkgames = True)
if isinstance(chilwell_finaldat, pd.DataFrame):
    chilwell_finaldat.to_csv(os.path.join(nest_folder_def, "chilwell_finaldat.csv"))

In [52]:
disasi_finaldat = compile_dat('ad82197c', 'Axel-Disasi', 'Disasi', checkgames = True)
if isinstance(disasi_finaldat, pd.DataFrame):
    disasi_finaldat.to_csv(os.path.join(nest_folder_def, "disasi_finaldat.csv"))

In [53]:
rjames_finaldat = compile_dat('1265a93a', 'Reece-James', 'Reece_James', checkgames = True)
if isinstance(rjames_finaldat, pd.DataFrame):
    rjames_finaldat.to_csv(os.path.join(nest_folder_def, "rjames_finaldat.csv"))

In [54]:
tosin_finaldat = compile_dat('c81d773d', 'Tosin-Adarabioyo', 'Tosin_Adarabioyo', checkgames = True)
if isinstance(tosin_finaldat, pd.DataFrame):
    tosin_finaldat.to_csv(os.path.join(nest_folder_def, "tosin_finaldat.csv"))

## Crystal Palace

In [182]:
nketiah_latedat = compile_dat('a53649b7', 'Eddie-Nketiah', 'Eddie_Nketiah', checkgames = False)
nketiah_earlydat = compile_dat('a53649b7', 'Eddie-Nketiah', 'Edward_Nketiah', checkgames = False)
nketiah_finaldat = pd.concat([nketiah_latedat, nketiah_earlydat], join = "inner", ignore_index = True)
if isinstance(nketiah_finaldat, pd.DataFrame):
    nketiah_finaldat.to_csv(os.path.join(nest_folder, "nketiah_finaldat.csv"))

In [149]:
mateta_finaldat = compile_dat('50e6dc35', 'Jean-Philippe-Mateta', 'Jean-Philippe_Mateta', checkgames = True)
if isinstance(mateta_finaldat, pd.DataFrame):
    mateta_finaldat.to_csv(os.path.join(nest_folder, "mateta_finaldat.csv"))

In [150]:
eze_finaldat = compile_dat('ae4fc6a4', 'Eberechi-Eze', 'Eberechi_Eze', checkgames = True)
if isinstance(eze_finaldat, pd.DataFrame):
    eze_finaldat.to_csv(os.path.join(nest_folder, "eze_finaldat.csv"))

In [151]:
wharton_finaldat = compile_dat('4b542852', 'Adam-Wharton', 'Adam_Wharton', checkgames = True)
if isinstance(wharton_finaldat, pd.DataFrame):
    wharton_finaldat.to_csv(os.path.join(nest_folder, "wharton_finaldat.csv"))

In [152]:
cheick_finaldat = compile_dat('ce4f40c7', 'Cheick-Doucoure', 'Cheick_Doucou', checkgames = True)
if isinstance(cheick_finaldat, pd.DataFrame):
    cheick_finaldat.to_csv(os.path.join(nest_folder, "cheick_finaldat.csv"))

In [153]:
lerma_finaldat = compile_dat('9b5ce51a', 'Jefferson-Lerma', 'Jefferson_Lerma', checkgames = True)
if isinstance(lerma_finaldat, pd.DataFrame):
    lerma_finaldat.to_csv(os.path.join(nest_folder, "lerma_finaldat.csv"))

In [183]:
ismailasarr_finaldat = compile_dat('bfdb33aa', 'Ismaila-Sarr', 'Ismaila_Sarr', checkgames = True)
if isinstance(ismailasarr_finaldat, pd.DataFrame):
    ismailasarr_finaldat.to_csv(os.path.join(nest_folder, "ismailasarr_finaldat.csv"))

In [155]:
hughes_finaldat = compile_dat('a0666d3e', 'Will-Hughes', 'Will_Hughes', checkgames = True)
if isinstance(hughes_finaldat, pd.DataFrame):
    hughes_finaldat.to_csv(os.path.join(nest_folder, "hughes_finaldat.csv"))

In [157]:
schlupp_finaldat = compile_dat('3312f911', 'Jeffrey-Schlupp', 'Jeffrey_Schlupp', checkgames = True)
if isinstance(schlupp_finaldat, pd.DataFrame):
    schlupp_finaldat.to_csv(os.path.join(nest_folder, "schlupp_finaldat.csv"))

Crystal Palace Defenders

In [55]:
munoz_finaldat = compile_dat('778ef829', 'Daniel-Munoz', 'Daniel_Mu', checkgames = True)
if isinstance(munoz_finaldat, pd.DataFrame):
    munoz_finaldat.to_csv(os.path.join(nest_folder_def, "munoz_finaldat.csv"))

In [56]:
mitchell_finaldat = compile_dat('5cbd1eb0', 'Tyrick-Mitchell', 'Tyrick_Mitchell', checkgames = True)
if isinstance(mitchell_finaldat, pd.DataFrame):
    mitchell_finaldat.to_csv(os.path.join(nest_folder_def, "mitchell_finaldat.csv"))

In [57]:
crichards_finaldat = compile_dat('0a3d6d2b', 'Chris-Richards', 'Chris_Richards', checkgames = True)
if isinstance(crichards_finaldat, pd.DataFrame):
    crichards_finaldat.to_csv(os.path.join(nest_folder_def, "crichards_finaldat.csv"))

In [58]:
guehi_finaldat = compile_dat('d0706b27', 'Marc-Guehi', 'Marc_Gu', checkgames = True)
if isinstance(guehi_finaldat, pd.DataFrame):
    guehi_finaldat.to_csv(os.path.join(nest_folder_def, "guehi_finaldat.csv"))


In [59]:
clyne_finaldat = compile_dat('0442183b', 'Nathaniel-Clyne', 'Nathaniel_Clyne', checkgames = True)
if isinstance(clyne_finaldat, pd.DataFrame):
    clyne_finaldat.to_csv(os.path.join(nest_folder_def, "clyne_finaldat.csv"))

In [62]:
tchalobah_finaldat = compile_dat('5515376c', 'Trevoh-Chalobah', 'Trevoh_Chalobah', checkgames = True)
if isinstance(tchalobah_finaldat, pd.DataFrame):
    tchalobah_finaldat.to_csv(os.path.join(nest_folder_def, "tchalobah_finaldat.csv"))

In [63]:
holding_finaldat = compile_dat('d79a8c87', 'Rob-Holding', 'Rob_Holding', checkgames = True)
if isinstance(holding_finaldat, pd.DataFrame):
    holding_finaldat.to_csv(os.path.join(nest_folder_def, "holding_finaldat.csv"))

In [64]:
jward_finaldat = compile_dat('8a7ff278', 'Joel-Ward', 'Joel_Ward', checkgames = True)
if isinstance(jward_finaldat, pd.DataFrame):
    jward_finaldat.to_csv(os.path.join(nest_folder_def, "jward_finaldat.csv"))

## Everton

In [158]:
calvertlewin_finaldat = compile_dat('59e6e5bf', 'Dominic-Calvert-Lewin', 'Dominic_Calvert', checkgames = True)
if isinstance(calvertlewin_finaldat, pd.DataFrame):
    calvertlewin_finaldat.to_csv(os.path.join(nest_folder, "calvertlewin_finaldat.csv"))

In [159]:
beto_finaldat = compile_dat('ed5c0319', 'Beto', 'Norberto Bercique', checkgames = True)
if isinstance(beto_finaldat, pd.DataFrame):
    beto_finaldat.to_csv(os.path.join(nest_folder, "beto_finaldat.csv"))

In [160]:
broja_finaldat = compile_dat('97220da2', 'Armando-Broja', 'Armando_Broja', checkgames = True)
if isinstance(broja_finaldat, pd.DataFrame):
    broja_finaldat.to_csv(os.path.join(nest_folder, "broja_finaldat.csv"))

In [184]:
mcneil_finaldat = compile_dat('fc15fb84', 'Dwight-McNeil', 'Dwight_McNeil', checkgames = True)
if isinstance(mcneil_finaldat, pd.DataFrame):
    mcneil_finaldat.to_csv(os.path.join(nest_folder, "mcneil_finaldat.csv"))

In [163]:
gueye_finaldat = compile_dat('72c812f3', 'Idrissa-Gana-Gueye', 'Idrissa_Gueye', checkgames = True)
if isinstance(gueye_finaldat, pd.DataFrame):
    gueye_finaldat.to_csv(os.path.join(nest_folder, "gueye_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [164]:
harrison_finaldat = compile_dat('aa849a12', 'Jack-Harrison', 'Jack_Harrison', checkgames = True)
if isinstance(harrison_finaldat, pd.DataFrame):
    harrison_finaldat.to_csv(os.path.join(nest_folder, "harrison_finaldat.csv"))

In [165]:
doucoure_finaldat = compile_dat('02b29014', 'Abdoulaye-Doucoure', 'Abdoulaye_Douco', checkgames = True)
if isinstance(doucoure_finaldat, pd.DataFrame):
    doucoure_finaldat.to_csv(os.path.join(nest_folder, "doucoure_finaldat.csv"))

In [166]:
garner_finaldat = compile_dat('4e015693', 'James-Garner', 'James_Garner', checkgames = True)
if isinstance(garner_finaldat, pd.DataFrame):
    garner_finaldat.to_csv(os.path.join(nest_folder, "garner_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


Everton Defenders

In [65]:
mkeane_finaldat = compile_dat('c5b7c315', 'Michael-Keane', 'Michael_Keane', checkgames = True)
if isinstance(mkeane_finaldat, pd.DataFrame):
    mkeane_finaldat.to_csv(os.path.join(nest_folder_def, "mkeane_finaldat.csv"))

In [67]:
mykolenko_finaldat = compile_dat('30d4a2e5', 'Vitaliy-Mykolenko', 'Vitalii_Mykolenko', checkgames = False)
if isinstance(mykolenko_finaldat, pd.DataFrame):
    mykolenko_finaldat.to_csv(os.path.join(nest_folder_def, "mykolenko_finaldat.csv"))

In [68]:
coleman_finaldat = compile_dat('0420d84f', 'Seamus-Coleman', 'amus_Coleman', checkgames = True)
if isinstance(coleman_finaldat, pd.DataFrame):
    coleman_finaldat.to_csv(os.path.join(nest_folder_def, "coleman_finaldat.csv"))

In [69]:
tarkowski_finaldat = compile_dat('15ea812b', 'James-Tarkowski', 'James_Tarkowski', checkgames = True)
if isinstance(tarkowski_finaldat, pd.DataFrame):
    tarkowski_finaldat.to_csv(os.path.join(nest_folder_def, "tarkowski_finaldat.csv"))

In [70]:
branthwaite_finaldat = compile_dat('c1949191', 'Jarrad-Branthwaite', 'Jarrad_Branthwaite', checkgames = True)
if isinstance(branthwaite_finaldat, pd.DataFrame):
    branthwaite_finaldat.to_csv(os.path.join(nest_folder_def, "branthwaite_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [71]:
npatterson_finaldat = compile_dat('230f0471', 'Nathan-Patterson', 'Nathan_Patterson', checkgames = True)
if isinstance(npatterson_finaldat, pd.DataFrame):
    npatterson_finaldat.to_csv(os.path.join(nest_folder_def, "npatterson_finaldat.csv"))

In [72]:
ayoung_finaldat = compile_dat('be927d03', 'Ashley-Young', 'Ashley_Young', checkgames = True)
if isinstance(ayoung_finaldat, pd.DataFrame):
    ayoung_finaldat.to_csv(os.path.join(nest_folder_def, "ayoung_finaldat.csv"))

## Fulham

In [167]:
muniz_finaldat = compile_dat('a755db8c', 'Rodrigo-Muniz', 'Rodrigo_Muniz', checkgames = True)
if isinstance(muniz_finaldat, pd.DataFrame):
    muniz_finaldat.to_csv(os.path.join(nest_folder, "muniz_finaldat.csv"))

In [170]:
raul_earlydat = compile_dat('b561db50', 'Raul-Jimenez', 'Raúl_Jiménez', checkgames = False)
raul_latedat = compile_dat('b561db50', 'Raul-Jimenez', 'Jiménez', checkgames = False)
raul_finaldat = pd.concat([raul_latedat, raul_earlydat], join = "inner", ignore_index = True)
if isinstance(raul_finaldat, pd.DataFrame):
    raul_finaldat.to_csv(os.path.join(nest_folder, "raul_finaldat.csv"))

In [169]:
carlosvini_finaldat = compile_dat('8b529245', 'Carlos-Vinicius', 'Carlos Vin', checkgames = True)
if isinstance(carlosvini_finaldat, pd.DataFrame):
    carlosvini_finaldat.to_csv(os.path.join(nest_folder, "carlosvini_finaldat.csv"))

In [181]:
adama_finaldat = compile_dat('9a28eba4', 'Adama-Traore', 'Adama_Traor', checkgames = True)
if isinstance(adama_finaldat, pd.DataFrame):
    adama_finaldat.to_csv(os.path.join(nest_folder, "adama_finaldat.csv"))

In [172]:
smithrowe_finaldat = compile_dat('17695062', 'Emile-Smith-Rowe', 'Emile_Smith', checkgames = True)
if isinstance(smithrowe_finaldat, pd.DataFrame):
    smithrowe_finaldat.to_csv(os.path.join(nest_folder, "smithrowe_finaldat.csv"))

In [173]:
iwobi_finaldat = compile_dat('6ca5ec4b', 'Alex-Iwobi', 'Alex_Iwobi', checkgames = True)
if isinstance(iwobi_finaldat, pd.DataFrame):
    iwobi_finaldat.to_csv(os.path.join(nest_folder, "iwobi_finaldat.csv"))

In [174]:
andreas_earlydat = compile_dat('6639e500', 'Andreas-Pereira', 'Andreas_Pereira', checkgames = False)
andreas_latedat = compile_dat('6639e500', 'Andreas-Pereira', 'Hoelgebaum', checkgames = False)
andreas_finaldat = pd.concat([andreas_latedat, andreas_earlydat], join = "inner", ignore_index = True)
if isinstance(andreas_finaldat, pd.DataFrame):
    andreas_finaldat.to_csv(os.path.join(nest_folder, "andreas_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [175]:
lukic_finaldat = compile_dat('c6e8cf1f', 'Sasa-Lukic', 'Lukic', checkgames = True)
if isinstance(lukic_finaldat, pd.DataFrame):
    lukic_finaldat.to_csv(os.path.join(nest_folder, "lukic_finaldat.csv"))

In [176]:
reed_finaldat = compile_dat('803ae100', 'Harrison-Reed', 'Harrison_Reed', checkgames = True)
if isinstance(reed_finaldat, pd.DataFrame):
    reed_finaldat.to_csv(os.path.join(nest_folder, "reed_finaldat.csv"))

In [177]:
wilson_finaldat = compile_dat('c6dc9ecd', 'Harry-Wilson', 'Harry_Wilson', checkgames = True)
if isinstance(wilson_finaldat, pd.DataFrame):
    wilson_finaldat.to_csv(os.path.join(nest_folder, "wilson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [178]:
nelson_finaldat = compile_dat('c5bdb6e3', 'Reiss-Nelson', 'Reiss_Nelson', checkgames = True)
if isinstance(nelson_finaldat, pd.DataFrame):
    nelson_finaldat.to_csv(os.path.join(nest_folder, "nelson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [185]:
cairney_finaldat = compile_dat('a8748947', 'Tom-Cairney', 'Tom_Cairney', checkgames = True)
if isinstance(cairney_finaldat, pd.DataFrame):
    cairney_finaldat.to_csv(os.path.join(nest_folder, "cairney_finaldat.csv"))

In [186]:
berge_finaldat = compile_dat('d0b6129f', 'Sander-Berge', 'Sander_Berge', checkgames = True)
if isinstance(berge_finaldat, pd.DataFrame):
    berge_finaldat.to_csv(os.path.join(nest_folder, "berge_finaldat.csv"))

Fulham Defenders

In [73]:
arobinson_finaldat = compile_dat('289601e6', 'Antonee-Robinson', 'Antonee_Robinson', checkgames = True)
if isinstance(arobinson_finaldat, pd.DataFrame):
    arobinson_finaldat.to_csv(os.path.join(nest_folder_def, "arobinson_finaldat.csv"))

In [74]:
diop_finaldat = compile_dat('a712ca2b', 'Issa-Diop', 'Issa_Diop', checkgames = True)
if isinstance(diop_finaldat, pd.DataFrame):
    diop_finaldat.to_csv(os.path.join(nest_folder_def, "diop_finaldat.csv"))

In [75]:
tete_finaldat = compile_dat('77d7c96f', 'Kenny-Tete', 'Kenny_Tete', checkgames = True)
if isinstance(tete_finaldat, pd.DataFrame):
    tete_finaldat.to_csv(os.path.join(nest_folder_def, "tete_finaldat.csv"))

In [76]:
bassey_finaldat = compile_dat('a36524bf', 'Calvin-Bassey', 'Calvin_Bassey', checkgames = True)
if isinstance(bassey_finaldat, pd.DataFrame):
    bassey_finaldat.to_csv(os.path.join(nest_folder_def, "bassey_finaldat.csv"))

In [77]:
jandersen_finaldat = compile_dat('e34fc66d', 'Joachim-Andersen', 'Joachim_Andersen', checkgames = True)
if isinstance(jandersen_finaldat, pd.DataFrame):
    jandersen_finaldat.to_csv(os.path.join(nest_folder_def, "jandersen_finaldat.csv"))

In [78]:
castagne_finaldat = compile_dat('197640fd', 'Timothy-Castagne', 'Timothy_Castagne', checkgames = True)
if isinstance(castagne_finaldat, pd.DataFrame):
    castagne_finaldat.to_csv(os.path.join(nest_folder_def, "castagne_finaldat.csv"))

In [79]:
sessegnon_finaldat = compile_dat('6aa3e78b', 'Ryan-Sessegnon', 'Ryan_Sessegnon', checkgames = True)
if isinstance(sessegnon_finaldat, pd.DataFrame):
    sessegnon_finaldat.to_csv(os.path.join(nest_folder_def, "sessegnon_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


## Leicester

In [187]:
vardy_finaldat = compile_dat('45963054', 'Jamie-Vardy', 'Jamie_Vardy', checkgames = True)
if isinstance(vardy_finaldat, pd.DataFrame):
    vardy_finaldat.to_csv(os.path.join(nest_folder, "vardy_finaldat.csv"))

In [188]:
edouard_finaldat = compile_dat('0562b7f1', 'Odsonne-Edouard', 'Odsonne_', checkgames = True)
if isinstance(edouard_finaldat, pd.DataFrame):
    edouard_finaldat.to_csv(os.path.join(nest_folder, "edouard_finaldat.csv"))

In [189]:
daka_finaldat = compile_dat('ca45605e', 'Patson-Daka', 'Patson_Daka', checkgames = True)
if isinstance(daka_finaldat, pd.DataFrame):
    daka_finaldat.to_csv(os.path.join(nest_folder, "daka_finaldat.csv"))

In [190]:
buonanotte_finaldat = compile_dat('468a7a91', 'Facundo-Buonanotte', 'Facundo_B', checkgames = True)
if isinstance(buonanotte_finaldat, pd.DataFrame):
    buonanotte_finaldat.to_csv(os.path.join(nest_folder, "buonanotte_finaldat.csv"))

In [194]:
ndidi_finaldat = compile_dat('6b47c5db', 'Wilfred-Ndidi', 'Wilfred_Ndidi', checkgames = True)
if isinstance(ndidi_finaldat, pd.DataFrame):
    ndidi_finaldat.to_csv(os.path.join(nest_folder, "ndidi_finaldat.csv"))

In [195]:
decordovareid_earlydat = compile_dat('0f7533cd', 'Bobby-Reid', 'Bobby_Reid', checkgames = False)
decordovareid_latedat = compile_dat('0f7533cd', 'Bobby-Reid', 'Bobby_De', checkgames = False)
decordovareid_finaldat = pd.concat([decordovareid_latedat, decordovareid_earlydat], join = "inner", ignore_index = True)
if isinstance(decordovareid_finaldat, pd.DataFrame):
    decordovareid_finaldat.to_csv(os.path.join(nest_folder, "decordovareid_finaldat.csv"))

In [196]:
winks_finaldat = compile_dat('2f7acede', 'Harry-Winks', 'Harry_Winks', checkgames = True)
if isinstance(winks_finaldat, pd.DataFrame):
    winks_finaldat.to_csv(os.path.join(nest_folder, "winks_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [197]:
jayew_finaldat = compile_dat('da052c14', 'Jordan-Ayew', 'Jordan_Ayew', checkgames = True)
if isinstance(jayew_finaldat, pd.DataFrame):
    jayew_finaldat.to_csv(os.path.join(nest_folder, "jayew_finaldat.csv"))

In [198]:
skipp_finaldat = compile_dat('6250083a', 'Oliver-Skipp', 'Oliver_Skipp', checkgames = True)
if isinstance(skipp_finaldat, pd.DataFrame):
    skipp_finaldat.to_csv(os.path.join(nest_folder, "skipp_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [199]:
soumare_finaldat = compile_dat('ae59b359', 'Boubakary-Soumare', 'Boubakary_Sou', checkgames = True)
if isinstance(soumare_finaldat, pd.DataFrame):
    soumare_finaldat.to_csv(os.path.join(nest_folder, "soumare_finaldat.csv"))

In [200]:
choudhury_finaldat = compile_dat('7d2d3329', 'Hamza-Choudhury', 'Hamza_Choudhury', checkgames = True)
if isinstance(choudhury_finaldat, pd.DataFrame):
    choudhury_finaldat.to_csv(os.path.join(nest_folder, "choudhury_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


Leicester Defenders

In [80]:
jjustin_finaldat = compile_dat('fb614c44', 'James-Justin', 'James_Justin', checkgames = True)
if isinstance(jjustin_finaldat, pd.DataFrame):
    jjustin_finaldat.to_csv(os.path.join(nest_folder_def, "jjustin_finaldat.csv"))

In [81]:
kristiansen_finaldat = compile_dat('505486d6', 'Victor-Bernth-Kristiansen', 'Victor_Kristiansen', checkgames = True)
if isinstance(kristiansen_finaldat, pd.DataFrame):
    kristiansen_finaldat.to_csv(os.path.join(nest_folder_def, "kristiansen_finaldat.csv"))

In [82]:
vestergaard_finaldat = compile_dat('1900032e', 'Jannik-Vestergaard', 'Jannik_Vestergaard', checkgames = True)
if isinstance(vestergaard_finaldat, pd.DataFrame):
    vestergaard_finaldat.to_csv(os.path.join(nest_folder_def, "vestergaard_finaldat.csv"))

In [83]:
coady_finaldat = compile_dat('2928dca2', 'Conor-Coady', 'Conor_Coady', checkgames = True)
if isinstance(coady_finaldat, pd.DataFrame):
    coady_finaldat.to_csv(os.path.join(nest_folder_def, "coady_finaldat.csv"))

In [84]:
ricardop_finaldat = compile_dat('75a72a99', 'Ricardo-Pereira', 'Barbosa Pereira', checkgames = True)
if isinstance(ricardop_finaldat, pd.DataFrame):
    ricardop_finaldat.to_csv(os.path.join(nest_folder_def, "ricardop_finaldat.csv"))

In [85]:
lthomas_finaldat = compile_dat('fc027d02', 'Luke-Thomas', 'Luke_Thomas', checkgames = True)
if isinstance(lthomas_finaldat, pd.DataFrame):
    lthomas_finaldat.to_csv(os.path.join(nest_folder_def, "lthomas_finaldat.csv"))

## Liverpool

In [201]:
szoboszlai_finaldat = compile_dat('934e1968', 'Dominik-Szoboszlai', 'Dominik_Szoboszlai', checkgames = True)
if isinstance(szoboszlai_finaldat, pd.DataFrame):
    szoboszlai_finaldat.to_csv(os.path.join(nest_folder, "szoboszlai_finaldat.csv"))

In [202]:
macallister_finaldat = compile_dat('83d074ff', 'Alexis-Mac-Allister', 'Alexis_Mac', checkgames = True)
if isinstance(macallister_finaldat, pd.DataFrame):
    macallister_finaldat.to_csv(os.path.join(nest_folder, "macallister_finaldat.csv"))

In [203]:
gravenberch_finaldat = compile_dat('b8e740fb', 'Ryan-Gravenberch', 'Ryan_Gravenberch', checkgames = True)
if isinstance(gravenberch_finaldat, pd.DataFrame):
    gravenberch_finaldat.to_csv(os.path.join(nest_folder, "gravenberch_finaldat.csv"))

In [204]:
elliott_finaldat = compile_dat('b9e1436c', 'Harvey-Elliott', 'Harvey_Elliott', checkgames = True)
if isinstance(elliott_finaldat, pd.DataFrame):
    elliott_finaldat.to_csv(os.path.join(nest_folder, "elliott_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [205]:
curtis_finaldat = compile_dat('4fb9c88f', 'Curtis-Jones', 'Curtis_Jones', checkgames = True)
if isinstance(curtis_finaldat, pd.DataFrame):
    curtis_finaldat.to_csv(os.path.join(nest_folder, "curtis_finaldat.csv"))

In [206]:
endo_finaldat = compile_dat('c149016b', 'Wataru-Endo', 'Wataru_Endo', checkgames = True)
if isinstance(endo_finaldat, pd.DataFrame):
    endo_finaldat.to_csv(os.path.join(nest_folder, "endo_finaldat.csv"))

Liverpool Defenders

In [88]:
taa_finaldat = compile_dat('cd1acf9d', 'Trent-Alexander-Arnold', 'Trent_Alexander', checkgames = True)
if isinstance(taa_finaldat, pd.DataFrame):
    taa_finaldat.to_csv(os.path.join(nest_folder_def, "taa_finaldat.csv"))

In [89]:
robertson_finaldat = compile_dat('2e4f5f03', 'Andrew-Robertson', 'Andrew_Robertson', checkgames = True)
if isinstance(robertson_finaldat, pd.DataFrame):
    robertson_finaldat.to_csv(os.path.join(nest_folder_def, "robertson_finaldat.csv"))

In [90]:
virgil_finaldat = compile_dat('e06683ca', 'Virgil-van-Dijk', 'Virgil_van', checkgames = True)
if isinstance(virgil_finaldat, pd.DataFrame):
    virgil_finaldat.to_csv(os.path.join(nest_folder_def, "virgil_finaldat.csv"))

In [91]:
konate_finaldat = compile_dat('5ed9b537', 'Ibrahima-Konate', 'Ibrahima_Kona', checkgames = True)
if isinstance(konate_finaldat, pd.DataFrame):
    konate_finaldat.to_csv(os.path.join(nest_folder_def, "konate_finaldat.csv"))

In [92]:
cbradley_finaldat = compile_dat('bbd67769', 'Conor-Bradley', 'Conor_Bradley', checkgames = True)
if isinstance(cbradley_finaldat, pd.DataFrame):
    cbradley_finaldat.to_csv(os.path.join(nest_folder_def, "cbradley_finaldat.csv"))

In [93]:
tsimikas_finaldat = compile_dat('f315ca93', 'Kostas-Tsimikas', 'Tsimikas', checkgames = True)
if isinstance(tsimikas_finaldat, pd.DataFrame):
    tsimikas_finaldat.to_csv(os.path.join(nest_folder_def, "tsimikas_finaldat.csv"))

In [94]:
quansah_finaldat = compile_dat('4125cb98', 'Jarell-Quansah', 'Jarell_Quansah', checkgames = True)
if isinstance(quansah_finaldat, pd.DataFrame):
    quansah_finaldat.to_csv(os.path.join(nest_folder_def, "quansah_finaldat.csv"))

In [95]:
gomez_finaldat = compile_dat('7a11550b', 'Joe-Gomez', 'Joseph_Gomez', checkgames = True)
if isinstance(gomez_finaldat, pd.DataFrame):
    gomez_finaldat.to_csv(os.path.join(nest_folder_def, "gomez_finaldat.csv"))

## Manchester City

In [207]:
haaland_finaldat = compile_dat('1f44ac21', 'Erling-Haaland', 'Erling_Haaland', checkgames = True)
if isinstance(haaland_finaldat, pd.DataFrame):
    haaland_finaldat.to_csv(os.path.join(nest_folder, "haaland_finaldat.csv"))

In [208]:
debruyne_finaldat = compile_dat('e46012d4', 'Kevin-De-Bruyne', 'Kevin_De Bruyne', checkgames = True)
if isinstance(debruyne_finaldat, pd.DataFrame):
    debruyne_finaldat.to_csv(os.path.join(nest_folder, "debruyne_finaldat.csv"))

In [209]:
bernardo_finaldat = compile_dat('3eb22ec9', 'Bernardo-Silva', 'Veiga de Carvalho', checkgames = True)
if isinstance(bernardo_finaldat, pd.DataFrame):
    bernardo_finaldat.to_csv(os.path.join(nest_folder, "bernardo_finaldat.csv"))

In [210]:
kovacic_finaldat = compile_dat('79c0821a', 'Mateo-Kovacic', 'Mateo_Kova', checkgames = True)
if isinstance(kovacic_finaldat, pd.DataFrame):
    kovacic_finaldat.to_csv(os.path.join(nest_folder, "kovacic_finaldat.csv"))

In [212]:
doku_finaldat = compile_dat('fffea3e5', 'Jeremy-Doku', 'Doku', checkgames = True)
if isinstance(doku_finaldat, pd.DataFrame):
    doku_finaldat.to_csv(os.path.join(nest_folder, "doku_finaldat.csv"))

In [247]:
matheusn_finaldat = compile_dat('e6af02e0', 'Matheus-Nunes', 'Matheus Luiz_Nunes', checkgames = False)
if isinstance(matheusn_finaldat, pd.DataFrame):
    matheusn_finaldat.to_csv(os.path.join(nest_folder, "matheusn_finaldat.csv"))

#checkgames false because there is an extra row, for a game that Matheus Nunes technically did not play against Leeds. However, that row exists 
#because he managed to pickup a red card despite not playing.

In [220]:
grealish_finaldat = compile_dat('b0b4fd3e', 'Jack-Grealish', 'Jack_Grealish', checkgames = True)
if isinstance(grealish_finaldat, pd.DataFrame):
    grealish_finaldat.to_csv(os.path.join(nest_folder, "grealish_finaldat.csv"))

In [221]:
gundogan_finaldat = compile_dat('819b3158', 'Ilkay-Gundogan', 'Ilkay_G', checkgames = True)
if isinstance(gundogan_finaldat, pd.DataFrame):
    gundogan_finaldat.to_csv(os.path.join(nest_folder, "gundogan_finaldat.csv"))

In [248]:
foden_finaldat = compile_dat('ed1e53f3', 'Phil-Foden', 'Phil_Foden', checkgames = True)
if isinstance(foden_finaldat, pd.DataFrame):
    foden_finaldat.to_csv(os.path.join(nest_folder, "foden_finaldat.csv"))

In [223]:
bobb_finaldat = compile_dat('eed2427e', 'Oscar-Bobb', 'Oscar_Bobb', checkgames = True)
if isinstance(bobb_finaldat, pd.DataFrame):
    bobb_finaldat.to_csv(os.path.join(nest_folder, "bobb_finaldat.csv"))

In [224]:
rodri_finaldat = compile_dat('6434f10d', 'Rodri', 'Rodrigo_Hernandez', checkgames = True)
if isinstance(rodri_finaldat, pd.DataFrame):
    rodri_finaldat.to_csv(os.path.join(nest_folder, "rodri_finaldat.csv"))

Manchester City Defenders

In [96]:
rlewis_finaldat = compile_dat('b57e066e', 'Rico-Lewis', 'Rico_Lewis', checkgames = True)
if isinstance(rlewis_finaldat, pd.DataFrame):
    rlewis_finaldat.to_csv(os.path.join(nest_folder_def, "rlewis_finaldat.csv"))

In [97]:
gvardiol_finaldat = compile_dat('5ad50391', 'Josko-Gvardiol', 'Gvardiol', checkgames = True)
if isinstance(gvardiol_finaldat, pd.DataFrame):
    gvardiol_finaldat.to_csv(os.path.join(nest_folder_def, "gvardiol_finaldat.csv"))

In [99]:
akanji_finaldat = compile_dat('89ac64a6', 'Manuel-Akanji', 'Manuel_Akanji', checkgames = True)
if isinstance(akanji_finaldat, pd.DataFrame):
    akanji_finaldat.to_csv(os.path.join(nest_folder_def, "akanji_finaldat.csv"))

In [101]:
ruben_finaldat = compile_dat('31c69ef1', 'Ruben-Dias', 'Gato Alves Dias', checkgames = True)
if isinstance(ruben_finaldat, pd.DataFrame):
    ruben_finaldat.to_csv(os.path.join(nest_folder_def, "ruben_finaldat.csv"))

In [102]:
ake_finaldat = compile_dat('eaeca114', 'Nathan-Ake', 'Nathan_Ak', checkgames = True)
if isinstance(ake_finaldat, pd.DataFrame):
    ake_finaldat.to_csv(os.path.join(nest_folder_def, "ake_finaldat.csv"))

In [103]:
stones_finaldat = compile_dat('5eecec3d', 'John-Stones', 'John_Stones', checkgames = True)
if isinstance(stones_finaldat, pd.DataFrame):
    stones_finaldat.to_csv(os.path.join(nest_folder_def, "stones_finaldat.csv"))

In [111]:
kwalker_finaldat = compile_dat('86dd77d1', 'Kyle-Walker', 'Kyle_Walker_', checkgames = True)
if isinstance(kwalker_finaldat, pd.DataFrame):
    kwalker_finaldat.to_csv(os.path.join(nest_folder_def, "kwalker_finaldat.csv"))

## Manchester United

In [225]:
hojlund_finaldat = compile_dat('491a433d', 'Rasmus-Hojlund', 'Rasmus_H', checkgames = True)
if isinstance(hojlund_finaldat, pd.DataFrame):
    hojlund_finaldat.to_csv(os.path.join(nest_folder, "hojlund_finaldat.csv"))

In [226]:
brunof_finaldat = compile_dat('507c7bdf', 'Bruno-Fernandes', 'Borges Fernandes', checkgames = True)
if isinstance(brunof_finaldat, pd.DataFrame):
    brunof_finaldat.to_csv(os.path.join(nest_folder, "brunof_finaldat.csv"))

In [227]:
garnacho_finaldat = compile_dat('7aa8adfe', 'Alejandro-Garnacho', 'Alejandro_Garnacho', checkgames = True)
if isinstance(garnacho_finaldat, pd.DataFrame):
    garnacho_finaldat.to_csv(os.path.join(nest_folder, "garnacho_finaldat.csv"))

In [228]:
rashford_finaldat = compile_dat('a1d5bd30', 'Marcus-Rashford', 'Marcus_Rashford', checkgames = True)
if isinstance(rashford_finaldat, pd.DataFrame):
    rashford_finaldat.to_csv(os.path.join(nest_folder, "rashford_finaldat.csv"))

In [229]:
casemiro_finaldat = compile_dat('4d224fe8', 'Casemiro', 'Casimiro', checkgames = True)
if isinstance(casemiro_finaldat, pd.DataFrame):
    casemiro_finaldat.to_csv(os.path.join(nest_folder, "casemiro_finaldat.csv"))

In [230]:
mainoo_finaldat = compile_dat('c6220452', 'Kobbie-Mainoo', 'Kobbie_Mainoo', checkgames = True)
if isinstance(mainoo_finaldat, pd.DataFrame):
    mainoo_finaldat.to_csv(os.path.join(nest_folder, "mainoo_finaldat.csv"))

In [231]:
mount_finaldat = compile_dat('9674002f', 'Mason-Mount', 'Mason_Mount', checkgames = True)
if isinstance(mount_finaldat, pd.DataFrame):
    mount_finaldat.to_csv(os.path.join(nest_folder, "mount_finaldat.csv"))

In [232]:
antony_finaldat = compile_dat('99127249', 'Antony', 'Antony', checkgames = True)
if isinstance(antony_finaldat, pd.DataFrame):
    antony_finaldat.to_csv(os.path.join(nest_folder, "antony_finaldat.csv"))

In [233]:
eriksen_finaldat = compile_dat('980522ec', 'Christian-Eriksen', 'Christian_Eriksen', checkgames = True)
if isinstance(eriksen_finaldat, pd.DataFrame):
    eriksen_finaldat.to_csv(os.path.join(nest_folder, "eriksen_finaldat.csv"))

Manchester United Defenders

In [105]:
dalot_finaldat = compile_dat('d9565625', 'Diogo-Dalot', 'Diogo_Dalot', checkgames = True)
if isinstance(dalot_finaldat, pd.DataFrame):
    dalot_finaldat.to_csv(os.path.join(nest_folder_def, "dalot_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [106]:
hmaguire_finaldat = compile_dat('d8931174', 'Harry-Maguire', 'Harry_Maguire', checkgames = True)
if isinstance(hmaguire_finaldat, pd.DataFrame):
    hmaguire_finaldat.to_csv(os.path.join(nest_folder_def, "hmaguire_finaldat.csv"))

In [107]:
lisandro_finaldat = compile_dat('bac46a10', 'Lisandro-Martinez', 'Lisandro_Mart', checkgames = True)
if isinstance(lisandro_finaldat, pd.DataFrame):
    lisandro_finaldat.to_csv(os.path.join(nest_folder_def, "lisandro_finaldat.csv"))

In [108]:
jevans_finaldat = compile_dat('f8fcd2a5', 'Jonny-Evans', 'Jonny_Evans', checkgames = True)
if isinstance(jevans_finaldat, pd.DataFrame):
    jevans_finaldat.to_csv(os.path.join(nest_folder_def, "jevans_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


The number of Premier League games played by Jonny-Evans(172) does not match the number of rows in the dataframe (144)


In [109]:
lindelof_finaldat = compile_dat('f5deef4c', 'Victor-Lindelof', 'Victor_Lindel', checkgames = True)
if isinstance(lindelof_finaldat, pd.DataFrame):
    lindelof_finaldat.to_csv(os.path.join(nest_folder_def, "lindelof_finaldat.csv"))

In [110]:
shaw_finaldat = compile_dat('9c94165b', 'Luke-Shaw', 'Luke_Shaw', checkgames = True)
if isinstance(shaw_finaldat, pd.DataFrame):
    shaw_finaldat.to_csv(os.path.join(nest_folder_def, "shaw_finaldat.csv"))

## Newcastle

In [234]:
isak_finaldat = compile_dat('8e92be30', 'Alexander-Isak', 'Alexander_Isak', checkgames = True)
if isinstance(isak_finaldat, pd.DataFrame):
    isak_finaldat.to_csv(os.path.join(nest_folder, "isak_finaldat.csv"))

In [235]:
cwilson_finaldat = compile_dat('c596fcb0', 'Callum-Wilson', 'Callum_Wilson', checkgames = True)
if isinstance(cwilson_finaldat, pd.DataFrame):
    cwilson_finaldat.to_csv(os.path.join(nest_folder, "cwilson_finaldat.csv"))

In [236]:
hbarnes_finaldat = compile_dat('3ea50f67', 'Harvey-Barnes', 'Harvey_Barnes', checkgames = True)
if isinstance(hbarnes_finaldat, pd.DataFrame):
    hbarnes_finaldat.to_csv(os.path.join(nest_folder, "hbarnes_finaldat.csv"))

In [237]:
gordon_finaldat = compile_dat('2bd83368', 'Anthony-Gordon', 'Anthony_Gordon', checkgames = True)
if isinstance(gordon_finaldat, pd.DataFrame):
    gordon_finaldat.to_csv(os.path.join(nest_folder, "gordon_finaldat.csv"))

In [238]:
joelinton_finaldat = compile_dat('c17bfb65', 'Joelinton', 'Joelinton', checkgames = True)
if isinstance(joelinton_finaldat, pd.DataFrame):
    joelinton_finaldat.to_csv(os.path.join(nest_folder, "joelinton_finaldat.csv"))

In [239]:
brunog_finaldat = compile_dat('82518f62', 'Bruno-Guimaraes', 'Bruno_Gui', checkgames = True)
if isinstance(brunog_finaldat, pd.DataFrame):
    brunog_finaldat.to_csv(os.path.join(nest_folder, "brunog_finaldat.csv"))

In [240]:
jmurphy_finaldat = compile_dat('de112b84', 'Jacob-Murphy', 'Jacob_Murphy', checkgames = True)
if isinstance(jmurphy_finaldat, pd.DataFrame):
    jmurphy_finaldat.to_csv(os.path.join(nest_folder, "jmurphy_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [241]:
slongstaff_finaldat = compile_dat('a2b105e0', 'Sean-Longstaff', 'Sean_Longstaff', checkgames = True)
if isinstance(slongstaff_finaldat, pd.DataFrame):
    slongstaff_finaldat.to_csv(os.path.join(nest_folder, "slongstaff_finaldat.csv"))

In [242]:
almiron_finaldat = compile_dat('862a1c15', 'Miguel-Almiron', 'Miguel_Almir', checkgames = True)
if isinstance(almiron_finaldat, pd.DataFrame):
    almiron_finaldat.to_csv(os.path.join(nest_folder, "almiron_finaldat.csv"))

In [243]:
willock_finaldat = compile_dat('a3b03921', 'Joe-Willock', 'Willock', checkgames = True)
if isinstance(willock_finaldat, pd.DataFrame):
    willock_finaldat.to_csv(os.path.join(nest_folder, "willock_finaldat.csv"))

In [245]:
hayden_finaldat = compile_dat('c951a6df', 'Isaac-Hayden', 'Isaac_Hayden', checkgames = True)
if isinstance(hayden_finaldat, pd.DataFrame):
    hayden_finaldat.to_csv(os.path.join(nest_folder, "hayden_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [246]:
miley_finaldat = compile_dat('2c6835e5', 'Lewis-Miley', 'Lewis_Miley', checkgames = True)
if isinstance(miley_finaldat, pd.DataFrame):
    miley_finaldat.to_csv(os.path.join(nest_folder, "miley_finaldat.csv"))

Newcastle Defenders

In [112]:
krafth_finaldat = compile_dat('77cf6852', 'Emil-Krafth', 'Emil_Krafth', checkgames = True)
if isinstance(krafth_finaldat, pd.DataFrame):
    krafth_finaldat.to_csv(os.path.join(nest_folder_def, "krafth_finaldat.csv"))

In [113]:
livramento_finaldat = compile_dat('afed6722', 'Valentino-Livramento', 'Livramento', checkgames = True)
if isinstance(livramento_finaldat, pd.DataFrame):
    livramento_finaldat.to_csv(os.path.join(nest_folder_def, "livramento_finaldat.csv"))

In [114]:
lhall_finaldat = compile_dat('da011f18', 'Lewis-Hall', 'Lewis_Hall', checkgames = True)
if isinstance(lhall_finaldat, pd.DataFrame):
    lhall_finaldat.to_csv(os.path.join(nest_folder_def, "lhall_finaldat.csv"))

In [115]:
lkelly_finaldat = compile_dat('f31def1e', 'Lloyd-Kelly', 'Lloyd_Kelly', checkgames = True)
if isinstance(lkelly_finaldat, pd.DataFrame):
    lkelly_finaldat.to_csv(os.path.join(nest_folder_def, "lkelly_finaldat.csv"))

In [116]:
burn_finaldat = compile_dat('b2d31e83', 'Dan-Burn', 'Dan_Burn', checkgames = True)
if isinstance(burn_finaldat, pd.DataFrame):
    burn_finaldat.to_csv(os.path.join(nest_folder_def, "burn_finaldat.csv"))

In [117]:
trippier_finaldat = compile_dat('21512407', 'Kieran-Trippier', 'Kieran_Trippier', checkgames = True)
if isinstance(trippier_finaldat, pd.DataFrame):
    trippier_finaldat.to_csv(os.path.join(nest_folder_def, "trippier_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [118]:
botman_finaldat = compile_dat('ccce7025', 'Sven-Botman', 'Sven_Botman', checkgames = True)
if isinstance(botman_finaldat, pd.DataFrame):
    botman_finaldat.to_csv(os.path.join(nest_folder_def, "botman_finaldat.csv"))

In [145]:
lascelles_finaldat = compile_dat('450ab6fc', 'Jamaal-Lascelles', 'Jamaal_Lascelles', checkgames = False)
if isinstance(lascelles_finaldat, pd.DataFrame):
    lascelles_finaldat.to_csv(os.path.join(nest_folder_def, "lascelles_finaldat.csv"))
    
#checkgames false because of the matheus nunes problem 

In [120]:
targett_finaldat = compile_dat('e514ab62', 'Matt-Targett', 'Matt_Targett', checkgames = True)
if isinstance(targett_finaldat, pd.DataFrame):
    targett_finaldat.to_csv(os.path.join(nest_folder_def, "targett_finaldat.csv"))

In [121]:
schar_finaldat = compile_dat('c8aa95da', 'Fabian-Schar', 'Fabian_Sch', checkgames = True)
if isinstance(schar_finaldat, pd.DataFrame):
    schar_finaldat.to_csv(os.path.join(nest_folder_def, "schar_finaldat.csv"))

## Nottingham Forest

In [249]:
wood_finaldat = compile_dat('4e9a0555', 'Chris-Wood', 'Chris_Wood', checkgames = True)
if isinstance(wood_finaldat, pd.DataFrame):
    wood_finaldat.to_csv(os.path.join(nest_folder, "wood_finaldat.csv"))

In [250]:
awoniyi_finaldat = compile_dat('e5478b87', 'Taiwo-Awoniyi', 'Taiwo_Awoniyi', checkgames = True)
if isinstance(awoniyi_finaldat, pd.DataFrame):
    awoniyi_finaldat.to_csv(os.path.join(nest_folder, "awoniyi_finaldat.csv"))

In [251]:
dennis_finaldat = compile_dat('2c44a35d', 'Emmanuel-Dennis', 'Emmanuel_Dennis', checkgames = True)
if isinstance(dennis_finaldat, pd.DataFrame):
    dennis_finaldat.to_csv(os.path.join(nest_folder, "dennis_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [252]:
gibbswhite_finaldat = compile_dat('32f60ed8', 'Morgan-Gibbs-White', 'Morgan_Gibbs', checkgames = True)
if isinstance(gibbswhite_finaldat, pd.DataFrame):
    gibbswhite_finaldat.to_csv(os.path.join(nest_folder, "gibbswhite_finaldat.csv"))

In [253]:
anderson_finaldat = compile_dat('de31038e', 'Elliot-Anderson', 'Elliot_Anderson', checkgames = True)
if isinstance(anderson_finaldat, pd.DataFrame):
    anderson_finaldat.to_csv(os.path.join(nest_folder, "anderson_finaldat.csv"))

In [254]:
yates_finaldat = compile_dat('c1fe2a48', 'Ryan-Yates', 'Ryan_Yates', checkgames = True)
if isinstance(yates_finaldat, pd.DataFrame):
    yates_finaldat.to_csv(os.path.join(nest_folder, "yates_finaldat.csv"))

In [255]:
hudsonodoi_finaldat = compile_dat('15f3ec41', 'Callum-Hudson-Odoi', 'Callum_Hudson', checkgames = True)
if isinstance(hudsonodoi_finaldat, pd.DataFrame):
    hudsonodoi_finaldat.to_csv(os.path.join(nest_folder, "hudsonodoi_finaldat.csv"))

In [258]:
elanga_finaldat = compile_dat('2fba6108', 'Anthony-Elanga', 'Anthony_Elanga', checkgames = True)
if isinstance(elanga_finaldat, pd.DataFrame):
    elanga_finaldat.to_csv(os.path.join(nest_folder, "elanga_finaldat.csv"))

In [259]:
sangare_finaldat = compile_dat('bced0375', 'Ibrahim-Sangare', 'Ibrahim_Sangar', checkgames = True)
if isinstance(sangare_finaldat, pd.DataFrame):
    sangare_finaldat.to_csv(os.path.join(nest_folder, "sangare_finaldat.csv"))

In [260]:
danilo_finaldat = compile_dat('a816dbfb', 'Danilo', 'Danilo_dos Santos', checkgames = True)
if isinstance(danilo_finaldat, pd.DataFrame):
    danilo_finaldat.to_csv(os.path.join(nest_folder, "danilo_finaldat.csv"))

In [261]:
dominguez_finaldat = compile_dat('20b3a502', 'Nicolas-Dominguez', 'Domínguez', checkgames = True)
if isinstance(dominguez_finaldat, pd.DataFrame):
    dominguez_finaldat.to_csv(os.path.join(nest_folder, "dominguez_finaldat.csv"))

In [262]:
wardprowse_finaldat = compile_dat('3515d404', 'James-Ward-Prowse', 'James_Ward', checkgames = True)
if isinstance(wardprowse_finaldat, pd.DataFrame):
    wardprowse_finaldat.to_csv(os.path.join(nest_folder, "wardprowse_finaldat.csv"))

Nottingham Forest Defenders

In [144]:
aina_finaldat = compile_dat('246d153b', 'Ola-Aina', 'Aina', checkgames = True)
if isinstance(aina_finaldat, pd.DataFrame):
    aina_finaldat.to_csv(os.path.join(nest_folder_def, "aina_finaldat.csv"))

In [123]:
murillo_finaldat = compile_dat('1704b0b8', 'Murillo', 'Murillo', checkgames = True)
if isinstance(murillo_finaldat, pd.DataFrame):
    murillo_finaldat.to_csv(os.path.join(nest_folder_def, "murillo_finaldat.csv"))

In [124]:
nwilliams_finaldat = compile_dat('dd323728', 'Neco-Williams', 'Neco_Williams', checkgames = True)
if isinstance(nwilliams_finaldat, pd.DataFrame):
    nwilliams_finaldat.to_csv(os.path.join(nest_folder_def, "nwilliams_finaldat.csv"))

In [125]:
boly_finaldat = compile_dat('a4ac4b8f', 'Willy-Boly', 'Willy_Boly', checkgames = True)
if isinstance(boly_finaldat, pd.DataFrame):
    boly_finaldat.to_csv(os.path.join(nest_folder_def, "boly_finaldat.csv"))

In [126]:
toffolo_finaldat = compile_dat('a105d46a', 'Harry-Toffolo', 'Harry_Toffolo', checkgames = True)
if isinstance(toffolo_finaldat, pd.DataFrame):
    toffolo_finaldat.to_csv(os.path.join(nest_folder_def, "toffolo_finaldat.csv"))

In [127]:
amoreno_finaldat = compile_dat('16e9d0ea', 'Alex-Moreno', 'Alexandre_Moreno', checkgames = True)
if isinstance(amoreno_finaldat, pd.DataFrame):
    amoreno_finaldat.to_csv(os.path.join(nest_folder_def, "amoreno_finaldat.csv"))

In [128]:
omobamidele_finaldat = compile_dat('c393a6c4', 'Andrew-Omobamidele', 'Andrew_Omobamidele', checkgames = True)
if isinstance(omobamidele_finaldat, pd.DataFrame):
    omobamidele_finaldat.to_csv(os.path.join(nest_folder_def, "omobamidele_finaldat.csv"))

## Southampton

In [263]:
aarmstrong_finaldat = compile_dat('68c720b5', 'Adam-Armstrong', 'Adam_Armstrong', checkgames = True)
if isinstance(aarmstrong_finaldat, pd.DataFrame):
    aarmstrong_finaldat.to_csv(os.path.join(nest_folder, "aarmstrong_finaldat.csv"))

In [264]:
archer_finaldat = compile_dat('05e8ca6d', 'Cameron-Archer', 'Cameron_Archer', checkgames = True)
if isinstance(archer_finaldat, pd.DataFrame):
    archer_finaldat.to_csv(os.path.join(nest_folder, "archer_finaldat.csv"))

In [267]:
aribo_finaldat = compile_dat('328f7d51', 'Joe-Aribo', 'Aribo', checkgames = True)
if isinstance(aribo_finaldat, pd.DataFrame):
    aribo_finaldat.to_csv(os.path.join(nest_folder, "aribo_finaldat.csv"))

In [268]:
smallbone_finaldat = compile_dat('5e105217', 'William-Smallbone', 'William_Smallbone', checkgames = True)
if isinstance(smallbone_finaldat, pd.DataFrame):
    smallbone_finaldat.to_csv(os.path.join(nest_folder, "smallbone_finaldat.csv"))

In [269]:
downes_finaldat = compile_dat('69fdb896', 'Flynn-Downes', 'Flynn_Downes', checkgames = True)
if isinstance(downes_finaldat, pd.DataFrame):
    downes_finaldat.to_csv(os.path.join(nest_folder, "downes_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [270]:
brereton_finaldat = compile_dat('57827369', 'Ben-Brereton', 'Ben_Brereton', checkgames = True)
if isinstance(brereton_finaldat, pd.DataFrame):
    brereton_finaldat.to_csv(os.path.join(nest_folder, "brereton_finaldat.csv"))

In [271]:
lallana_finaldat = compile_dat('99813635', 'Adam-Lallana', 'Adam_Lallana', checkgames = True)
if isinstance(lallana_finaldat, pd.DataFrame):
    lallana_finaldat.to_csv(os.path.join(nest_folder, "lallana_finaldat.csv"))

In [272]:
rfraser_finaldat = compile_dat('d56543a0', 'Ryan-Fraser', 'Ryan_Fraser', checkgames = True)
if isinstance(rfraser_finaldat, pd.DataFrame):
    rfraser_finaldat.to_csv(os.path.join(nest_folder, "rfraser_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [273]:
sulemana_finaldat = compile_dat('a62f8bf1', 'Kamaldeen-Sulemana', 'Kamaldeen_Sulemana', checkgames = True)
if isinstance(sulemana_finaldat, pd.DataFrame):
    sulemana_finaldat.to_csv(os.path.join(nest_folder, "sulemana_finaldat.csv"))

Southampton Defenders

In [129]:
walkerpeters_finaldat = compile_dat('984a5a64', 'Kyle-Walker-Peters', 'Walker-Peters', checkgames = True)
if isinstance(walkerpeters_finaldat, pd.DataFrame):
    walkerpeters_finaldat.to_csv(os.path.join(nest_folder_def, "walkerpeters_finaldat.csv"))


In [130]:
bednarek_finaldat = compile_dat('4115ce4e', 'Jan-Bednarek', 'Jan_Bednarek', checkgames = True)
if isinstance(bednarek_finaldat, pd.DataFrame):
    bednarek_finaldat.to_csv(os.path.join(nest_folder_def, "bednarek_finaldat.csv"))

In [131]:
jstephens_finaldat = compile_dat('6adbc307', 'Jack-Stephens', 'Jack_Stephens', checkgames = True)
if isinstance(jstephens_finaldat, pd.DataFrame):
    jstephens_finaldat.to_csv(os.path.join(nest_folder_def, "jstephens_finaldat.csv"))

In [132]:
ctaylor_finaldat = compile_dat('cc2d7ad5', 'Charlie-Taylor', 'Charlie_Taylor', checkgames = True)
if isinstance(ctaylor_finaldat, pd.DataFrame):
    ctaylor_finaldat.to_csv(os.path.join(nest_folder_def, "ctaylor_finaldat.csv"))

In [133]:
bellakotchap_finaldat = compile_dat('a39d11bc', 'Armel-Bella-Kotchap', 'Armel', checkgames = True)
if isinstance(bellakotchap_finaldat, pd.DataFrame):
    bellakotchap_finaldat.to_csv(os.path.join(nest_folder_def, "bellakotchap_finaldat.csv"))

## Spurs

In [274]:
solanke_finaldat = compile_dat('e77dc3b2', 'Dominic-Solanke', 'Dominic_Solanke', checkgames = True)
if isinstance(solanke_finaldat, pd.DataFrame):
    solanke_finaldat.to_csv(os.path.join(nest_folder, "solanke_finaldat.csv"))

In [275]:
richarlison_finaldat = compile_dat('fa031b34', 'Richarlison', 'Richarlison', checkgames = True)
if isinstance(richarlison_finaldat, pd.DataFrame):
    richarlison_finaldat.to_csv(os.path.join(nest_folder, "richarlison_finaldat.csv"))

In [276]:
son_finaldat = compile_dat('92e7e919', 'Son-Heung-min', 'Heung', checkgames = True)
if isinstance(son_finaldat, pd.DataFrame):
    son_finaldat.to_csv(os.path.join(nest_folder, "son_finaldat.csv"))

In [277]:
maddison_finaldat = compile_dat('ee38d9c5', 'James-Maddison', 'James_Maddison', checkgames = True)
if isinstance(maddison_finaldat, pd.DataFrame):
    maddison_finaldat.to_csv(os.path.join(nest_folder, "maddison_finaldat.csv"))

In [278]:
bissouma_finaldat = compile_dat('6c203af0', 'Yves-Bissouma', 'Yves_Bissouma', checkgames = True)
if isinstance(bissouma_finaldat, pd.DataFrame):
    bissouma_finaldat.to_csv(os.path.join(nest_folder, "bissouma_finaldat.csv"))

In [279]:
bjohnson_finaldat = compile_dat('0cd31129', 'Brennan-Johnson', 'Brennan_Johnson', checkgames = True)
if isinstance(bjohnson_finaldat, pd.DataFrame):
    bjohnson_finaldat.to_csv(os.path.join(nest_folder, "bjohnson_finaldat.csv"))

In [280]:
kulusevski_finaldat = compile_dat('df3cda47', 'Dejan-Kulusevski', 'Dejan_Kulusevski', checkgames = True)
if isinstance(kulusevski_finaldat, pd.DataFrame):
    kulusevski_finaldat.to_csv(os.path.join(nest_folder, "kulusevski_finaldat.csv"))

In [281]:
odobert_finaldat = compile_dat('35516eac', 'Wilson-Odobert', 'Wilson_Odobert', checkgames = True)
if isinstance(odobert_finaldat, pd.DataFrame):
    odobert_finaldat.to_csv(os.path.join(nest_folder, "odobert_finaldat.csv"))

In [282]:
pmsarr_finaldat = compile_dat('feb5d972', 'Pape-Matar-Sarr', 'Pape Matar', checkgames = True)
if isinstance(pmsarr_finaldat, pd.DataFrame):
    pmsarr_finaldat.to_csv(os.path.join(nest_folder, "pmsarr_finaldat.csv"))

In [283]:
werner_finaldat = compile_dat('49fe9070', 'Timo-Werner', 'Timo_Werner', checkgames = True)
if isinstance(werner_finaldat, pd.DataFrame):
    werner_finaldat.to_csv(os.path.join(nest_folder, "werner_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [284]:
bentancur_finaldat = compile_dat('3b8674e6', 'Rodrigo=Bentacur', 'Rodrigo_Bentancur', checkgames = True)
if isinstance(bentancur_finaldat, pd.DataFrame):
    bentancur_finaldat.to_csv(os.path.join(nest_folder, "bentancur_finaldat.csv"))

Spurs Defenders

In [134]:
romero_finaldat = compile_dat('a3d94a58', 'Cristian-Romero', 'Cristian_Romero', checkgames = True)
if isinstance(romero_finaldat, pd.DataFrame):
    romero_finaldat.to_csv(os.path.join(nest_folder_def, "romero_finaldat.csv"))

In [135]:
porro_finaldat = compile_dat('27d0a506', 'Pedro-Porro', 'Porro', checkgames = True)
if isinstance(porro_finaldat, pd.DataFrame):
    porro_finaldat.to_csv(os.path.join(nest_folder_def, "porro_finaldat.csv"))

In [136]:
vandeven_finaldat = compile_dat('8fe2a392', 'Micky-van-de-Ven', 'Micky', checkgames = True)
if isinstance(vandeven_finaldat, pd.DataFrame):
    vandeven_finaldat.to_csv(os.path.join(nest_folder_def, "vandeven_finaldat.csv"))

In [137]:
udogie_finaldat = compile_dat('7cd520e8', 'Destiny-Udogie', 'Destiny_Udogie', checkgames = True)
if isinstance(udogie_finaldat, pd.DataFrame):
    udogie_finaldat.to_csv(os.path.join(nest_folder_def, "udogie_finaldat.csv"))

In [138]:
bdavies_finaldat = compile_dat('44781702', 'Ben-Davies', 'Ben_Davies', checkgames = True)
if isinstance(bdavies_finaldat, pd.DataFrame):
    bdavies_finaldat.to_csv(os.path.join(nest_folder_def, "bdavies_finaldat.csv"))

The number of Premier League games played by Ben-Davies(171) does not match the number of rows in the dataframe (135)


In [139]:
reguilon_finaldat = compile_dat('3353737a', 'Sergio-Reguilon', 'Sergio_Regui', checkgames = True)
if isinstance(reguilon_finaldat, pd.DataFrame):
    reguilon_finaldat.to_csv(os.path.join(nest_folder_def, "reguilon_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


## West Ham

In [285]:
antonio_finaldat = compile_dat('ac05f970', 'Michail-Antonio', 'Michail_Antonio', checkgames = True)
if isinstance(antonio_finaldat, pd.DataFrame):
    antonio_finaldat.to_csv(os.path.join(nest_folder, "antonio_finaldat.csv"))

In [286]:
ings_finaldat = compile_dat('07802f7f', 'Danny-Ings', 'Danny_Ings', checkgames = True)
if isinstance(ings_finaldat, pd.DataFrame):
    ings_finaldat.to_csv(os.path.join(nest_folder, "ings_finaldat.csv"))

In [287]:
bowen_finaldat = compile_dat('79c84d1c', 'Jarrod-Bowen', 'Jarrod_Bowen', checkgames = True)
if isinstance(bowen_finaldat, pd.DataFrame):
    bowen_finaldat.to_csv(os.path.join(nest_folder, "bowen_finaldat.csv"))

In [288]:
soucek_finaldat = compile_dat('6613c819', 'Tomas-Soucek', 'Tomas_Soucek', checkgames = False)
if isinstance(soucek_finaldat, pd.DataFrame):
    soucek_finaldat.to_csv(os.path.join(nest_folder, "soucek_finaldat.csv"))

In [289]:
paqueta_finaldat = compile_dat('9b6f7fd5', 'Lucas-Paqueta', 'Lucas_Tolentino', checkgames = True)
if isinstance(paqueta_finaldat, pd.DataFrame):
    paqueta_finaldat.to_csv(os.path.join(nest_folder, "paqueta_finaldat.csv"))

In [290]:
kudus_finaldat = compile_dat('b62878a5', 'Mohammed-Kudus', 'Mohammed_Kudus', checkgames = True)
if isinstance(kudus_finaldat, pd.DataFrame):
    kudus_finaldat.to_csv(os.path.join(nest_folder, "kudus_finaldat.csv"))

In [291]:
edsonalvarez_finaldat = compile_dat('8b3ab7ad', 'Edson-Alvarez', 'Edson', checkgames = True)
if isinstance(edsonalvarez_finaldat, pd.DataFrame):
    edsonalvarez_finaldat.to_csv(os.path.join(nest_folder, "edsonalvarez_finaldat.csv"))

In [292]:
summerville_finaldat = compile_dat('df04eb4b', 'Crysencio-Summerville', 'Crysencio_Summerville', checkgames = True)
if isinstance(summerville_finaldat, pd.DataFrame):
    summerville_finaldat.to_csv(os.path.join(nest_folder, "summerville_finaldat.csv"))

West Ham Defenders

In [146]:
kilman_finaldat = compile_dat('d0f72bf1', 'Max-Kilman', 'Max_Kilman', checkgames = True)
if isinstance(kilman_finaldat, pd.DataFrame):
    kilman_finaldat.to_csv(os.path.join(nest_folder_def, "kilman_finaldat.csv"))

In [147]:
mavropanos_finaldat = compile_dat('00963611', 'Konstantinos-Mavropanos', 'Mavropanos', checkgames = True)
if isinstance(mavropanos_finaldat, pd.DataFrame):
    mavropanos_finaldat.to_csv(os.path.join(nest_folder_def, "mavropanos_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [148]:
coufal_finaldat = compile_dat('fdf3cb77', 'Vladimir-Coufal', 'Coufal', checkgames = True)
if isinstance(coufal_finaldat, pd.DataFrame):
    coufal_finaldat.to_csv(os.path.join(nest_folder_def, "coufal_finaldat.csv"))

In [149]:
emerson_finaldat = compile_dat('e0bc6fdc', 'Emerson-Palmieri', 'Palmieri', checkgames = True)
if isinstance(emerson_finaldat, pd.DataFrame):
    emerson_finaldat.to_csv(os.path.join(nest_folder_def, "emerson_finaldat.csv"))

In [150]:
awb_finaldat = compile_dat('9e525177', 'Aaron-Wan-Bissaka', 'Bissaka', checkgames = True)
if isinstance(awb_finaldat, pd.DataFrame):
    awb_finaldat.to_csv(os.path.join(nest_folder_def, "awb_finaldat.csv"))

In [151]:
cresswell_finaldat = compile_dat('4f974391', 'Aaron-Cresswell', 'Aaron_Cresswell', checkgames = True)
if isinstance(cresswell_finaldat, pd.DataFrame):
    cresswell_finaldat.to_csv(os.path.join(nest_folder_def, "cresswell_finaldat.csv"))

## Wolves

In [293]:
cunha_finaldat = compile_dat('dc62b55d', 'Matheus-Cunha', 'Matheus_Santos Carneiro', checkgames = True)
if isinstance(cunha_finaldat, pd.DataFrame):
    cunha_finaldat.to_csv(os.path.join(nest_folder, "cunha_finaldat.csv"))

In [294]:
bellegarde_finaldat = compile_dat('10f0fdd3', 'Jean-Ricner-Bellegarde', 'Bellegarde', checkgames = True)
if isinstance(bellegarde_finaldat, pd.DataFrame):
    bellegarde_finaldat.to_csv(os.path.join(nest_folder, "bellegarde_finaldat.csv"))

In [295]:
lemina_finaldat = compile_dat('2b471f99', 'Mario-Lemina', 'Mario_Lemina', checkgames = True)
if isinstance(lemina_finaldat, pd.DataFrame):
    lemina_finaldat.to_csv(os.path.join(nest_folder, "lemina_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [296]:
heechan_finaldat = compile_dat('169fd162', 'Hwang-Hee-chan', 'Hwang', checkgames = True)
if isinstance(heechan_finaldat, pd.DataFrame):
    heechan_finaldat.to_csv(os.path.join(nest_folder, "heechan_finaldat.csv"))

In [297]:
jgomes_finaldat = compile_dat('8b57ad2c', 'Joao-Gomes', 'Victor_Gomes', checkgames = True)
if isinstance(jgomes_finaldat, pd.DataFrame):
    jgomes_finaldat.to_csv(os.path.join(nest_folder, "jgomes_finaldat.csv"))

In [298]:
doyle_finaldat = compile_dat('5526deb1', 'Tommy-Doyle', 'Tommy_Doyle', checkgames = True)
if isinstance(doyle_finaldat, pd.DataFrame):
    doyle_finaldat.to_csv(os.path.join(nest_folder, "doyle_finaldat.csv"))

In [299]:
sarabia_finaldat = compile_dat('9744ff80', 'Pablo-Sarabia', 'Pablo_Sarabia', checkgames = True)
if isinstance(sarabia_finaldat, pd.DataFrame):
    sarabia_finaldat.to_csv(os.path.join(nest_folder, "sarabia_finaldat.csv"))

In [300]:
guedes_finaldat = compile_dat('e6bc67d7', 'Goncalo-Guedes', 'Ganchinho', checkgames = True)
if isinstance(guedes_finaldat, pd.DataFrame):
    guedes_finaldat.to_csv(os.path.join(nest_folder, "guedes_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


Wolves Defenders

In [152]:
aitnouri_finaldat = compile_dat('9b398aea', 'Rayan-Ait-Nouri', 'Rayan', checkgames = True)
if isinstance(aitnouri_finaldat, pd.DataFrame):
    aitnouri_finaldat.to_csv(os.path.join(nest_folder_def, "aitnouri_finaldat.csv"))

In [153]:
dawson_finaldat = compile_dat('3e9e06cb', 'Craig-Dawson', 'Craig_Dawson', checkgames = True)
if isinstance(dawson_finaldat, pd.DataFrame):
    dawson_finaldat.to_csv(os.path.join(nest_folder_def, "dawson_finaldat.csv"))

In [154]:
toti_finaldat = compile_dat('f0caab96', 'Toti-Gomes', 'Toti', checkgames = True)
if isinstance(toti_finaldat, pd.DataFrame):
    toti_finaldat.to_csv(os.path.join(nest_folder_def, "toti_finaldat.csv"))

In [155]:
semedo_finaldat = compile_dat('d04b94db', 'Nelson-Semedo', 'Semedo', checkgames = True)
if isinstance(semedo_finaldat, pd.DataFrame):
    semedo_finaldat.to_csv(os.path.join(nest_folder_def, "semedo_finaldat.csv"))

In [156]:
mdoherty_finaldat = compile_dat('d557d734', 'Matt-Doherty', 'Matt_Doherty', checkgames = True)
if isinstance(mdoherty_finaldat, pd.DataFrame):
    mdoherty_finaldat.to_csv(os.path.join(nest_folder_def, "mdoherty_finaldat.csv"))

# General Scrape

Now, we want to get the data for players that are not in the 2024-25 season. This could be previous premier league players who left their club, players who retired etc. 

In [210]:
#loading all of the raw data 

rawdat_2324 = pd.read_csv('/Users/fungs4/Desktop/Skye/Personal Education/FbrefWebscraping/Fantasy-Premier-League/data/2023-24/cleaned_players.csv')
rawdat_2324 = rawdat_2324[['first_name', 'second_name', 'minutes']]

rawdat_2223 = pd.read_csv('/Users/fungs4/Desktop/Skye/Personal Education/FbrefWebscraping/Fantasy-Premier-League/data/2022-23/cleaned_players.csv')
rawdat_2223 = rawdat_2223[['first_name', 'second_name', 'minutes']]

rawdat_2122 = pd.read_csv('/Users/fungs4/Desktop/Skye/Personal Education/FbrefWebscraping/Fantasy-Premier-League/data/2021-22/cleaned_players.csv')
rawdat_2122 = rawdat_2122[['first_name', 'second_name', 'minutes']]

rawdat_2021 = pd.read_csv('/Users/fungs4/Desktop/Skye/Personal Education/FbrefWebscraping/Fantasy-Premier-League/data/2020-21/cleaned_players.csv')
rawdat_2021 = rawdat_2021[['first_name', 'second_name', 'minutes']]

rawdat_1920 = pd.read_csv('/Users/fungs4/Desktop/Skye/Personal Education/FbrefWebscraping/Fantasy-Premier-League/data/2019-20/cleaned_players.csv')
rawdat_1920 = rawdat_1920[['first_name', 'second_name', 'minutes']]

rawdat_1819 = pd.read_csv('/Users/fungs4/Desktop/Skye/Personal Education/FbrefWebscraping/Fantasy-Premier-League/data/2018-19/cleaned_players.csv')
rawdat_1819 = rawdat_1819[['first_name', 'second_name', 'minutes']]

rawdat_1718 = pd.read_csv('/Users/fungs4/Desktop/Skye/Personal Education/FbrefWebscraping/Fantasy-Premier-League/data/2017-18/cleaned_players.csv')
rawdat_1718 = rawdat_1718[['first_name', 'second_name', 'minutes']]

In [188]:
#concatenate all of the raw data into one large dataframe 
rawdat_final = pd.concat([rawdat_1718, rawdat_1819, rawdat_1920, rawdat_2021, rawdat_2122, rawdat_2223, rawdat_2324], 
                         join = "inner", ignore_index = True)

In [190]:
#now, we remove all rows for players that have played less than 630 minutes 
rawdat_final = rawdat_final[rawdat_final['minutes'] > 630]

In [191]:
#now we assign each player a unique player id, we do this by first combining first name and second name into a full name 
rawdat_final['full_name'] = rawdat_final['first_name'] + ' ' + rawdat_final['second_name']
rawdat_final['player_id'] = rawdat_final.groupby('full_name').ngroup() + 1
rawdat_final.drop(columns = ['first_name', 'second_name'], inplace = True)

In [198]:
#we now filter out the dataframe so that we only keep one row for each player (or one row for each player_id)
rawdat_final_filtered = rawdat_final.drop_duplicates(subset = 'player_id')
rawdat_final_filtered = rawdat_final_filtered.drop(columns = ['minutes'])

In [203]:
#we now want to get the raw data from the current season. we are doing this so that we can further get rid of rows of players whose data we already 
#have 

rawdat_2425 = pd.read_csv('/Users/fungs4/Desktop/Skye/Personal Education/FbrefWebscraping/Fantasy-Premier-League/data/2024-25/cleaned_players.csv')
rawdat_2425 = rawdat_2425[['first_name', 'second_name', 'minutes']]
rawdat_2425['full_name'] = rawdat_2425['first_name'] + ' ' + rawdat_2425['second_name']
rawdat_2425 = rawdat_2425.drop(columns = ['first_name', 'second_name'])

In [205]:
#these are the names of the players whom we haven't gotten data for (i.e. the players that are not playing in the 24-25 season)
final_names = rawdat_final_filtered[~rawdat_final_filtered['full_name'].isin(rawdat_2425['full_name'])]

In [214]:
#this removes the players who were still in the final_names df, but who we already have data for. This is because there are certain players 
#whose names were changed throughout the years (diogo jota to diogo jota teixeira da silva)
playerid_remove = [928, 115, 514, 122, 845, 524, 708, 260, 515, 342, 265, 861, 61, 143, 969, 290, 979, 339, 685, 114, 130, 886, 281, 946, 
                   724, 639, 404, 380, 42, 557, 618, 529, 1009]
final_names = final_names[~final_names['player_id'].isin(playerid_remove)]

In [219]:
from fuzzywuzzy import process

#this function checks for similar names 
def find_similar_names(name, df, threshold = 80):
    #get all names from dataframe 
    all_names = df['full_name'].tolist()
    #use fuzzy matching to find similar names 
    matches = process.extract(name, all_names, limit = None)
    #filter matches 
    matched_names = [match[0] for match in matches if match[1] >= threshold]
    return(df[df['full_name'].isin(matched_names)])

In [275]:
#using the function from above to run through the final_names dataframe to check for similar names 
all_similar_names = pd.DataFrame()
for index, row in final_names.iterrows():
    name = row['full_name']
    similar_check = find_similar_names(name, final_names, threshold = 90)
    if similar_check.shape[0] > 1:
        similar_check = similar_check.iloc[1:]
        all_similar_names = pd.concat([all_similar_names, similar_check])

In [279]:
#dropping duplicate entries from all_similar_names
all_similar_names = all_similar_names.drop_duplicates(subset = 'player_id')

#getting the player_id's and turning it into a list, and then removing players with these player_ids from final_names
playerid_remove.clear()
playerid_remove = all_similar_names['player_id'].tolist()
final_names = final_names[~final_names['player_id'].isin(playerid_remove)]

In [309]:
#modifying the names in final_names to just have english letters 
from unidecode import unidecode

def remove_accents(input_str):
    return unidecode(input_str)

final_names.loc[:, 'cleaned_name'] = final_names['full_name'].apply(remove_accents)

In [310]:
final_names.to_csv(os.path.join(nest_folder, "final_names.csv"))

# Final Scrape
This is where we get the data for the remaining players

In [312]:
koscielny_finaldat = compile_dat('f5efb153', 'Laurent-Koscielny', 'Koscielny', checkgames = True)
if isinstance(koscielny_finaldat, pd.DataFrame):
    koscielny_finaldat.to_csv(os.path.join(nest_folder_def, "koscielny_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [314]:
bellerin_finaldat = compile_dat('f874dd44', 'Hector-Bellerin', 'Beller', checkgames = True)
if isinstance(koscielny_finaldat, pd.DataFrame):
    koscielny_finaldat.to_csv(os.path.join(nest_folder_def, "koscielny_finaldat.csv"))


In [315]:
monreal_finaldat = compile_dat('d4cb83cc', 'Nacho-Monreal', 'Monreal', checkgames = True)
if isinstance(monreal_finaldat, pd.DataFrame):
    monreal_finaldat.to_csv(os.path.join(nest_folder_def, "monreal_finaldat.csv"))

In [316]:
mustafi_finaldat = compile_dat('3f2d59fe', 'Shkodran-Mustafi', 'Mustafi', checkgames = True)
if isinstance(mustafi_finaldat, pd.DataFrame):
    mustafi_finaldat.to_csv(os.path.join(nest_folder_def, "mustafi_finaldat.csv"))

In [317]:
kolasinac_finaldat = compile_dat('3935e52e', 'Sead-Kolasinac', 'Sead_Kola', checkgames = True)
if isinstance(kolasinac_finaldat, pd.DataFrame):
    kolasinac_finaldat.to_csv(os.path.join(nest_folder_def, "kolasinac_finaldat.csv"))

In [318]:
ozil_finaldat = compile_dat('16380240', 'Mesut-Ozil', 'Mesut', checkgames = True)
if isinstance(ozil_finaldat, pd.DataFrame):
    ozil_finaldat.to_csv(os.path.join(nest_folder, "ozil_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [319]:
aramsey_finaldat = compile_dat('ef619b0b', 'Aaron-Ramsey', 'Aaron_Ramsey_Arsenal', checkgames = True)
if isinstance(aramsey_finaldat, pd.DataFrame):
    aramsey_finaldat.to_csv(os.path.join(nest_folder, "aramsey_finaldat.csv"))

In [320]:
elneny_finaldat = compile_dat('47a34c4f', 'Mohamed-Elneny', 'Elneny', checkgames = True)
if isinstance(elneny_finaldat, pd.DataFrame):
    elneny_finaldat.to_csv(os.path.join(nest_folder, "elneny_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [321]:
xhaka_finaldat = compile_dat('e61b8aee', 'Granit-Xhaka', 'Xhaka', checkgames = True)
if isinstance(xhaka_finaldat, pd.DataFrame):
    xhaka_finaldat.to_csv(os.path.join(nest_folder, "xhaka_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [322]:
wilshere_finaldat = compile_dat('9c318325', 'Jack-Wilshere', 'Jack_Wilshere', checkgames = True)
if isinstance(wilshere_finaldat, pd.DataFrame):
    wilshere_finaldat.to_csv(os.path.join(nest_folder, "wilshere_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [323]:
lacazette_finaldat = compile_dat('9dbb75ca', 'Alexandre-Lacazette', 'Lacazette', checkgames = True)
if isinstance(lacazette_finaldat, pd.DataFrame):
    lacazette_finaldat.to_csv(os.path.join(nest_folder, "lacazette_finaldat.csv"))

In [324]:
mkhitaryan_finaldat = compile_dat('dd0daf32', 'Henrikh-Mkhitaryan', 'Mkhitaryan', checkgames = True)
if isinstance(mkhitaryan_finaldat, pd.DataFrame):
    mkhitaryan_finaldat.to_csv(os.path.join(nest_folder, "mkhitaryan_finaldat.csv"))

In [325]:
cchambers_finaldat = compile_dat('dc6f5bdd', 'Calum-Chambers', 'Calum_Chambers', checkgames = True)
if isinstance(cchambers_finaldat, pd.DataFrame):
    cchambers_finaldat.to_csv(os.path.join(nest_folder_def, "cchambers_finaldat.csv"))

In [326]:
maitlandniles_finaldat = compile_dat('e3a5814e', 'Ainsley-Maitland-Niles', 'Maitland-Niles', checkgames = True)
if isinstance(maitlandniles_finaldat, pd.DataFrame):
    maitlandniles_finaldat.to_csv(os.path.join(nest_folder_def, "maitlandniles_finaldat.csv"))

In [327]:
aubameyang_finaldat = compile_dat('d5dd5f1f', 'Pierre-Emerick-Aubameyang', 'Aubameyang', checkgames = True)
if isinstance(aubameyang_finaldat, pd.DataFrame):
    aubameyang_finaldat.to_csv(os.path.join(nest_folder, "aubameyang_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [328]:
sfrancis_finaldat = compile_dat('1e0b6a73', 'Simon-Francis', 'Simon_Francis', checkgames = True)
if isinstance(sfrancis_finaldat, pd.DataFrame):
    sfrancis_finaldat.to_csv(os.path.join(nest_folder_def, "sfrancis_finaldat.csv"))

In [329]:
scook_finaldat = compile_dat('3e5e4e63', 'Steve-Cook', 'Steve_Cook', checkgames = True)
if isinstance(scook_finaldat, pd.DataFrame):
    scook_finaldat.to_csv(os.path.join(nest_folder_def, "scook_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [330]:
cdaniels_finaldat = compile_dat('9d107085', 'Charlie-Daniels', 'Charlie_Daniels', checkgames = True)
if isinstance(cdaniels_finaldat, pd.DataFrame):
    cdaniels_finaldat.to_csv(os.path.join(nest_folder_def, "cdaniels_finaldat.csv"))

In [331]:
arter_finaldat = compile_dat('2f4c5a52', 'Harry-Arter', 'Harry_Arter', checkgames = True)
if isinstance(arter_finaldat, pd.DataFrame):
    arter_finaldat.to_csv(os.path.join(nest_folder, "arter_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [332]:
pugh_finaldat = compile_dat('9aacb284', 'Marc-Pugh', 'Marc_Pugh', checkgames = True)
if isinstance(pugh_finaldat, pd.DataFrame):
    pugh_finaldat.to_csv(os.path.join(nest_folder, "pugh_finaldat.csv"))

In [333]:
surman_finaldat = compile_dat('7d41b619', 'Andrew-Surman', 'Andrew_Surman', checkgames = True)
if isinstance(surman_finaldat, pd.DataFrame):
    surman_finaldat.to_csv(os.path.join(nest_folder, "surman_finaldat.csv"))

In [334]:
stanislas_finaldat = compile_dat('6992791d', 'Junior-Stanislas', 'Junior_Stanislas', checkgames = True)
if isinstance(stanislas_finaldat, pd.DataFrame):
    stanislas_finaldat.to_csv(os.path.join(nest_folder, "stanislas_finaldat.csv"))

In [335]:
gosling_finaldat = compile_dat('7fec31a0', 'Dan-Gosling', 'Dan_Gosling', checkgames = True)
if isinstance(gosling_finaldat, pd.DataFrame):
    gosling_finaldat.to_csv(os.path.join(nest_folder, "gosling_finaldat.csv"))

In [336]:
ibe_finaldat = compile_dat('9f736ed2', 'Jordon-Ibe', 'Jordon_Ibe', checkgames = True)
if isinstance(ibe_finaldat, pd.DataFrame):
    ibe_finaldat.to_csv(os.path.join(nest_folder, "ibe_finaldat.csv"))

In [338]:
jking_finaldat = compile_dat('43458538', 'Joshua-King', 'Joshua_King', checkgames = True)
if isinstance(jking_finaldat, pd.DataFrame):
    jking_finaldat.to_csv(os.path.join(nest_folder, "jking_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [339]:
defoe_finaldat = compile_dat('f1e8372d', 'Jermain-Defoe', 'Jermain_Defoe', checkgames = True)
if isinstance(defoe_finaldat, pd.DataFrame):
    defoe_finaldat.to_csv(os.path.join(nest_folder, "defoe_finaldat.csv"))

In [340]:
duffy_finaldat = compile_dat('7314bba4', 'Shane-Duffy', 'Shane_Duffy', checkgames = True)
if isinstance(duffy_finaldat, pd.DataFrame):
    duffy_finaldat.to_csv(os.path.join(nest_folder_def, "duffy_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [341]:
saltor_finaldat = compile_dat('78803d03', 'Bruno', 'Saltor', checkgames = True)
if isinstance(saltor_finaldat, pd.DataFrame):
    saltor_finaldat.to_csv(os.path.join(nest_folder_def, "saltor_finaldat.csv"))

In [342]:
bong_finaldat = compile_dat('cfd5e123', 'Gaeton-Bong', 'Bong', checkgames = True)
if isinstance(bong_finaldat, pd.DataFrame):
    bong_finaldat.to_csv(os.path.join(nest_folder_def, "bong_finaldat.csv"))

In [343]:
knockaert_finaldat = compile_dat('6c679c6d', 'Anthony-Knockaert', 'Knockaert', checkgames = True)
if isinstance(knockaert_finaldat, pd.DataFrame):
    knockaert_finaldat.to_csv(os.path.join(nest_folder, "knockaert_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [344]:
dstephens_finaldat = compile_dat('653ee42d', 'Dale-Stephens', 'Dale_Stephens', checkgames = True)
if isinstance(dstephens_finaldat, pd.DataFrame):
    dstephens_finaldat.to_csv(os.path.join(nest_folder, "dstephens_finaldat.csv"))

In [345]:
kayal_finaldat = compile_dat('26679cdf', 'Beram-Kayal', 'Kayal', checkgames = True)
if isinstance(kayal_finaldat, pd.DataFrame):
    kayal_finaldat.to_csv(os.path.join(nest_folder, "kayal_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [346]:
gmurray_finaldat = compile_dat('f2f29dda', 'Glenn-Murray', 'Glenn_Murray', checkgames = True)
if isinstance(gmurray_finaldat, pd.DataFrame):
    gmurray_finaldat.to_csv(os.path.join(nest_folder, "gmurray_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [347]:
hemed_finaldat = compile_dat('27d7453c', 'Tomer-Hemed', 'Hemed', checkgames = True)
if isinstance(hemed_finaldat, pd.DataFrame):
    hemed_finaldat.to_csv(os.path.join(nest_folder, "hemed_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [351]:
propper_finaldat = compile_dat('fdd85f94', 'Davy-Propper', 'Davy_Pr', checkgames = True)
if isinstance(propper_finaldat, pd.DataFrame):
    propper_finaldat.to_csv(os.path.join(nest_folder, "propper_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [352]:
izquierdo_finaldat = compile_dat('ca2bc3df', 'Jose-Izquierdo', 'Izquierdo', checkgames = True)
if isinstance(izquierdo_finaldat, pd.DataFrame):
    izquierdo_finaldat.to_csv(os.path.join(nest_folder, "izquierdo_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [530]:
schelotto_finaldat = compile_dat('a6e316c1', 'Ezequiel-Schelotto', 'Schelotto', checkgames = False)
if isinstance(schelotto_finaldat, pd.DataFrame):
    schelotto_finaldat.to_csv(os.path.join(nest_folder_def, "schelotto_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [357]:
lowton_finaldat = compile_dat('9a9c05db', 'Matthew-Lowton', 'Lowton', checkgames = True)
if isinstance(lowton_finaldat, pd.DataFrame):
    lowton_finaldat.to_csv(os.path.join(nest_folder_def, "lowton_finaldat.csv"))

In [355]:
sward_finaldat = compile_dat('3831b9fe', 'Stephen-Ward', 'Stephen_Ward', checkgames = True)
if isinstance(sward_finaldat, pd.DataFrame):
    sward_finaldat.to_csv(os.path.join(nest_folder_def, "sward_finaldat.csv"))

In [358]:
klong_finaldat = compile_dat('5a4280b1', 'Kevin-Long', 'Kevin_Long', checkgames = True)
if isinstance(klong_finaldat, pd.DataFrame):
    klong_finaldat.to_csv(os.path.join(nest_folder_def, "klong_finaldat.csv"))

In [359]:
arfield_finaldat = compile_dat('b1313ec4', 'Scott-Arfield', 'Arfield', checkgames = True)
if isinstance(arfield_finaldat, pd.DataFrame):
    arfield_finaldat.to_csv(os.path.join(nest_folder, "arfield_finaldat.csv"))

In [360]:
gudmundsson_finaldat = compile_dat('f72293fc', 'Johann-Berg-Gudmundsson', 'Berg_Gud', checkgames = True)
if isinstance(gudmundsson_finaldat, pd.DataFrame):
    gudmundsson_finaldat.to_csv(os.path.join(nest_folder, "gudmundsson_finaldat.csv"))

In [361]:
defour_finaldat = compile_dat('73489527', 'Steven-Defour', 'Steven_Defour', checkgames = True)
if isinstance(defour_finaldat, pd.DataFrame):
    defour_finaldat.to_csv(os.path.join(nest_folder, "defour_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [362]:
hendrick_finaldat = compile_dat('989d5705', 'Jeff-Hendrick', 'Jeff_Hendrick', checkgames = True)
if isinstance(hendrick_finaldat, pd.DataFrame):
    hendrick_finaldat.to_csv(os.path.join(nest_folder, "hendrick_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [363]:
brady_finaldat = compile_dat('96bb8cc2', 'Robbie-Brady', 'Robbie_Brady', checkgames = True)
if isinstance(brady_finaldat, pd.DataFrame):
    brady_finaldat.to_csv(os.path.join(nest_folder, "brady_finaldat.csv"))

In [364]:
awestwood_finaldat = compile_dat('495945ef', 'Ashley-Westwood', 'Ashley_Westwood', checkgames = True)
if isinstance(awestwood_finaldat, pd.DataFrame):
    awestwood_finaldat.to_csv(os.path.join(nest_folder, "awestwood_finaldat.csv"))

In [365]:
jcork_finaldat = compile_dat('74e12f1e', 'Jack-Cork', 'Jack_Cork', checkgames = True)
if isinstance(jcork_finaldat, pd.DataFrame):
    jcork_finaldat.to_csv(os.path.join(nest_folder, "jcork_finaldat.csv"))

In [366]:
vokes_finaldat = compile_dat('1d6ac165', 'Sam-Vokes', 'Vokes', checkgames = True)
if isinstance(vokes_finaldat, pd.DataFrame):
    vokes_finaldat.to_csv(os.path.join(nest_folder, "vokes_finaldat.csv"))

In [367]:
abarnes_finaldat = compile_dat('2691c772', 'Ashley-Barnes', 'Ashley_Barnes', checkgames = True)
if isinstance(abarnes_finaldat, pd.DataFrame):
    abarnes_finaldat.to_csv(os.path.join(nest_folder, "abarnes_finaldat.csv"))

In [368]:
alennon_finaldat = compile_dat('2ff964a0', 'Aaron-Lennon', 'Aaron_Lennon', checkgames = True)
if isinstance(alennon_finaldat, pd.DataFrame):
    alennon_finaldat.to_csv(os.path.join(nest_folder, "alennon_finaldat.csv"))

In [369]:
bardsley_finaldat = compile_dat('1ae30132', 'Phil-Bardsley', 'Bardsley', checkgames = True)
if isinstance(bardsley_finaldat, pd.DataFrame):
    bardsley_finaldat.to_csv(os.path.join(nest_folder_def, "bardsley_finaldat.csv"))

In [370]:
giroud_finaldat = compile_dat('16ceb862', 'Olivier-Giroud', 'Giroud', checkgames = True)
if isinstance(giroud_finaldat, pd.DataFrame):
    giroud_finaldat.to_csv(os.path.join(nest_folder, "giroud_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [371]:
azpilicueta_finaldat = compile_dat('53cad200', 'Cesar-Azpilicueta', 'Azpilicueta', checkgames = True)
if isinstance(azpilicueta_finaldat, pd.DataFrame):
    azpilicueta_finaldat.to_csv(os.path.join(nest_folder_def, "azpilicueta_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [372]:
cahill_finaldat = compile_dat('7914b9fe', 'Gary-Cahill', 'Cahill', checkgames = True)
if isinstance(cahill_finaldat, pd.DataFrame):
    cahill_finaldat.to_csv(os.path.join(nest_folder_def, "cahill_finaldat.csv"))

In [373]:
malonso_finaldat = compile_dat('f4290206', 'Marcos-Alonso', 'Marcos_Alonso', checkgames = True)
if isinstance(malonso_finaldat, pd.DataFrame):
    malonso_finaldat.to_csv(os.path.join(nest_folder_def, "malonso_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [375]:
davidluiz_finaldat = compile_dat('c0e27d1a', 'David-Luiz', 'David_Luiz', checkgames = True)
if isinstance(davidluiz_finaldat, pd.DataFrame):
    davidluiz_finaldat.to_csv(os.path.join(nest_folder_def, "davidluiz_finaldat.csv"))

In [531]:
moses_finaldat = compile_dat('e726e11e', 'Victor-Moses', 'Moses', checkgames = False)
if isinstance(moses_finaldat, pd.DataFrame):
    moses_finaldat.to_csv(os.path.join(nest_folder, "moses_finaldat.csv"))

In [377]:
rudiger_finaldat = compile_dat('18b896d6', 'Antonio-Rudiger', 'Antonio_R', checkgames = True)
if isinstance(rudiger_finaldat, pd.DataFrame):
    rudiger_finaldat.to_csv(os.path.join(nest_folder_def, "rudiger_finaldat.csv"))

In [378]:
hazard_finaldat = compile_dat('a39bb753', 'Eden-Hazard', 'Hazard', checkgames = True)
if isinstance(hazard_finaldat, pd.DataFrame):
    hazard_finaldat.to_csv(os.path.join(nest_folder, "hazard_finaldat.csv"))

In [379]:
fabregas_finaldat = compile_dat('33508c78', 'Cesc-Fabregas', 'Cesc', checkgames = True)
if isinstance(fabregas_finaldat, pd.DataFrame):
    fabregas_finaldat.to_csv(os.path.join(nest_folder, "fabregas_finaldat.csv"))

In [380]:
willian_finaldat = compile_dat('8b9ebd03', 'Willian', 'Willian', checkgames = False)
if isinstance(willian_finaldat, pd.DataFrame):
    willian_finaldat.to_csv(os.path.join(nest_folder, "willian_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [381]:
pedro_finaldat = compile_dat('3ca7254a', 'Pedro', 'Ledesma', checkgames = True)
if isinstance(pedro_finaldat, pd.DataFrame):
    pedro_finaldat.to_csv(os.path.join(nest_folder, "pedro_finaldat.csv"))


In [383]:
kante_finaldat = compile_dat('b9fbae28', 'NGolo-Kante', 'Golo', checkgames = True)
if isinstance(kante_finaldat, pd.DataFrame):
    kante_finaldat.to_csv(os.path.join(nest_folder, "kante_finaldat.csv"))

In [384]:
bakayoko_finaldat = compile_dat('59bf127b', 'Tiemoue-Bakayoko', 'Bakayoko', checkgames = True)
if isinstance(bakayoko_finaldat, pd.DataFrame):
    bakayoko_finaldat.to_csv(os.path.join(nest_folder, "bakayoko_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [385]:
christensen_finaldat = compile_dat('1cb49278', 'Andreas-Christensen', 'Andreas_Christensen', checkgames = True)
if isinstance(christensen_finaldat, pd.DataFrame):
    christensen_finaldat.to_csv(os.path.join(nest_folder_def, "christensen_finaldat.csv"))

In [386]:
morata_finaldat = compile_dat('129af0db', 'Alvaro-Morata', 'Morata', checkgames = True)
if isinstance(morata_finaldat, pd.DataFrame):
    morata_finaldat.to_csv(os.path.join(nest_folder, "morata_finaldat.csv"))

In [387]:
zappacosta_finaldat = compile_dat('7fe90ccd', 'Davide-Zappacosta', 'Zappacosta', checkgames = True)
if isinstance(zappacosta_finaldat, pd.DataFrame):
    zappacosta_finaldat.to_csv(os.path.join(nest_folder_def, "zappacosta_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [389]:
loftuscheek_finaldat = compile_dat('e97fd090', 'Ruben-Loftus-Cheek', 'Loftus-Cheek', checkgames = True)
if isinstance(loftuscheek_finaldat, pd.DataFrame):
    loftuscheek_finaldat.to_csv(os.path.join(nest_folder, "loftuscheek_finaldat.csv"))

In [390]:
tomkins_finaldat = compile_dat('e7ca0439', 'James-Tomkins', 'James_Tomkins', checkgames = True)
if isinstance(tomkins_finaldat, pd.DataFrame):
    tomkins_finaldat.to_csv(os.path.join(nest_folder_def, "tomkins_finaldat.csv"))

In [391]:
dann_finaldat = compile_dat('52105203', 'Scott-Dann', 'Scott_Dann', checkgames = True)
if isinstance(dann_finaldat, pd.DataFrame):
    dann_finaldat.to_csv(os.path.join(nest_folder_def, "dann_finaldat.csv"))

In [392]:
mkelly_finaldat = compile_dat('d7492450', 'Martin-Kelly', 'Martin_Kelly', checkgames = True)
if isinstance(mkelly_finaldat, pd.DataFrame):
    mkelly_finaldat.to_csv(os.path.join(nest_folder_def, "mkelly_finaldat.csv"))

In [393]:
vanaanholt_finaldat = compile_dat('5f09991f', 'Patrick-van-Aanholt', 'van Aanholt', checkgames = True)
if isinstance(vanaanholt_finaldat, pd.DataFrame):
    vanaanholt_finaldat.to_csv(os.path.join(nest_folder_def, "vanaanholt_finaldat.csv"))

In [394]:
sakho_finaldat = compile_dat('87935cf3', 'Mamadou-Sakho', 'Mamadou_Sakho', checkgames = True)
if isinstance(sakho_finaldat, pd.DataFrame):
    sakho_finaldat.to_csv(os.path.join(nest_folder_def, "sakho_finaldat.csv"))

In [395]:
fosumensah_finaldat = compile_dat('a9142dc3', 'Timothy-Fosu-Mensah', 'Fosu-Mensah', checkgames = True)
if isinstance(fosumensah_finaldat, pd.DataFrame):
    fosumensah_finaldat.to_csv(os.path.join(nest_folder_def, "fosumensah_finaldat.csv"))

In [396]:
zaha_finaldat = compile_dat('b2bc3b1f', 'Wilfried-Zaha', 'Wilfried_Zaha', checkgames = True)
if isinstance(zaha_finaldat, pd.DataFrame):
    zaha_finaldat.to_csv(os.path.join(nest_folder, "zaha_finaldat.csv"))

In [397]:
mcarthur_finaldat = compile_dat('cee31595', 'James-McArthur', 'James_McArthur', checkgames = True)
if isinstance(mcarthur_finaldat, pd.DataFrame):
    mcarthur_finaldat.to_csv(os.path.join(nest_folder, "mcarthur_finaldat.csv"))

In [398]:
cabaye_finaldat = compile_dat('ad7b19ef', 'Yohan-Cabaye', 'Yohan_Cabaye', checkgames = True)
if isinstance(cabaye_finaldat, pd.DataFrame):
    cabaye_finaldat.to_csv(os.path.join(nest_folder, "cabaye_finaldat.csv"))

In [399]:
townsend_finaldat = compile_dat('b28bbd58', 'Andros-Townsend', 'Andros_Townsend', checkgames = True)
if isinstance(townsend_finaldat, pd.DataFrame):
    townsend_finaldat.to_csv(os.path.join(nest_folder, "townsend_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [400]:
milivojevic_finaldat = compile_dat('2ce2508a', 'Luka-Milivojevic', 'Milivojevi', checkgames = True)
if isinstance(milivojevic_finaldat, pd.DataFrame):
    milivojevic_finaldat.to_csv(os.path.join(nest_folder, "milivojevic_finaldat.csv"))

In [401]:
benteke_finaldat = compile_dat('ab070c55', 'Christian-Benteke', 'Christian_Benteke', checkgames = True)
if isinstance(benteke_finaldat, pd.DataFrame):
    benteke_finaldat.to_csv(os.path.join(nest_folder, "benteke_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [402]:
walcott_finaldat = compile_dat('1ecb65be', 'Theo-Walcott', 'Walcott', checkgames = True)
if isinstance(walcott_finaldat, pd.DataFrame):
    walcott_finaldat.to_csv(os.path.join(nest_folder, "walcott_finaldat.csv"))

In [403]:
baines_finaldat = compile_dat('b18095b6', 'Leighton-Baines', 'Baines', checkgames = True)
if isinstance(baines_finaldat, pd.DataFrame):
    baines_finaldat.to_csv(os.path.join(nest_folder_def, "baines_finaldat.csv"))

In [404]:
jagielka_finaldat = compile_dat('3233eefd', 'Phil-Jagielka', 'Jagielka', checkgames = True)
if isinstance(jagielka_finaldat, pd.DataFrame):
    jagielka_finaldat.to_csv(os.path.join(nest_folder_def, "jagielka_finaldat.csv"))

In [405]:
awilliams_finaldat = compile_dat('a498e276', 'Ashley-Williams', 'Ashley_Williams', checkgames = True)
if isinstance(awilliams_finaldat, pd.DataFrame):
    awilliams_finaldat.to_csv(os.path.join(nest_folder_def, "awilliams_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [406]:
jkenny_finaldat = compile_dat('099e666f', 'Jonjoe-Kenny', 'Jonjoe_Kenny', checkgames = True)
if isinstance(jkenny_finaldat, pd.DataFrame):
    jkenny_finaldat.to_csv(os.path.join(nest_folder_def, "jkenny_finaldat.csv"))

In [407]:
bolasie_finaldat = compile_dat('715fc0fe', 'Yannick-Bolasie', 'Bolasie', checkgames = True)
if isinstance(bolasie_finaldat, pd.DataFrame):
    bolasie_finaldat.to_csv(os.path.join(nest_folder, "bolasie_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [408]:
schneiderlin_finaldat = compile_dat('51b41c5c', 'Morgan-Schneiderlin', 'Schneiderlin', checkgames = True)
if isinstance(schneiderlin_finaldat, pd.DataFrame):
    schneiderlin_finaldat.to_csv(os.path.join(nest_folder, "schneiderlin_finaldat.csv"))

In [409]:
tdavies_finaldat = compile_dat('9228b07c', 'Tom-Davies', 'Tom_Davies', checkgames = True)
if isinstance(tdavies_finaldat, pd.DataFrame):
    tdavies_finaldat.to_csv(os.path.join(nest_folder, "tdavies_finaldat.csv"))

In [410]:
rooney_finaldat = compile_dat('f07be45a', 'Wayne-Rooney', 'Rooney', checkgames = True)
if isinstance(rooney_finaldat, pd.DataFrame):
    rooney_finaldat.to_csv(os.path.join(nest_folder, "rooney_finaldat.csv"))

In [411]:
sigurdsson_finaldat = compile_dat('76dd1480', 'Gylfi-Sigurdsson', 'Gylfi', checkgames = True)
if isinstance(sigurdsson_finaldat, pd.DataFrame):
    sigurdsson_finaldat.to_csv(os.path.join(nest_folder, "sigurdsson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [412]:
cmartina_finaldat = compile_dat('22551915', 'Cuco-Martina', 'Cuco_Martina', checkgames = True)
if isinstance(cmartina_finaldat, pd.DataFrame):
    cmartina_finaldat.to_csv(os.path.join(nest_folder_def, "cmartina_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [413]:
niasse_finaldat = compile_dat('af721f98', 'Oumar-Niasse', 'Oumar_Niasse', checkgames = True)
if isinstance(niasse_finaldat, pd.DataFrame):
    niasse_finaldat.to_csv(os.path.join(nest_folder, "niasse_finaldat.csv"))

In [417]:
tosun_finaldat = compile_dat('9a104dee', 'Cenk-Tosun', 'Tosun', checkgames = True)
if isinstance(tosun_finaldat, pd.DataFrame):
    tosun_finaldat.to_csv(os.path.join(nest_folder, "tosun_finaldat.csv"))

In [418]:
schindler_finaldat = compile_dat('8ae9869d', 'Christopher-Schindler', 'Schindler', checkgames = True)
if isinstance(schindler_finaldat, pd.DataFrame):
    schindler_finaldat.to_csv(os.path.join(nest_folder_def, "schindler_finaldat.csv"))

In [419]:
clowe_finaldat = compile_dat('c6896eab', 'Chris-Lowe', 'Chris_L', checkgames = True)
if isinstance(clowe_finaldat, pd.DataFrame):
    clowe_finaldat.to_csv(os.path.join(nest_folder_def, "clowe_finaldat.csv"))

In [420]:
tsmith_finaldat = compile_dat('e9208654', 'Tommy-Smith', 'Tommy_Smith', checkgames = True)
if isinstance(tsmith_finaldat, pd.DataFrame):
    tsmith_finaldat.to_csv(os.path.join(nest_folder_def, "tsmith_finaldat.csv"))

In [421]:
mooy_finaldat = compile_dat('47b7e3af', 'Aaron-Mooy', 'Mooy', checkgames = True)
if isinstance(mooy_finaldat, pd.DataFrame):
    mooy_finaldat.to_csv(os.path.join(nest_folder, "mooy_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [422]:
vanlaparra_finaldat = compile_dat('1931bde9', 'Rajiv-van-La-Parra', 'Rajiv', checkgames = True)
if isinstance(vanlaparra_finaldat, pd.DataFrame):
    vanlaparra_finaldat.to_csv(os.path.join(nest_folder, "vanlaparra_finaldat.csv"))

In [423]:
hogg_finaldat = compile_dat('9306d5ea', 'Jonathan-Hogg', 'Hogg', checkgames = True)
if isinstance(hogg_finaldat, pd.DataFrame):
    hogg_finaldat.to_csv(os.path.join(nest_folder, "hogg_finaldat.csv"))

In [424]:
kachunga_finaldat = compile_dat('5bddccc9', 'Elias-Kachunga', 'Kachunga', checkgames = True)
if isinstance(kachunga_finaldat, pd.DataFrame):
    kachunga_finaldat.to_csv(os.path.join(nest_folder, "kachunga_finaldat.csv"))

In [425]:
ince_finaldat = compile_dat('67d23ab0', 'Tom-Ince', 'Ince', checkgames = True)
if isinstance(ince_finaldat, pd.DataFrame):
    ince_finaldat.to_csv(os.path.join(nest_folder, "ince_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [426]:
dwilliams_finaldat = compile_dat('ec9a28ce', 'Danny-Williams', 'Danny_Williams', checkgames = True)
if isinstance(dwilliams_finaldat, pd.DataFrame):
    dwilliams_finaldat.to_csv(os.path.join(nest_folder, "dwilliams_finaldat.csv"))

In [427]:
quaner_finaldat = compile_dat('19b8b0b9', 'Collin-Quaner', 'Quaner', checkgames = True)
if isinstance(quaner_finaldat, pd.DataFrame):
    quaner_finaldat.to_csv(os.path.join(nest_folder, "quaner_finaldat.csv"))

In [428]:
depoitre_finaldat = compile_dat('aa5859cb', 'Laurent-Depoitre', 'Depoitre', checkgames = True)
if isinstance(depoitre_finaldat, pd.DataFrame):
    depoitre_finaldat.to_csv(os.path.join(nest_folder, "depoitre_finaldat.csv"))

In [429]:
mounie_finaldat = compile_dat('19ba6c6d', 'Steve-Mounie', 'Steve_Mouni', checkgames = True)
if isinstance(mounie_finaldat, pd.DataFrame):
    mounie_finaldat.to_csv(os.path.join(nest_folder, "mounie_finaldat.csv"))

In [430]:
malone_finaldat = compile_dat('f15a50e8', 'Scott-Malone', 'Scott_Malone', checkgames = True)
if isinstance(malone_finaldat, pd.DataFrame):
    malone_finaldat.to_csv(os.path.join(nest_folder_def, "malone_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [431]:
hadergjonaj_finaldat = compile_dat('9aa186b2', 'Florent-Hadergjonaj', 'Hadergjonaj', checkgames = True)
if isinstance(hadergjonaj_finaldat, pd.DataFrame):
    hadergjonaj_finaldat.to_csv(os.path.join(nest_folder_def, "hadergjonaj_finaldat.csv"))

In [432]:
kongolo_finaldat = compile_dat('1f9a3a04', 'Terence-Kongolo', 'Kongolo', checkgames = True)
if isinstance(kongolo_finaldat, pd.DataFrame):
    kongolo_finaldat.to_csv(os.path.join(nest_folder_def, "kongolo_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [433]:
pritchard_finaldat = compile_dat('ff9fcd4d', 'Alex-Pritchard', 'Pritchard', checkgames = True)
if isinstance(pritchard_finaldat, pd.DataFrame):
    pritchard_finaldat.to_csv(os.path.join(nest_folder, "pritchard_finaldat.csv"))

In [434]:
wmorgan_finaldat = compile_dat('80c31c97', 'Wes-Morgan', 'Wes_Morgan', checkgames = True)
if isinstance(wmorgan_finaldat, pd.DataFrame):
    wmorgan_finaldat.to_csv(os.path.join(nest_folder_def, "wmorgan_finaldat.csv"))

In [435]:
fuchs_finaldat = compile_dat('a67a09fb', 'Christian-Fuchs', 'Fuchs', checkgames = True)
if isinstance(fuchs_finaldat, pd.DataFrame):
    fuchs_finaldat.to_csv(os.path.join(nest_folder_def, "fuchs_finaldat.csv"))

In [436]:
dsimpson_finaldat = compile_dat('3201b03d', 'Danny-Simpson', 'Danny_Simpson', checkgames = True)
if isinstance(dsimpson_finaldat, pd.DataFrame):
    dsimpson_finaldat.to_csv(os.path.join(nest_folder_def, "dsimpson_finaldat.csv"))

In [437]:
mahrez_finaldat = compile_dat('892d5bb1', 'Riyad-Mahrez', 'Mahrez', checkgames = True)
if isinstance(mahrez_finaldat, pd.DataFrame):
    mahrez_finaldat.to_csv(os.path.join(nest_folder, "mahrez_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [438]:
albrighton_finaldat = compile_dat('b827d5b3', 'Marc-Albrighton', 'Albrighton', checkgames = True)
if isinstance(albrighton_finaldat, pd.DataFrame):
    albrighton_finaldat.to_csv(os.path.join(nest_folder, "albrighton_finaldat.csv"))

In [439]:
dgray_finaldat = compile_dat('4468ec10', 'Demarai-Gray', 'Demarai', checkgames = True)
if isinstance(dgray_finaldat, pd.DataFrame):
    dgray_finaldat.to_csv(os.path.join(nest_folder, "dgray_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [440]:
iborra_finaldat = compile_dat('ad1ffcfd', 'Vincente-Iborra', 'Iborra', checkgames = True)
if isinstance(iborra_finaldat, pd.DataFrame):
    iborra_finaldat.to_csv(os.path.join(nest_folder, "iborra_finaldat.csv"))

In [441]:
okazaki_finaldat = compile_dat('68796d61', 'Shinji-Okazaki', 'Okazaki', checkgames = True)
if isinstance(okazaki_finaldat, pd.DataFrame):
    okazaki_finaldat.to_csv(os.path.join(nest_folder, "okazaki_finaldat.csv"))

In [442]:
iheanacho_finaldat = compile_dat('c92e1a31', 'Kelechi-Iheanacho', 'Iheanacho', checkgames = True)
if isinstance(iheanacho_finaldat, pd.DataFrame):
    iheanacho_finaldat.to_csv(os.path.join(nest_folder, "iheanacho_finaldat.csv"))

In [443]:
mjames_finaldat = compile_dat('ebd0a048', 'Matty-James', 'Matty_James', checkgames = True)
if isinstance(mjames_finaldat, pd.DataFrame):
    mjames_finaldat.to_csv(os.path.join(nest_folder, "mjames_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [444]:
adragovic_finaldat = compile_dat('6fd8a334', 'Aleksandar-Dragovic', 'Aleksandar_Dragovic', checkgames = False)
if isinstance(adragovic_finaldat, pd.DataFrame):
    adragovic_finaldat.to_csv(os.path.join(nest_folder_def, "adragovic_finaldat.csv"))

In [445]:
asilva_finaldat = compile_dat('d6f88a1d', 'Adrien-Silva', 'Perruchet', checkgames = True)
if isinstance(asilva_finaldat, pd.DataFrame):
    asilva_finaldat.to_csv(os.path.join(nest_folder, "asilva_finaldat.csv"))

In [446]:
oxlade_finaldat = compile_dat('38c7feef', 'Alex-Oxlade-Chamberlain', 'Oxlade', checkgames = True)
if isinstance(oxlade_finaldat, pd.DataFrame):
    oxlade_finaldat.to_csv(os.path.join(nest_folder, "oxlade_finaldat.csv"))

In [447]:
lovren_finaldat = compile_dat('1a2935f6', 'Dejan-Lovren', 'Lovren', checkgames = False)
if isinstance(lovren_finaldat, pd.DataFrame):
    lovren_finaldat.to_csv(os.path.join(nest_folder_def, "lovren_finaldat.csv"))

In [448]:
moreno_finaldat = compile_dat('11b9265b', 'Alberto-Moreno', 'Alberto_Moreno', checkgames = True)
if isinstance(moreno_finaldat, pd.DataFrame):
    moreno_finaldat.to_csv(os.path.join(nest_folder_def, "moreno_finaldat.csv"))

In [449]:
matip_finaldat = compile_dat('b217ef29', 'Joel-Matip', 'Matip', checkgames = True)
if isinstance(matip_finaldat, pd.DataFrame):
    matip_finaldat.to_csv(os.path.join(nest_folder_def, "matip_finaldat.csv"))

In [450]:
klavan_finaldat = compile_dat('008a6e6d', 'Ragnar-Klavan', 'Klavan', checkgames = True)
if isinstance(klavan_finaldat, pd.DataFrame):
    klavan_finaldat.to_csv(os.path.join(nest_folder_def, "klavan_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [451]:
jhenderson_finaldat = compile_dat('935e6b8f', 'Jordan-Henderson', 'Jordan_Henderson', checkgames = True)
if isinstance(jhenderson_finaldat, pd.DataFrame):
    jhenderson_finaldat.to_csv(os.path.join(nest_folder, "jhenderson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [452]:
coutinho_finaldat = compile_dat('0ef89a37', 'Philippe-Coutinho', 'Coutinho', checkgames = True)
if isinstance(coutinho_finaldat, pd.DataFrame):
    coutinho_finaldat.to_csv(os.path.join(nest_folder, "coutinho_finaldat.csv"))

In [453]:
can_finaldat = compile_dat('a65ce836', 'Emre-Can', 'Emre_Can', checkgames = True)
if isinstance(can_finaldat, pd.DataFrame):
    can_finaldat.to_csv(os.path.join(nest_folder, "can_finaldat.csv"))

In [454]:
mane_finaldat = compile_dat('c691bfe2', 'Sadio-Mane', 'Sadio_Man', checkgames = True)
if isinstance(mane_finaldat, pd.DataFrame):
    mane_finaldat.to_csv(os.path.join(nest_folder, "mane_finaldat.csv"))

In [455]:
wijnaldum_finaldat = compile_dat('eb58eef0', 'Georginio-Wijnaldum', 'Wijnaldum', checkgames = True)
if isinstance(wijnaldum_finaldat, pd.DataFrame):
    wijnaldum_finaldat.to_csv(os.path.join(nest_folder, "wijnaldum_finaldat.csv"))

In [456]:
firmino_finaldat = compile_dat('4c370d81', 'Roberto-Firmino', 'Firmino', checkgames = True)
if isinstance(firmino_finaldat, pd.DataFrame):
    firmino_finaldat.to_csv(os.path.join(nest_folder, "firmino_finaldat.csv"))

In [457]:
kompany_finaldat = compile_dat('d2bff301', 'Vincent-Kompany', 'Kompany', checkgames = True)
if isinstance(kompany_finaldat, pd.DataFrame):
    kompany_finaldat.to_csv(os.path.join(nest_folder_def, "kompany_finaldat.csv"))

In [458]:
otamendi_finaldat = compile_dat('0d267745', 'Nicolas-Otamendi', 'Otamendi', checkgames = True)
if isinstance(otamendi_finaldat, pd.DataFrame):
    otamendi_finaldat.to_csv(os.path.join(nest_folder_def, "otamendi_finaldat.csv"))

In [459]:
delph_finaldat = compile_dat('111c3236', 'Fabian-Delph', 'Fabian_Delph', checkgames = True)
if isinstance(delph_finaldat, pd.DataFrame):
    delph_finaldat.to_csv(os.path.join(nest_folder_def, "delph_finaldat.csv"))

In [460]:
dsilva_finaldat = compile_dat('e2716bd0', 'David-Silva', 'David_Silva', checkgames = True)
if isinstance(dsilva_finaldat, pd.DataFrame):
    dsilva_finaldat.to_csv(os.path.join(nest_folder, "dsilva_finaldat.csv"))

In [461]:
fernandinho_finaldat = compile_dat('101da2b5', 'Fernandinho', 'Luiz Rosa', checkgames = True)
if isinstance(fernandinho_finaldat, pd.DataFrame):
    fernandinho_finaldat.to_csv(os.path.join(nest_folder, "fernandinho_finaldat.csv"))

In [462]:
sane_finaldat = compile_dat('2b114be3', 'Leroy-Sane', 'Leroy_San', checkgames = True)
if isinstance(sane_finaldat, pd.DataFrame):
    sane_finaldat.to_csv(os.path.join(nest_folder, "sane_finaldat.csv"))

In [463]:
aguero_finaldat = compile_dat('4d034881', 'Sergio-Aguero', 'Sergio_Ag', checkgames = True)
if isinstance(aguero_finaldat, pd.DataFrame):
    aguero_finaldat.to_csv(os.path.join(nest_folder, "aguero_finaldat.csv"))

In [464]:
danilo_finaldat = compile_dat('94b2001f', 'Danilo', 'Danilo Luiz', checkgames = True)
if isinstance(danilo_finaldat, pd.DataFrame):
    danilo_finaldat.to_csv(os.path.join(nest_folder_def, "danilo_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [465]:
laporte_finaldat = compile_dat('119b9a8e', 'Aymeric-Laporte', 'Aymeric', checkgames = True)
if isinstance(laporte_finaldat, pd.DataFrame):
    laporte_finaldat.to_csv(os.path.join(nest_folder_def, "laporte_finaldat.csv"))

In [466]:
asanchez_finaldat = compile_dat('2c0f8e9e', 'Alexis-Sanchez', 'Alexis_S', checkgames = True)
if isinstance(asanchez_finaldat, pd.DataFrame):
    asanchez_finaldat.to_csv(os.path.join(nest_folder, "asanchez_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [467]:
matic_finaldat = compile_dat('eb7822d7', 'Nemanja-Matic', 'Nemanja_Mati', checkgames = True)
if isinstance(matic_finaldat, pd.DataFrame):
    matic_finaldat.to_csv(os.path.join(nest_folder, "matic_finaldat.csv"))

In [468]:
smalling_finaldat = compile_dat('b6964eb6', 'Chris-Smalling', 'Smalling', checkgames = True)
if isinstance(smalling_finaldat, pd.DataFrame):
    smalling_finaldat.to_csv(os.path.join(nest_folder_def, "smalling_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [469]:
pjones_finaldat = compile_dat('def899bd', 'Phil-Jones', 'Phil_Jones', checkgames = True)
if isinstance(pjones_finaldat, pd.DataFrame):
    pjones_finaldat.to_csv(os.path.join(nest_folder_def, "pjones_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [470]:
rojo_finaldat = compile_dat('e0991c04', 'Marcos-Rojo', 'Rojo', checkgames = True)
if isinstance(rojo_finaldat, pd.DataFrame):
    rojo_finaldat.to_csv(os.path.join(nest_folder_def, "rojo_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [471]:
valencia_finaldat = compile_dat('08aa17d0', 'Antonio-Valencia', 'Valencia', checkgames = True)
if isinstance(valencia_finaldat, pd.DataFrame):
    valencia_finaldat.to_csv(os.path.join(nest_folder_def, "valencia_finaldat.csv"))

In [473]:
bailly_finaldat = compile_dat('a1232f4e', 'Eric-Bailly', 'Bailly', checkgames = True)
if isinstance(bailly_finaldat, pd.DataFrame):
    bailly_finaldat.to_csv(os.path.join(nest_folder_def, "bailly_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [474]:
mata_finaldat = compile_dat('29733c72', 'Juan-Mata', 'Juan_Mata', checkgames = True)
if isinstance(mata_finaldat, pd.DataFrame):
    mata_finaldat.to_csv(os.path.join(nest_folder, "mata_finaldat.csv"))

In [475]:
herrera_finaldat = compile_dat('436a9dd0', 'Ander-Herrera', 'Herrera', checkgames = True)
if isinstance(herrera_finaldat, pd.DataFrame):
    herrera_finaldat.to_csv(os.path.join(nest_folder, "herrera_finaldat.csv"))

In [476]:
lingard_finaldat = compile_dat('810e3c74', 'Jesse-Lingard', 'Lingard', checkgames = True)
if isinstance(lingard_finaldat, pd.DataFrame):
    lingard_finaldat.to_csv(os.path.join(nest_folder, "lingard_finaldat.csv"))

In [477]:
martial_finaldat = compile_dat('8b788c01', 'Anthony-Martial', 'Martial', checkgames = True)
if isinstance(martial_finaldat, pd.DataFrame):
    martial_finaldat.to_csv(os.path.join(nest_folder, "martial_finaldat.csv"))

In [478]:
pogba_finaldat = compile_dat('867239d3', 'Paul-Pogba', 'Pogba', checkgames = True)
if isinstance(pogba_finaldat, pd.DataFrame):
    pogba_finaldat.to_csv(os.path.join(nest_folder, "pogba_finaldat.csv"))

In [479]:
lukaku_finaldat = compile_dat('5eae500a', 'Romelu-Lukaku', 'Romelu', checkgames = True)
if isinstance(lukaku_finaldat, pd.DataFrame):
    lukaku_finaldat.to_csv(os.path.join(nest_folder, "lukaku_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [480]:
kenedy_finaldat = compile_dat('5eebbfdf', 'Kenedy', 'Robert Kenedy', checkgames = True)
if isinstance(kenedy_finaldat, pd.DataFrame):
    kenedy_finaldat.to_csv(os.path.join(nest_folder, "kenedy_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [481]:
yedlin_finaldat = compile_dat('ae0a7ddf', 'DeAndre-Yedlin', 'Yedlin', checkgames = True)
if isinstance(yedlin_finaldat, pd.DataFrame):
    yedlin_finaldat.to_csv(os.path.join(nest_folder_def, "yedlin_finaldat.csv"))

In [483]:
cclark_finaldat = compile_dat('2b91cf8e', 'Ciaran-Clark', 'Ciaran_Clark', checkgames = True)
if isinstance(cclark_finaldat, pd.DataFrame):
    cclark_finaldat.to_csv(os.path.join(nest_folder_def, "cclark_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [484]:
dummett_finaldat = compile_dat('3fe6bd8c', 'Paul-Dummett', 'Dummett', checkgames = True)
if isinstance(dummett_finaldat, pd.DataFrame):
    dummett_finaldat.to_csv(os.path.join(nest_folder_def, "dummett_finaldat.csv"))


In [485]:
lejeune_finaldat = compile_dat('6eaed4eb', 'Florian-Lejeune', 'Lejeune', checkgames = True)
if isinstance(lejeune_finaldat, pd.DataFrame):
    lejeune_finaldat.to_csv(os.path.join(nest_folder_def, "lejeune_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [486]:
ritchie_finaldat = compile_dat('71ef519e', 'Matt-Ritchie', 'Matt_Ritchie', checkgames = True)
if isinstance(ritchie_finaldat, pd.DataFrame):
    ritchie_finaldat.to_csv(os.path.join(nest_folder, "ritchie_finaldat.csv"))

In [649]:
shelvey_finaldat = compile_dat('3727dd3c', 'Jonjo-Shelvey', 'Jonjo_Shelvey', checkgames = True)
if isinstance(shelvey_finaldat, pd.DataFrame):
    shelvey_finaldat.to_csv(os.path.join(nest_folder, "shelvey_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [488]:
diame_finaldat = compile_dat('b7d082af', 'Mohamed-Diame', 'Mohamed_Diam', checkgames = True)
if isinstance(diame_finaldat, pd.DataFrame):
    diame_finaldat.to_csv(os.path.join(nest_folder, "diame_finaldat.csv"))

In [489]:
atsu_finaldat = compile_dat('8816329f', 'Christian-Atsu', 'Christian_Atsu', checkgames = True)
if isinstance(atsu_finaldat, pd.DataFrame):
    atsu_finaldat.to_csv(os.path.join(nest_folder, "atsu_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [490]:
gayle_finaldat = compile_dat('e58a03c6', 'Dwight-Gayle', 'Gayle', checkgames = True)
if isinstance(gayle_finaldat, pd.DataFrame):
    gayle_finaldat.to_csv(os.path.join(nest_folder, "gayle_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [491]:
ayoze_finaldat = compile_dat('819aa8e7', 'Ayoze-Perez', 'Ayoze', checkgames = True)
if isinstance(ayoze_finaldat, pd.DataFrame):
    ayoze_finaldat.to_csv(os.path.join(nest_folder, "ayoze_finaldat.csv"))

In [492]:
manquillo_finaldat = compile_dat('758dd7f0', 'Javier-Manquillo', 'Manquillo', checkgames = True)
if isinstance(manquillo_finaldat, pd.DataFrame):
    manquillo_finaldat.to_csv(os.path.join(nest_folder_def, "manquillo_finaldat.csv"))

In [493]:
joselu_finaldat = compile_dat('6265208f', 'Joselu', 'Mato Sanmar', checkgames = True)
if isinstance(joselu_finaldat, pd.DataFrame):
    joselu_finaldat.to_csv(os.path.join(nest_folder, "joselu_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [494]:
mbemba_finaldat = compile_dat('e76cb9cc', 'Chancel-Mbemba', 'Mbemba', checkgames = True)
if isinstance(mbemba_finaldat, pd.DataFrame):
    mbemba_finaldat.to_csv(os.path.join(nest_folder_def, "mbemba_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [495]:
merino_finaldat = compile_dat('d080ed5e', 'Mikel-Merino', 'Merino', checkgames = True)
if isinstance(merino_finaldat, pd.DataFrame):
    merino_finaldat.to_csv(os.path.join(nest_folder, "merino_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [496]:
bertrand_finaldat = compile_dat('01226327', 'Ryan-Bertrand', 'Ryan_Bertrand', checkgames = True)
if isinstance(bertrand_finaldat, pd.DataFrame):
    bertrand_finaldat.to_csv(os.path.join(nest_folder_def, "bertrand_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [497]:
yoshida_finaldat = compile_dat('caffaf5a', 'Maya-Yoshida', 'Yoshida', checkgames = True)
if isinstance(yoshida_finaldat, pd.DataFrame):
    yoshida_finaldat.to_csv(os.path.join(nest_folder_def, "yoshida_finaldat.csv"))

In [650]:
cedric_finaldat = compile_dat('839c14e1', 'Cedric-Soares', 'dric_Soares', checkgames = False)
if isinstance(cedric_finaldat, pd.DataFrame):
    cedric_finaldat.to_csv(os.path.join(nest_folder_def, "cedric_finaldat.csv"))

In [499]:
sdavis_finaldat = compile_dat('cf15bbb1', 'Steven-Davis', 'Steven_Davis', checkgames = True)
if isinstance(sdavis_finaldat, pd.DataFrame):
    sdavis_finaldat.to_csv(os.path.join(nest_folder, "sdavis_finaldat.csv"))

In [500]:
tadic_finaldat = compile_dat('2475dcc6', 'Dusan-Tadic', 'an_Tadi', checkgames = True)
if isinstance(tadic_finaldat, pd.DataFrame):
    tadic_finaldat.to_csv(os.path.join(nest_folder, "tadic_finaldat.csv"))

In [501]:
romeu_finaldat = compile_dat('4b511457', 'Oriol-Romeu', 'Oriol', checkgames = True)
if isinstance(romeu_finaldat, pd.DataFrame):
    romeu_finaldat.to_csv(os.path.join(nest_folder, "romeu_finaldat.csv"))

In [502]:
boufal_finaldat = compile_dat('b0c71810', 'Sofiane-Boufal', 'Boufal', checkgames = True)
if isinstance(boufal_finaldat, pd.DataFrame):
    boufal_finaldat.to_csv(os.path.join(nest_folder, "boufal_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [503]:
redmond_finaldat = compile_dat('ab651565', 'Nathan-Redmond', 'Redmond', checkgames = True)
if isinstance(redmond_finaldat, pd.DataFrame):
    redmond_finaldat.to_csv(os.path.join(nest_folder, "redmond_finaldat.csv"))

In [504]:
slong_finaldat = compile_dat('9f472354', 'Shane-Long', 'Shane_Long', checkgames = True)
if isinstance(slong_finaldat, pd.DataFrame):
    slong_finaldat.to_csv(os.path.join(nest_folder, "slong_finaldat.csv"))

In [505]:
caustin_finaldat = compile_dat('8d228291', 'Charlie-Austin', 'Charlie_Austin', checkgames = True)
if isinstance(caustin_finaldat, pd.DataFrame):
    caustin_finaldat.to_csv(os.path.join(nest_folder, "caustin_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [506]:
gabbiadini_finaldat = compile_dat('8f866fe8', 'Manolo-Gabbiadini', 'Gabbiadini', checkgames = True)
if isinstance(gabbiadini_finaldat, pd.DataFrame):
    gabbiadini_finaldat.to_csv(os.path.join(nest_folder, "gabbiadini_finaldat.csv"))

In [507]:
hoedt_finaldat = compile_dat('875898c7', 'Wesley-Hoedt', 'Hoedt', checkgames = True)
if isinstance(hoedt_finaldat, pd.DataFrame):
    hoedt_finaldat.to_csv(os.path.join(nest_folder_def, "hoedt_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [508]:
shawcross_finaldat = compile_dat('285a3d44', 'Ryan-Shawcross', 'Shawcross', checkgames = True)
if isinstance(shawcross_finaldat, pd.DataFrame):
    shawcross_finaldat.to_csv(os.path.join(nest_folder_def, "shawcross_finaldat.csv"))

In [509]:
gcameron_finaldat = compile_dat('1a160ea3', 'Geoff-Cameron', 'Geoff_Cameron', checkgames = True)
if isinstance(gcameron_finaldat, pd.DataFrame):
    gcameron_finaldat.to_csv(os.path.join(nest_folder_def, "gcameron_finaldat.csv"))

In [510]:
pieters_finaldat = compile_dat('1ef37668', 'Erik-Pieters', 'Pieters', checkgames = True)
if isinstance(pieters_finaldat, pd.DataFrame):
    pieters_finaldat.to_csv(os.path.join(nest_folder_def, "pieters_finaldat.csv"))

In [511]:
jallen_finaldat = compile_dat('9feb8f24', 'Joe-Allen', 'Joe_Allen', checkgames = True)
if isinstance(jallen_finaldat, pd.DataFrame):
    jallen_finaldat.to_csv(os.path.join(nest_folder, "jallen_finaldat.csv"))

In [512]:
shaqiri_finaldat = compile_dat('6421ec64', 'Xherdan-Shaqiri', 'Shaqiri', checkgames = True)
if isinstance(shaqiri_finaldat, pd.DataFrame):
    shaqiri_finaldat.to_csv(os.path.join(nest_folder, "shaqiri_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [513]:
sobhi_finaldat = compile_dat('98be0d02', 'Ramadan-Sobhi', 'Sobhi', checkgames = True)
if isinstance(sobhi_finaldat, pd.DataFrame):
    sobhi_finaldat.to_csv(os.path.join(nest_folder, "sobhi_finaldat.csv"))

In [514]:
dfletcher_finaldat = compile_dat('184e4970', 'Darren-Fletcher', 'Darren_Fletcher', checkgames = True)
if isinstance(dfletcher_finaldat, pd.DataFrame):
    dfletcher_finaldat.to_csv(os.path.join(nest_folder, "dfletcher_finaldat.csv"))

In [515]:
crouch_finaldat = compile_dat('73d9bdc7', 'Peter-Crouch', 'Crouch', checkgames = True)
if isinstance(crouch_finaldat, pd.DataFrame):
    crouch_finaldat.to_csv(os.path.join(nest_folder, "crouch_finaldat.csv"))

In [516]:
biramdiouf_finaldat = compile_dat('7f4f3e49', 'Mame-Biram-Diouf', 'Mame Biram', checkgames = True)
if isinstance(biramdiouf_finaldat, pd.DataFrame):
    biramdiouf_finaldat.to_csv(os.path.join(nest_folder, "biramdiouf_finaldat.csv"))

In [517]:
wimmer_finaldat = compile_dat('faf55ce4', 'Kevin-Wimmer', 'Wimmer', checkgames = True)
if isinstance(wimmer_finaldat, pd.DataFrame):
    wimmer_finaldat.to_csv(os.path.join(nest_folder_def, "wimmer_finaldat.csv"))

In [518]:
choupomoting_finaldat = compile_dat('0e6bc38c', 'Eric-Maxim-Choupo-Moting', 'Choupo-Moting', checkgames = True)
if isinstance(choupomoting_finaldat, pd.DataFrame):
    choupomoting_finaldat.to_csv(os.path.join(nest_folder, "choupomoting_finaldat.csv"))

In [519]:
martinsindi_finaldat = compile_dat('7bc34048', 'Bruno-Martins-Indi', 'Martins Indi', checkgames = True)
if isinstance(martinsindi_finaldat, pd.DataFrame):
    martinsindi_finaldat.to_csv(os.path.join(nest_folder_def, "martinsindi_finaldat.csv"))


In [652]:
mbauer_finaldat = compile_dat('cfe6238d', 'Moritz-Bauer', 'Moritz_Bauer', checkgames = False)
if isinstance(mbauer_finaldat, pd.DataFrame):
    mbauer_finaldat.to_csv(os.path.join(nest_folder_def, "mbauer_finaldat.csv"))

In [521]:
bndiaye_finaldat = compile_dat('960acc0a', 'Badou-Ndiaye', 'Papa Alioune', checkgames = True)
if isinstance(bndiaye_finaldat, pd.DataFrame):
    bndiaye_finaldat.to_csv(os.path.join(nest_folder, "bndiaye_finaldat.csv"))

In [522]:
aking_finaldat = compile_dat('d790907f', 'Andy-King', 'Andy_King', checkgames = True)
if isinstance(aking_finaldat, pd.DataFrame):
    aking_finaldat.to_csv(os.path.join(nest_folder, "aking_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [523]:
naughton_finaldat = compile_dat('e860c42f', 'Kyle-Naughton', 'Naughton', checkgames = True)
if isinstance(naughton_finaldat, pd.DataFrame):
    naughton_finaldat.to_csv(os.path.join(nest_folder_def, "naughton_finaldat.csv"))

In [524]:
ffernandez_finaldat = compile_dat('f25b010f', 'Federico-Fernandez', 'Federico_Fern', checkgames = True)
if isinstance(ffernandez_finaldat, pd.DataFrame):
    ffernandez_finaldat.to_csv(os.path.join(nest_folder_def, "ffernandez_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [525]:
vanderhoorn_finaldat = compile_dat('25f50295', 'Mike-van-der-Hoorn', 'Hoorn', checkgames = True)
if isinstance(vanderhoorn_finaldat, pd.DataFrame):
    vanderhoorn_finaldat.to_csv(os.path.join(nest_folder_def, "vanderhoorn_finaldat.csv"))

In [526]:
mawson_finaldat = compile_dat('d368feb8', 'Alfie-Mawson', 'Mawson', checkgames = True)
if isinstance(mawson_finaldat, pd.DataFrame):
    mawson_finaldat.to_csv(os.path.join(nest_folder_def, "mawson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [527]:
molsson_finaldat = compile_dat('2f72f79e', 'Martin-Olsson', 'Martin_Olsson', checkgames = True)
if isinstance(molsson_finaldat, pd.DataFrame):
    molsson_finaldat.to_csv(os.path.join(nest_folder_def, "molsson_finaldat.csv"))

In [529]:
sungyeungki_finaldat = compile_dat('1d8c99b0', 'Ki-Sung-yeung', 'Sung-yueng', checkgames = True)
if isinstance(sungyeungki_finaldat, pd.DataFrame):
    sungyeungki_finaldat.to_csv(os.path.join(nest_folder, "sungyeungki_finaldat.csv"))

In [541]:
fer_finaldat = compile_dat('3cefa66a', 'Leroy-Fer', 'Leroy_Fer', checkgames = True)
if isinstance(fer_finaldat, pd.DataFrame):
    fer_finaldat.to_csv(os.path.join(nest_folder, "fer_finaldat.csv"))

In [542]:
ndyer_finaldat = compile_dat('6dfd5af0', 'Nathan-Dyer', 'Nathan_Dyer', checkgames = True)
if isinstance(ndyer_finaldat, pd.DataFrame):
    ndyer_finaldat.to_csv(os.path.join(nest_folder, "ndyer_finaldat.csv"))

In [543]:
tcarroll_finaldat = compile_dat('5a1900a4', 'Tom-Carroll', 'Tom_Carroll', checkgames = True)
if isinstance(tcarroll_finaldat, pd.DataFrame):
    tcarroll_finaldat.to_csv(os.path.join(nest_folder, "tcarroll_finaldat.csv"))

In [544]:
mesa_finaldat = compile_dat('7c4243d4', 'Roque-Mesa', 'Mesa', checkgames = True)
if isinstance(mesa_finaldat, pd.DataFrame):
    mesa_finaldat.to_csv(os.path.join(nest_folder, "mesa_finaldat.csv"))

In [545]:
abraham_finaldat = compile_dat('f586779e', 'Tammy-Abraham', 'Tammy_Abraham', checkgames = True)
if isinstance(abraham_finaldat, pd.DataFrame):
    abraham_finaldat.to_csv(os.path.join(nest_folder, "abraham_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [548]:
aayew_finaldat = compile_dat('5dbefa9d', 'Andre-Ayew', 'Andre_Ayew', checkgames = True)
if isinstance(aayew_finaldat, pd.DataFrame):
    aayew_finaldat.to_csv(os.path.join(nest_folder, "aayew_finaldat.csv"))

In [549]:
clucas_finaldat = compile_dat('987718f5', 'Sam-Clucas', 'Clucas', checkgames = True)
if isinstance(clucas_finaldat, pd.DataFrame):
    clucas_finaldat.to_csv(os.path.join(nest_folder, "clucas_finaldat.csv"))

In [550]:
renato_finaldat = compile_dat('032f894e', 'Renato-Sanches', 'Renato', checkgames = True)
if isinstance(renato_finaldat, pd.DataFrame):
    renato_finaldat.to_csv(os.path.join(nest_folder, "renato_finaldat.csv"))

In [551]:
bony_finaldat = compile_dat('61abe9b6', 'Wilfried-Bony', 'Wilfried_Bony', checkgames = True)
if isinstance(bony_finaldat, pd.DataFrame):
    bony_finaldat.to_csv(os.path.join(nest_folder, "bony_finaldat.csv"))

In [552]:
drose_finaldat = compile_dat('89d10e53', 'Danny-Rose', 'Danny_Rose', checkgames = True)
if isinstance(drose_finaldat, pd.DataFrame):
    drose_finaldat.to_csv(os.path.join(nest_folder_def, "drose_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [553]:
vertonghen_finaldat = compile_dat('ba23a904', 'Jan-Vertonghen', 'Vertonghen', checkgames = True)
if isinstance(vertonghen_finaldat, pd.DataFrame):
    vertonghen_finaldat.to_csv(os.path.join(nest_folder_def, "vertonghen_finaldat.csv"))

In [554]:
dier_finaldat = compile_dat('ac861941', 'Eric-Dier', 'Eric_Dier', checkgames = True)
if isinstance(dier_finaldat, pd.DataFrame):
    dier_finaldat.to_csv(os.path.join(nest_folder_def, "dier_finaldat.csv"))

In [555]:
mdembele_finaldat = compile_dat('f7ee69fb', 'Mousa-Dembele', 'Mousa_Demb', checkgames = True)
if isinstance(mdembele_finaldat, pd.DataFrame):
    mdembele_finaldat.to_csv(os.path.join(nest_folder, "mdembele_finaldat.csv"))

In [666]:
alli_finaldat = compile_dat('cea4ee8f', 'Dele-Alli', 'ele_Alli', checkgames = True)
if isinstance(alli_finaldat, pd.DataFrame):
    alli_finaldat.to_csv(os.path.join(nest_folder, "alli_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [557]:
wanyama_finaldat = compile_dat('e0900238', 'Victor-Wanyama', 'Wanyama', checkgames = True)
if isinstance(wanyama_finaldat, pd.DataFrame):
    wanyama_finaldat.to_csv(os.path.join(nest_folder, "wanyama_finaldat.csv"))

In [571]:
msissoko_finaldat = compile_dat('2acd49b9', 'Moussa-Sissoko', 'Moussa_Sissoko', checkgames = False)
if isinstance(msissoko_finaldat, pd.DataFrame):
    msissoko_finaldat.to_csv(os.path.join(nest_folder, "msissoko_finaldat.csv"))

In [559]:
kane_finaldat = compile_dat('21a66f6a', 'Harry-Kane', 'Harry_Kane', checkgames = True)
if isinstance(kane_finaldat, pd.DataFrame):
    kane_finaldat.to_csv(os.path.join(nest_folder, "kane_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [560]:
dsanchez_finaldat = compile_dat('da7b447d', 'Davinson-Sanchez', 'Davinson_S', checkgames = True)
if isinstance(dsanchez_finaldat, pd.DataFrame):
    dsanchez_finaldat.to_csv(os.path.join(nest_folder_def, "dsanchez_finaldat.csv"))

In [561]:
aurier_finaldat = compile_dat('5c2b4f07', 'Serge-Aurier', 'Aurier', checkgames = True)
if isinstance(aurier_finaldat, pd.DataFrame):
    aurier_finaldat.to_csv(os.path.join(nest_folder_def, "aurier_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [562]:
agray_finaldat = compile_dat('c24dba2a', 'Andre-Gray', 'Andre_Gray', checkgames = True)
if isinstance(agray_finaldat, pd.DataFrame):
    agray_finaldat.to_csv(os.path.join(nest_folder, "agray_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [563]:
prodl_finaldat = compile_dat('f5e6d08d', 'Sebastian-Prodl', 'Sebastian_Pr', checkgames = True)
if isinstance(prodl_finaldat, pd.DataFrame):
    prodl_finaldat.to_csv(os.path.join(nest_folder_def, "prodl_finaldat.csv"))

In [564]:
holebas_finaldat = compile_dat('12aa7661', 'Jose-Holebas', 'Holebas', checkgames = True)
if isinstance(holebas_finaldat, pd.DataFrame):
    holebas_finaldat.to_csv(os.path.join(nest_folder_def, "holebas_finaldat.csv"))

In [565]:
britos_finaldat = compile_dat('356a9a62', 'Miguel-Britos', 'Britos', checkgames = True)
if isinstance(britos_finaldat, pd.DataFrame):
    britos_finaldat.to_csv(os.path.join(nest_folder_def, "britos_finaldat.csv"))

In [566]:
kabasele_finaldat = compile_dat('914ab9a1', 'Christian-Kabasele', 'Kabasele', checkgames = True)
if isinstance(kabasele_finaldat, pd.DataFrame):
    kabasele_finaldat.to_csv(os.path.join(nest_folder_def, "kabasele_finaldat.csv"))

In [567]:
janmaat_finaldat = compile_dat('45da6694', 'Daryl-Janmaat', 'Janmaat', checkgames = True)
if isinstance(janmaat_finaldat, pd.DataFrame):
    janmaat_finaldat.to_csv(os.path.join(nest_folder_def, "janmaat_finaldat.csv"))

In [568]:
mariappa_finaldat = compile_dat('8aa5f52c', 'Adrian-Mariappa', 'Mariappa', checkgames = True)
if isinstance(mariappa_finaldat, pd.DataFrame):
    mariappa_finaldat.to_csv(os.path.join(nest_folder_def, "mariappa_finaldat.csv"))

In [569]:
femenia_finaldat = compile_dat('e5d01ecf', 'Kiko-Femenia', 'Francisco_Femen', checkgames = True)
if isinstance(femenia_finaldat, pd.DataFrame):
    femenia_finaldat.to_csv(os.path.join(nest_folder_def, "femenia_finaldat.csv"))

In [532]:
cleverley_finaldat = compile_dat('6cdd8245', 'Tom-Cleverley', 'Cleverley', checkgames = True)
if isinstance(cleverley_finaldat, pd.DataFrame):
    cleverley_finaldat.to_csv(os.path.join(nest_folder, "cleverley_finaldat.csv"))

In [533]:
capoue_finaldat = compile_dat('5acc4a10', 'Etienne-Capoue', 'Capoue', checkgames = True)
if isinstance(capoue_finaldat, pd.DataFrame):
    capoue_finaldat.to_csv(os.path.join(nest_folder, "capoue_finaldat.csv"))

In [534]:
rpereyra_finaldat = compile_dat('75f2c59f', 'Roberto-Pereyra', 'Roberto_Pereyra', checkgames = True)
if isinstance(rpereyra_finaldat, pd.DataFrame):
    rpereyra_finaldat.to_csv(os.path.join(nest_folder, "rpereyra_finaldat.csv"))

In [535]:
deeney_finaldat = compile_dat('82e34b69', 'Troy-Deeney', 'Troy_Deeney', checkgames = True)
if isinstance(deeney_finaldat, pd.DataFrame):
    deeney_finaldat.to_csv(os.path.join(nest_folder, "deeney_finaldat.csv"))

In [667]:
carrillo_finaldat = compile_dat('7527558b', 'Andre-Carrillo', 'Andre_Carrillo', checkgames = True)
if isinstance(carrillo_finaldat, pd.DataFrame):
    carrillo_finaldat.to_csv(os.path.join(nest_folder, "carrillo_finaldat.csv"))

In [537]:
zeegelaar_finaldat = compile_dat('8abd0c03', 'Marvin-Zeegelaar', 'Zeegelaar', checkgames = True)
if isinstance(zeegelaar_finaldat, pd.DataFrame):
    zeegelaar_finaldat.to_csv(os.path.join(nest_folder_def, "zeegelaar_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [538]:
gibbs_finaldat = compile_dat('46ee5234', 'Kieran-Gibbs', 'Gibbs', checkgames = True)
if isinstance(gibbs_finaldat, pd.DataFrame):
    gibbs_finaldat.to_csv(os.path.join(nest_folder_def, "gibbs_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [570]:
gbarry_finaldat = compile_dat('7dbbe367', 'Gareth-Barry', 'Gareth_Barry', checkgames = True)
if isinstance(gbarry_finaldat, pd.DataFrame):
    gbarry_finaldat.to_csv(os.path.join(nest_folder, "gbarry_finaldat.csv"))

In [572]:
nyom_finaldat = compile_dat('97572117', 'Allan-Nyom', 'Nyom', checkgames = True)
if isinstance(nyom_finaldat, pd.DataFrame):
    nyom_finaldat.to_csv(os.path.join(nest_folder_def, "nyom_finaldat.csv"))

In [573]:
livermore_finaldat = compile_dat('b98570d1', 'Jake-Livermore', 'Jake_Livermore', checkgames = True)
if isinstance(livermore_finaldat, pd.DataFrame):
    livermore_finaldat.to_csv(os.path.join(nest_folder, "livermore_finaldat.csv"))

In [575]:
yacob_finaldat = compile_dat('bda6ffd4', 'Claudio-Yacob', 'Yacob', checkgames = True)
if isinstance(yacob_finaldat, pd.DataFrame):
    yacob_finaldat.to_csv(os.path.join(nest_folder, "yacob_finaldat.csv"))

In [576]:
mcclean_finaldat = compile_dat('53538b39', 'James-McClean', 'McClean', checkgames = True)
if isinstance(mcclean_finaldat, pd.DataFrame):
    mcclean_finaldat.to_csv(os.path.join(nest_folder, "mcclean_finaldat.csv"))

In [646]:
mphillips_finaldat = compile_dat('7bb5f40e', 'Matt-Phillips', 'Phillips', checkgames = True)
if isinstance(mphillips_finaldat, pd.DataFrame):
    mphillips_finaldat.to_csv(os.path.join(nest_folder, "mphillips_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


The number of Premier League games played by Matt-Phillips(63) does not match the number of rows in the dataframe (45)


In [578]:
brunt_finaldat = compile_dat('ec7e6ad1', 'Chris-Brunt', 'Brunt', checkgames = True)
if isinstance(brunt_finaldat, pd.DataFrame):
    brunt_finaldat.to_csv(os.path.join(nest_folder, "brunt_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [668]:
rondon_finaldat = compile_dat('b318a643', 'Salomon-Rondon', 'n_Rond', checkgames = False)
if isinstance(rondon_finaldat, pd.DataFrame):
    rondon_finaldat.to_csv(os.path.join(nest_folder, "rondon_finaldat.csv"))

In [580]:
robsonkanu_finaldat = compile_dat('fb2dec57', 'Hal-Robson-Kanu', 'Hal_Robson', checkgames = True)
if isinstance(robsonkanu_finaldat, pd.DataFrame):
    robsonkanu_finaldat.to_csv(os.path.join(nest_folder, "robsonkanu_finaldat.csv"))

In [581]:
jrodriguez_finaldat = compile_dat('4ab53cdb', 'Jay-Rodriguez', 'Jay_Rodriguez', checkgames = True)
if isinstance(jrodriguez_finaldat, pd.DataFrame):
    jrodriguez_finaldat.to_csv(os.path.join(nest_folder, "jrodriguez_finaldat.csv"))

In [669]:
hegazi_finaldat = compile_dat('5540c9dc', 'Ahmed-Hegazi', 'Hegazi', checkgames = True)
if isinstance(hegazi_finaldat, pd.DataFrame):
    hegazi_finaldat.to_csv(os.path.join(nest_folder_def, "hegazi_finaldat.csv"))

In [670]:
krychowiak_finaldat = compile_dat('580d52dd', 'Grzegorz-Krychowiak', 'Krychowiak', checkgames = False)
if isinstance(krychowiak_finaldat, pd.DataFrame):
    krychowiak_finaldat.to_csv(os.path.join(nest_folder, "krychowiak_finaldat.csv"))

In [584]:
arnautovic_finaldat = compile_dat('00459419', 'Marko-Arnautovic', 'Marko_Arnautovi', checkgames = True)
if isinstance(arnautovic_finaldat, pd.DataFrame):
    arnautovic_finaldat.to_csv(os.path.join(nest_folder, "arnautovic_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [585]:
fonte_finaldat = compile_dat('90f87966', 'Jose-Fonte', 'Fonte', checkgames = True)
if isinstance(fonte_finaldat, pd.DataFrame):
    fonte_finaldat.to_csv(os.path.join(nest_folder_def, "fonte_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [586]:
jcollins_finaldat = compile_dat('c61f1719', 'James-Collins', 'James_Collins', checkgames = True)
if isinstance(jcollins_finaldat, pd.DataFrame):
    jcollins_finaldat.to_csv(os.path.join(nest_folder_def, "jcollins_finaldat.csv"))

In [587]:
wreid_finaldat = compile_dat('562eed5c', 'Winston-Reid', 'Winston_Reid', checkgames = True)
if isinstance(wreid_finaldat, pd.DataFrame):
    wreid_finaldat.to_csv(os.path.join(nest_folder_def, "wreid_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [589]:
ogbonna_finaldat = compile_dat('74b2cd7e', 'Angelo-Ogbonna', 'Ogbonna', checkgames = True)
if isinstance(ogbonna_finaldat, pd.DataFrame):
    ogbonna_finaldat.to_csv(os.path.join(nest_folder_def, "ogbonna_finaldat.csv"))

In [590]:
masuaku_finaldat = compile_dat('57df7a11', 'Arthur-Masuaku', 'Masuaku', checkgames = True)
if isinstance(masuaku_finaldat, pd.DataFrame):
    masuaku_finaldat.to_csv(os.path.join(nest_folder_def, "masuaku_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [591]:
zabaleta_finaldat = compile_dat('6813ac34', 'Pablo-Zabaleta', 'Zabaleta', checkgames = True)
if isinstance(zabaleta_finaldat, pd.DataFrame):
    zabaleta_finaldat.to_csv(os.path.join(nest_folder_def, "zabaleta_finaldat.csv"))

In [592]:
mnoble_finaldat = compile_dat('ff527768', 'Mark-Noble', 'Mark_Noble', checkgames = True)
if isinstance(mnoble_finaldat, pd.DataFrame):
    mnoble_finaldat.to_csv(os.path.join(nest_folder, "mnoble_finaldat.csv"))

In [594]:
kouyate_finaldat = compile_dat('8d3c902d', 'Cheikhou-Kouyate', 'Cheikhou_Kouyat', checkgames = True)
if isinstance(kouyate_finaldat, pd.DataFrame):
    kouyate_finaldat.to_csv(os.path.join(nest_folder, "kouyate_finaldat.csv"))

In [595]:
obiang_finaldat = compile_dat('ba1e04b3', 'Pedro-Obiang', 'Obiang', checkgames = True)
if isinstance(obiang_finaldat, pd.DataFrame):
    obiang_finaldat.to_csv(os.path.join(nest_folder, "obiang_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [596]:
lanzini_finaldat = compile_dat('26f6dca8', 'Manuel-Lanzini', 'Lanzini', checkgames = True)
if isinstance(lanzini_finaldat, pd.DataFrame):
    lanzini_finaldat.to_csv(os.path.join(nest_folder, "lanzini_finaldat.csv"))

In [597]:
efernandes_finaldat = compile_dat('3bb625ac', 'Edimilson-Fernandes', 'Edimilson_Fernandes', checkgames = True)
if isinstance(efernandes_finaldat, pd.DataFrame):
    efernandes_finaldat.to_csv(os.path.join(nest_folder, "efernandes_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [598]:
carroll_finaldat = compile_dat('b581987a', 'Andy-Carroll', 'Andy_Carroll', checkgames = True)
if isinstance(carroll_finaldat, pd.DataFrame):
    carroll_finaldat.to_csv(os.path.join(nest_folder, "carroll_finaldat.csv"))


In [599]:
jhernandez_finaldat = compile_dat('189cee7b', 'Javier-Hernandez', 'Javier_Hern', checkgames = True)
if isinstance(jhernandez_finaldat, pd.DataFrame):
    jhernandez_finaldat.to_csv(os.path.join(nest_folder, "jhernandez_finaldat.csv"))

In [671]:
jmario_finaldat = compile_dat('f0021247', 'Joao-Mario', 'Naval Costa Eduardo', checkgames = False)
if isinstance(jmario_finaldat, pd.DataFrame):
    jmario_finaldat.to_csv(os.path.join(nest_folder, "jmario_finaldat.csv"))

In [601]:
lichtsteiner_finaldat = compile_dat('99b1d0e6', 'Stephan-Lichtsteiner', 'Lichtsteiner', checkgames = True)
if isinstance(lichtsteiner_finaldat, pd.DataFrame):
    lichtsteiner_finaldat.to_csv(os.path.join(nest_folder_def, "lichtsteiner_finaldat.csv"))

In [602]:
sokratis_finaldat = compile_dat('b682dcf6', 'Sokratis-Papastathopoulos', 'Papastathopoulos', checkgames = True)
if isinstance(sokratis_finaldat, pd.DataFrame):
    sokratis_finaldat.to_csv(os.path.join(nest_folder_def, "sokratis_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [603]:
torreira_finaldat = compile_dat('52611641', 'Lucas-Torreira', 'Lucas_Torreira', checkgames = True)
if isinstance(torreira_finaldat, pd.DataFrame):
    torreira_finaldat.to_csv(os.path.join(nest_folder, "torreira_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [604]:
guendouzi_finaldat = compile_dat('5ea8ef20', 'Matteo-Guendouzi', 'Guendouzi', checkgames = True)
if isinstance(guendouzi_finaldat, pd.DataFrame):
    guendouzi_finaldat.to_csv(os.path.join(nest_folder, "guendouzi_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [605]:
locadia_finaldat = compile_dat('6cc47ae1', 'Jurgen-Locadia', 'Locadia', checkgames = True)
if isinstance(locadia_finaldat, pd.DataFrame):
    locadia_finaldat.to_csv(os.path.join(nest_folder, "locadia_finaldat.csv"))

In [606]:
andone_finaldat = compile_dat('dc008610', 'Florin-Andone', 'Andone', checkgames = True)
if isinstance(andone_finaldat, pd.DataFrame):
    andone_finaldat.to_csv(os.path.join(nest_folder, "andone_finaldat.csv"))

In [607]:
bernardof_finaldat = compile_dat('17370b95', 'Bernardo', 'Bernardo_Fernandes da Silva Junior', checkgames = True)
if isinstance(bernardof_finaldat, pd.DataFrame):
    bernardof_finaldat.to_csv(os.path.join(nest_folder_def, "bernardof_finaldat.csv"))

In [608]:
alireza_finaldat = compile_dat('08452314', 'Alireza-Jahanbakhsh', 'Alireza', checkgames = True)
if isinstance(alireza_finaldat, pd.DataFrame):
    alireza_finaldat.to_csv(os.path.join(nest_folder, "alireza_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [609]:
montoya_finaldat = compile_dat('5cec2487', 'Martin-Montoya', 'Montoya', checkgames = True)
if isinstance(montoya_finaldat, pd.DataFrame):
    montoya_finaldat.to_csv(os.path.join(nest_folder_def, "montoya_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [610]:
morrison_finaldat = compile_dat('09fec029', 'Sean-Morrison', 'Sean_Morrison', checkgames = True)
if isinstance(morrison_finaldat, pd.DataFrame):
    morrison_finaldat.to_csv(os.path.join(nest_folder_def, "morrison_finaldat.csv"))

In [611]:
sbamba_finaldat = compile_dat('5534217d', 'Sol-Bamba', 'Sol_Bamba', checkgames = True)
if isinstance(sbamba_finaldat, pd.DataFrame):
    sbamba_finaldat.to_csv(os.path.join(nest_folder_def, "sbamba_finaldat.csv"))

In [612]:
manga_finaldat = compile_dat('72a0fb65', 'Bruno-Ecuele-Manga', 'Bruno_Ecuele', checkgames = True)
if isinstance(manga_finaldat, pd.DataFrame):
    manga_finaldat.to_csv(os.path.join(nest_folder_def, "manga_finaldat.csv"))

In [613]:
bennett_finaldat = compile_dat('dc936144', 'Joe-Bennett', 'Joe_Bennett', checkgames = True)
if isinstance(bennett_finaldat, pd.DataFrame):
    bennett_finaldat.to_csv(os.path.join(nest_folder_def, "bennett_finaldat.csv"))

In [614]:
peltier_finaldat = compile_dat('af341b88', 'Lee-Peltier', 'Peltier', checkgames = True)
if isinstance(peltier_finaldat, pd.DataFrame):
    peltier_finaldat.to_csv(os.path.join(nest_folder_def, "peltier_finaldat.csv"))

In [615]:
ralls_finaldat = compile_dat('3730fcbc', 'Joe-Ralls', 'Joe_Ralls', checkgames = True)
if isinstance(ralls_finaldat, pd.DataFrame):
    ralls_finaldat.to_csv(os.path.join(nest_folder, "ralls_finaldat.csv"))

In [616]:
hoilett_finaldat = compile_dat('33af5b9e', 'Junior-Hoilett', 'Hoilett', checkgames = True)
if isinstance(hoilett_finaldat, pd.DataFrame):
    hoilett_finaldat.to_csv(os.path.join(nest_folder, "hoilett_finaldat.csv"))

In [617]:
mendezlaing_finaldat = compile_dat('b4b2f632', 'Nathaniel-Mendez-Laing', 'Mendez-Laing', checkgames = True)
if isinstance(mendezlaing_finaldat, pd.DataFrame):
    mendezlaing_finaldat.to_csv(os.path.join(nest_folder, "mendezlaing_finaldat.csv"))

In [618]:
paterson_finaldat = compile_dat('d2583bce', 'Callum-Paterson', 'Paterson', checkgames = True)
if isinstance(paterson_finaldat, pd.DataFrame):
    paterson_finaldat.to_csv(os.path.join(nest_folder, "paterson_finaldat.csv"))

In [619]:
gunnarsson_finaldat = compile_dat('e2533c20', 'Aron-Gunnarsson', 'Gunnarsson', checkgames = True)
if isinstance(gunnarsson_finaldat, pd.DataFrame):
    gunnarsson_finaldat.to_csv(os.path.join(nest_folder, "gunnarsson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [620]:
joshmurphy_finaldat = compile_dat('b30c7426', 'Josh-Murphy', 'Josh_Murphy', checkgames = True)
if isinstance(joshmurphy_finaldat, pd.DataFrame):
    joshmurphy_finaldat.to_csv(os.path.join(nest_folder, "joshmurphy_finaldat.csv"))

In [621]:
zohore_finaldat = compile_dat('5e508d0d', 'Kenneth-Zohore', 'Zohore', checkgames = True)
if isinstance(zohore_finaldat, pd.DataFrame):
    zohore_finaldat.to_csv(os.path.join(nest_folder, "zohore_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [622]:
camarasa_finaldat = compile_dat('b2d03887', 'Victor-Camarasa', 'Camarasa', checkgames = True)
if isinstance(camarasa_finaldat, pd.DataFrame):
    camarasa_finaldat.to_csv(os.path.join(nest_folder, "camarasa_finaldat.csv"))

In [623]:
higuain_finaldat = compile_dat('8a08491a', 'Gonzalo-Higuain', 'Gonzalo_Higua', checkgames = True)
if isinstance(higuain_finaldat, pd.DataFrame):
    higuain_finaldat.to_csv(os.path.join(nest_folder, "higuain_finaldat.csv"))

In [624]:
meyer_finaldat = compile_dat('eba42d2b', 'Max-Meyer', 'Meyer', checkgames = True)
if isinstance(meyer_finaldat, pd.DataFrame):
    meyer_finaldat.to_csv(os.path.join(nest_folder, "meyer_finaldat.csv"))

In [625]:
batshuayi_finaldat = compile_dat('2973d8ff', 'Michy-Batshuayi', 'Batshuayi', checkgames = True)
if isinstance(batshuayi_finaldat, pd.DataFrame):
    batshuayi_finaldat.to_csv(os.path.join(nest_folder, "batshuayi_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [626]:
mina_finaldat = compile_dat('6d5701f2', 'Yerry-Mina', 'Yerry', checkgames = True)
if isinstance(mina_finaldat, pd.DataFrame):
    mina_finaldat.to_csv(os.path.join(nest_folder_def, "mina_finaldat.csv"))

In [627]:
bernard_finaldat = compile_dat('ab4b3e35', 'Bernard', 'Caldeira Duarte', checkgames = False)
if isinstance(bernard_finaldat, pd.DataFrame):
    bernard_finaldat.to_csv(os.path.join(nest_folder, "bernard_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [628]:
agomes_finaldat = compile_dat('1961b2aa', 'Anazdre-Gomes', 'Tavares Gomes', checkgames = True)
if isinstance(agomes_finaldat, pd.DataFrame):
    agomes_finaldat.to_csv(os.path.join(nest_folder, "agomes_finaldat.csv"))

In [672]:
odoi_finaldat = compile_dat('f7750aa0', 'Denis-Odoi', 'Denis_Odoi', checkgames = True)
if isinstance(odoi_finaldat, pd.DataFrame):
    odoi_finaldat.to_csv(os.path.join(nest_folder_def, "odoi_finaldat.csv"))

In [630]:
kmcdonald_finaldat = compile_dat('71a08f97', 'Kevin-McDonald', 'Kevin_McDonald', checkgames = True)
if isinstance(kmcdonald_finaldat, pd.DataFrame):
    kmcdonald_finaldat.to_csv(os.path.join(nest_folder, "kmcdonald_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [631]:
akamara_finaldat = compile_dat('4047c976', 'Aboubakar-Kamara', 'Aboubakar_Kamara', checkgames = True)
if isinstance(akamara_finaldat, pd.DataFrame):
    akamara_finaldat.to_csv(os.path.join(nest_folder, "akamara_finaldat.csv"))

In [632]:
lemarchand_finaldat = compile_dat('321a71a7', 'Maxime-Le-Marchand', 'Marchand', checkgames = True)
if isinstance(lemarchand_finaldat, pd.DataFrame):
    lemarchand_finaldat.to_csv(os.path.join(nest_folder_def, "lemarchand_finaldat.csv"))

In [633]:
seri_finaldat = compile_dat('10efd0e1', 'Jean-Seri', 'Michael_Seri', checkgames = True)
if isinstance(seri_finaldat, pd.DataFrame):
    seri_finaldat.to_csv(os.path.join(nest_folder, "seri_finaldat.csv"))

In [634]:
cchristie_finaldat = compile_dat('7eb67dad', 'Cyrus-Christie', 'Cyrus_Christie', checkgames = True)
if isinstance(cchristie_finaldat, pd.DataFrame):
    cchristie_finaldat.to_csv(os.path.join(nest_folder_def, "cchristie_finaldat.csv"))

In [673]:
schurrle_finaldat = compile_dat('ecbb3fdf', 'Andre-Schurrle', 'Andre_Sch', checkgames = False)
if isinstance(schurrle_finaldat, pd.DataFrame):
    schurrle_finaldat.to_csv(os.path.join(nest_folder, "schurrle_finaldat.csv"))

In [636]:
mitrovic_finaldat = compile_dat('3925dbd6', 'Aleksandar-Mitrovic', 'Mitrovi', checkgames = True)
if isinstance(mitrovic_finaldat, pd.DataFrame):
    mitrovic_finaldat.to_csv(os.path.join(nest_folder, "mitrovic_finaldat.csv"))

In [637]:
jbryan_finaldat = compile_dat('af69e7f0', 'Joe-Bryan', 'Joe_Bryan', checkgames = True)
if isinstance(jbryan_finaldat, pd.DataFrame):
    jbryan_finaldat.to_csv(os.path.join(nest_folder_def, "jbryan_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [638]:
anguissa_finaldat = compile_dat('6bb2c084', 'Andre-Frank-Zambo-Anguissa', 'Anguissa', checkgames = True)
if isinstance(anguissa_finaldat, pd.DataFrame):
    anguissa_finaldat.to_csv(os.path.join(nest_folder, "anguissa_finaldat.csv"))

In [639]:
vietto_finaldat = compile_dat('2513a588', 'Luciano-Vietto', 'Vietto', checkgames = True)
if isinstance(vietto_finaldat, pd.DataFrame):
    vietto_finaldat.to_csv(os.path.join(nest_folder, "vietto_finaldat.csv"))

In [640]:
babel_finaldat = compile_dat('6438d426', 'Ryan-Babel', 'Ryan_Babel', checkgames = True)
if isinstance(babel_finaldat, pd.DataFrame):
    babel_finaldat.to_csv(os.path.join(nest_folder, "babel_finaldat.csv"))

In [641]:
bacuna_finaldat = compile_dat('aaf1bf05', 'Juninho-Bacuna', 'Bacuna', checkgames = True)
if isinstance(bacuna_finaldat, pd.DataFrame):
    bacuna_finaldat.to_csv(os.path.join(nest_folder, "bacuna_finaldat.csv"))

In [642]:
durm_finaldat = compile_dat('cf623c78', 'Erik-Durm', 'Erik_Durm', checkgames = True)
if isinstance(durm_finaldat, pd.DataFrame):
    durm_finaldat.to_csv(os.path.join(nest_folder_def, "durm_finaldat.csv"))

In [643]:
mbenza_finaldat = compile_dat('aa1a8c7e', 'Issac-Mbenza', 'Mbenza', checkgames = True)
if isinstance(mbenza_finaldat, pd.DataFrame):
    mbenza_finaldat.to_csv(os.path.join(nest_folder, "mbenza_finaldat.csv"))

In [644]:
jgstankovic_finaldat = compile_dat('9c74fa25', 'Jon-Gorenc-Stankovic', 'Jon Gorenc', checkgames = True)
if isinstance(jgstankovic_finaldat, pd.DataFrame):
    jgstankovic_finaldat.to_csv(os.path.join(nest_folder_def, "jgstankovic_finaldat.csv"))

In [645]:
kgrant_finaldat = compile_dat('87f7500e', 'Karlan-Grant', 'Karlan_Grant', checkgames = True)
if isinstance(kgrant_finaldat, pd.DataFrame):
    kgrant_finaldat.to_csv(os.path.join(nest_folder, "kgrant_finaldat.csv"))

In [683]:
amartey_finaldat = compile_dat('1a4ef233', 'Daniel-Amartey', 'Amartey', checkgames = True)
if isinstance(amartey_finaldat, pd.DataFrame):
    amartey_finaldat.to_csv(os.path.join(nest_folder_def, "amartey_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


The number of Premier League games played by Daniel-Amartey(77) does not match the number of rows in the dataframe (78)


In [684]:
ghezzal_finaldat = compile_dat('dc3a9a0d', 'Rachid-Ghezzal', 'Ghezzal', checkgames = True)
if isinstance(ghezzal_finaldat, pd.DataFrame):
    ghezzal_finaldat.to_csv(os.path.join(nest_folder, "ghezzal_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [685]:
nmendy_finaldat = compile_dat('1598599e', 'Nampalys-Mendy', 'Nampalys_Mendy', checkgames = True)
if isinstance(nmendy_finaldat, pd.DataFrame):
    nmendy_finaldat.to_csv(os.path.join(nest_folder, "nmendy_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [686]:
fabinho_finaldat = compile_dat('7f3b388c', 'Fabinho', 'Henrique_Tavares', checkgames = True)
if isinstance(fabinho_finaldat, pd.DataFrame):
    fabinho_finaldat.to_csv(os.path.join(nest_folder, "fabinho_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [687]:
nkeita_finaldat = compile_dat('f25c8e3a', 'Naby-Keita', 'Naby_Ke', checkgames = True)
if isinstance(nkeita_finaldat, pd.DataFrame):
    nkeita_finaldat.to_csv(os.path.join(nest_folder, "nkeita_finaldat.csv"))

In [688]:
bmendy_finaldat = compile_dat('5271c8f1', 'Benjamin-Mendy', 'Benjamin_Mendy', checkgames = True)
if isinstance(bmendy_finaldat, pd.DataFrame):
    bmendy_finaldat.to_csv(os.path.join(nest_folder_def, "bmendy_finaldat.csv"))

In [689]:
fellaini_finaldat = compile_dat('87575fd9', 'Marouane-Fellaini', 'Fellaini', checkgames = True)
if isinstance(fellaini_finaldat, pd.DataFrame):
    fellaini_finaldat.to_csv(os.path.join(nest_folder, "fellaini_finaldat.csv"))

In [690]:
fred_finaldat = compile_dat('b853e0ad', 'Fred', 'Frederico_Rodrigues de Paula Santos', checkgames = True)
if isinstance(fred_finaldat, pd.DataFrame):
    fred_finaldat.to_csv(os.path.join(nest_folder, "fred_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [691]:
sarmstrong_finaldat = compile_dat('97333cf5', 'Stuart-Armstrong', 'Stuart_Armstrong', checkgames = True)
if isinstance(sarmstrong_finaldat, pd.DataFrame):
    sarmstrong_finaldat.to_csv(os.path.join(nest_folder, "sarmstrong_finaldat.csv"))

In [692]:
elyounoussi_finaldat = compile_dat('68dc0dac', 'Mohamed-Elyounoussi', 'Elyounoussi', checkgames = True)
if isinstance(elyounoussi_finaldat, pd.DataFrame):
    elyounoussi_finaldat.to_csv(os.path.join(nest_folder, "elyounoussi_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [693]:
valery_finaldat = compile_dat('531a4aa8', 'Yan-Valery', 'Valery', checkgames = True)
if isinstance(valery_finaldat, pd.DataFrame):
    valery_finaldat.to_csv(os.path.join(nest_folder_def, "valery_finaldat.csv"))

In [694]:
lucasmoura_finaldat = compile_dat('2b622f01', 'Lucas-Moura', 'Lucas_Rodrigues Moura da Silva', checkgames = True)
if isinstance(lucasmoura_finaldat, pd.DataFrame):
    lucasmoura_finaldat.to_csv(os.path.join(nest_folder, "lucasmoura_finaldat.csv"))

In [695]:
fllorente_finaldat = compile_dat('82dbf623', 'Fernando-Llorente', 'Fernando_Llorente', checkgames = True)
if isinstance(fllorente_finaldat, pd.DataFrame):
    fllorente_finaldat.to_csv(os.path.join(nest_folder, "fllorente_finaldat.csv"))

In [696]:
foyth_finaldat = compile_dat('6c7762c3', 'Juan-Foyth', 'Foyth', checkgames = True)
if isinstance(foyth_finaldat, pd.DataFrame):
    foyth_finaldat.to_csv(os.path.join(nest_folder_def, "foyth_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [697]:
cathcart_finaldat = compile_dat('89e4b358', 'Craig-Cathcart', 'Craig_Cathcart', checkgames = True)
if isinstance(cathcart_finaldat, pd.DataFrame):
    cathcart_finaldat.to_csv(os.path.join(nest_folder_def, "cathcart_finaldat.csv"))

In [698]:
masina_finaldat = compile_dat('de0f152c', 'Adam-Masina', 'Adam_Masina', checkgames = True)
if isinstance(masina_finaldat, pd.DataFrame):
    masina_finaldat.to_csv(os.path.join(nest_folder_def, "masina_finaldat.csv"))

In [699]:
deulofeu_finaldat = compile_dat('39583cfd', 'Gerard-Deulofeu', 'Deulofeu', checkgames = True)
if isinstance(deulofeu_finaldat, pd.DataFrame):
    deulofeu_finaldat.to_csv(os.path.join(nest_folder, "deulofeu_finaldat.csv"))

In [700]:
sema_finaldat = compile_dat('1770bd68', 'Ken-Sema', 'Ken_Sema', checkgames = True)
if isinstance(sema_finaldat, pd.DataFrame):
    sema_finaldat.to_csv(os.path.join(nest_folder, "sema_finaldat.csv"))

In [701]:
success_finaldat = compile_dat('f573a8bf', 'Issac-Success', 'Success', checkgames = True)
if isinstance(success_finaldat, pd.DataFrame):
    success_finaldat.to_csv(os.path.join(nest_folder, "success_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


The number of Premier League games played by Issac-Success(35) does not match the number of rows in the dataframe (5)


In [703]:
fredericks_finaldat = compile_dat('c067631c', 'Ryan-Fredericks', 'Ryan_Fredericks', checkgames = True)
if isinstance(fredericks_finaldat, pd.DataFrame):
    fredericks_finaldat.to_csv(os.path.join(nest_folder_def, "fredericks_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [704]:
balbuena_finaldat = compile_dat('3626f7d9', 'Fabian-Balbuena', 'Balbuena', checkgames = True)
if isinstance(balbuena_finaldat, pd.DataFrame):
    balbuena_finaldat.to_csv(os.path.join(nest_folder_def, "balbuena_finaldat.csv"))

In [705]:
fanderson_finaldat = compile_dat('9f1893b9', 'Felipe-Anderson', 'Felipe Anderson', checkgames = True)
if isinstance(fanderson_finaldat, pd.DataFrame):
    fanderson_finaldat.to_csv(os.path.join(nest_folder, "fanderson_finaldat.csv"))

In [706]:
snodgrass_finaldat = compile_dat('9f4647e9', 'Robert-Snodgrass', 'Snodgrass', checkgames = True)
if isinstance(snodgrass_finaldat, pd.DataFrame):
    snodgrass_finaldat.to_csv(os.path.join(nest_folder, "snodgrass_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [707]:
saiss_finaldat = compile_dat('9fbbcf39', 'Romain-Saiss', 'Romain_Sa', checkgames = True)
if isinstance(saiss_finaldat, pd.DataFrame):
    saiss_finaldat.to_csv(os.path.join(nest_folder_def, "saiss_finaldat.csv"))

In [708]:
rbennett_finaldat = compile_dat('0bad4516', 'Ryan-Bennett', 'Ryan_Bennett', checkgames = True)
if isinstance(rbennett_finaldat, pd.DataFrame):
    rbennett_finaldat.to_csv(os.path.join(nest_folder_def, "rbennett_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [709]:
vinagre_finaldat = compile_dat('b937e490', 'Ruben-Vinagre', 'Vinagre', checkgames = True)
if isinstance(vinagre_finaldat, pd.DataFrame):
    vinagre_finaldat.to_csv(os.path.join(nest_folder_def, "vinagre_finaldat.csv"))

In [710]:
rneves_finaldat = compile_dat('44bfb6c5', 'Ruben-Neves', 'da Silva Neves', checkgames = True)
if isinstance(rneves_finaldat, pd.DataFrame):
    rneves_finaldat.to_csv(os.path.join(nest_folder, "rneves_finaldat.csv"))

In [711]:
cavaleiro_finaldat = compile_dat('979b811d', 'Ivan-Cavaleiro', 'Cavaleiro', checkgames = True)
if isinstance(cavaleiro_finaldat, pd.DataFrame):
    cavaleiro_finaldat.to_csv(os.path.join(nest_folder, "cavaleiro_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [712]:
hcosta_finaldat = compile_dat('f13d02a6', 'Helder-Costa', 'lder_Costa', checkgames = True)
if isinstance(hcosta_finaldat, pd.DataFrame):
    hcosta_finaldat.to_csv(os.path.join(nest_folder, "hcosta_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [713]:
moutinho_finaldat = compile_dat('f5a00fa4', 'Joao-Moutinho', 'Moutinho', checkgames = True)
if isinstance(moutinho_finaldat, pd.DataFrame):
    moutinho_finaldat.to_csv(os.path.join(nest_folder, "moutinho_finaldat.csv"))

In [714]:
jonnyotto_finaldat = compile_dat('e28868f3', 'Jonny-Castro', 'Castro Otto', checkgames = True)
if isinstance(jonnyotto_finaldat, pd.DataFrame):
    jonnyotto_finaldat.to_csv(os.path.join(nest_folder_def, "jonnyotto_finaldat.csv"))

In [715]:
ceballos_finaldat = compile_dat('c0617e2b', 'Dani-Ceballos', 'Ceballos', checkgames = True)
if isinstance(ceballos_finaldat, pd.DataFrame):
    ceballos_finaldat.to_csv(os.path.join(nest_folder, "ceballos_finaldat.csv"))

In [716]:
npepe_finaldat = compile_dat('57e3f0c7', 'Nicolas-Pepe', 'Nicolas_P', checkgames = True)
if isinstance(npepe_finaldat, pd.DataFrame):
    npepe_finaldat.to_csv(os.path.join(nest_folder, "npepe_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [717]:
elmohamady_finaldat = compile_dat('615ef733', 'Ahmed-Elmohamady', 'El Mohamady', checkgames = True)
if isinstance(elmohamady_finaldat, pd.DataFrame):
    elmohamady_finaldat.to_csv(os.path.join(nest_folder_def, "elmohamady_finaldat.csv"))

In [718]:
ntaylor_finaldat = compile_dat('c2f9d19f', 'Neil-Taylor', 'Neil_Taylor', checkgames = True)
if isinstance(ntaylor_finaldat, pd.DataFrame):
    ntaylor_finaldat.to_csv(os.path.join(nest_folder_def, "ntaylor_finaldat.csv"))

In [719]:
hourihane_finaldat = compile_dat('fcab00a0', 'Conor-Hourihane', 'Hourihane', checkgames = True)
if isinstance(hourihane_finaldat, pd.DataFrame):
    hourihane_finaldat.to_csv(os.path.join(nest_folder, "hourihane_finaldat.csv"))

In [720]:
elghazi_finaldat = compile_dat('ffa90327', 'Anwar-El-Ghazi', 'El Ghazi', checkgames = True)
if isinstance(elghazi_finaldat, pd.DataFrame):
    elghazi_finaldat.to_csv(os.path.join(nest_folder, "elghazi_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [721]:
wesley_finaldat = compile_dat('bfae110f', 'Wesley-Moraes', 'Wesley_Moraes', checkgames = True)
if isinstance(wesley_finaldat, pd.DataFrame):
    wesley_finaldat.to_csv(os.path.join(nest_folder, "wesley_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [722]:
engels_finaldat = compile_dat('4fdf1fd9', 'Bjorn-Engels', 'rn_Engels', checkgames = True)
if isinstance(engels_finaldat, pd.DataFrame):
    engels_finaldat.to_csv(os.path.join(nest_folder_def, "engels_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [723]:
guilbert_finaldat = compile_dat('5d8d7454', 'Frederic-Guilbert', 'Guilbert', checkgames = True)
if isinstance(guilbert_finaldat, pd.DataFrame):
    guilbert_finaldat.to_csv(os.path.join(nest_folder_def, "guilbert_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [724]:
trezeguet_finaldat = compile_dat('3ae14ed1', 'Trezeguet', 'Mahmoud Ahmed_Ibrahim', checkgames = True)
if isinstance(trezeguet_finaldat, pd.DataFrame):
    trezeguet_finaldat.to_csv(os.path.join(nest_folder, "trezeguet_finaldat.csv"))

In [725]:
douglasluiz_finaldat = compile_dat('6f7d826d', 'Douglas-Luiz', 'Douglas Luiz_Soares', checkgames = True)
if isinstance(douglasluiz_finaldat, pd.DataFrame):
    douglasluiz_finaldat.to_csv(os.path.join(nest_folder, "douglasluiz_finaldat.csv"))

In [726]:
nakamba_finaldat = compile_dat('de5b5570', 'Marvelous-Nakamba', 'Nakamba', checkgames = True)
if isinstance(nakamba_finaldat, pd.DataFrame):
    nakamba_finaldat.to_csv(os.path.join(nest_folder, "nakamba_finaldat.csv"))

In [727]:
samatta_finaldat = compile_dat('dae8623e', 'Mbwana-Samatta', 'Samatta', checkgames = True)
if isinstance(samatta_finaldat, pd.DataFrame):
    samatta_finaldat.to_csv(os.path.join(nest_folder, "samatta_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [728]:
stacey_finaldat = compile_dat('558807c0', 'Jack-Stacey', 'Jack_Stacey', checkgames = True)
if isinstance(stacey_finaldat, pd.DataFrame):
    stacey_finaldat.to_csv(os.path.join(nest_folder_def, "stacey_finaldat.csv"))

In [729]:
connolly_finaldat = compile_dat('27c01749', 'Aaron-Connolly', 'Aaron_Connolly', checkgames = True)
if isinstance(connolly_finaldat, pd.DataFrame):
    connolly_finaldat.to_csv(os.path.join(nest_folder, "connolly_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [730]:
alzate_finaldat = compile_dat('ad1f61f3', 'Steven-Alzate', 'Alzate', checkgames = True)
if isinstance(alzate_finaldat, pd.DataFrame):
    alzate_finaldat.to_csv(os.path.join(nest_folder, "alzate_finaldat.csv"))

In [731]:
vydra_finaldat = compile_dat('0d4ceb32', 'Matej-Vydra', 'Vydra', checkgames = True)
if isinstance(vydra_finaldat, pd.DataFrame):
    vydra_finaldat.to_csv(os.path.join(nest_folder, "vydra_finaldat.csv"))

In [732]:
brownhill_finaldat = compile_dat('47369df8', 'Josh-Brownhill', 'Brownhill', checkgames = True)
if isinstance(brownhill_finaldat, pd.DataFrame):
    brownhill_finaldat.to_csv(os.path.join(nest_folder, "brownhill_finaldat.csv"))

In [733]:
pulisic_finaldat = compile_dat('1bf33a9a', 'Christian-Pulisic', 'Pulisic', checkgames = True)
if isinstance(pulisic_finaldat, pd.DataFrame):
    pulisic_finaldat.to_csv(os.path.join(nest_folder, "pulisic_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [734]:
tomori_finaldat = compile_dat('7edfbb8a', 'Fikayo-Tomori', 'Tomori', checkgames = True)
if isinstance(tomori_finaldat, pd.DataFrame):
    tomori_finaldat.to_csv(os.path.join(nest_folder_def, "tomori_finaldat.csv"))

In [735]:
jmccarthy_finaldat = compile_dat('64b08de7', 'James-McCarthy', 'James_McCarthy', checkgames = True)
if isinstance(jmccarthy_finaldat, pd.DataFrame):
    jmccarthy_finaldat.to_csv(os.path.join(nest_folder, "jmccarthy_finaldat.csv"))

In [736]:
riedewald_finaldat = compile_dat('7a1f2c08', 'Jairo-Riedewald', 'Riedewald', checkgames = True)
if isinstance(riedewald_finaldat, pd.DataFrame):
    riedewald_finaldat.to_csv(os.path.join(nest_folder_def, "riedewald_finaldat.csv"))

In [737]:
mkean_finaldat = compile_dat('3d50bcdb', 'Moise-Kean', 'Moise_Kean', checkgames = True)
if isinstance(mkean_finaldat, pd.DataFrame):
    mkean_finaldat.to_csv(os.path.join(nest_folder, "mkean_finaldat.csv"))

In [738]:
sidibe_finaldat = compile_dat('4c5b714f', 'Djibril-Sidibe', 'Djibril_Sidib', checkgames = True)
if isinstance(sidibe_finaldat, pd.DataFrame):
    sidibe_finaldat.to_csv(os.path.join(nest_folder_def, "sidibe_finaldat.csv"))

In [739]:
soyuncu_finaldat = compile_dat('21166ff4', 'Caglar-Soyuncu', 'lar_S', checkgames = True)
if isinstance(soyuncu_finaldat, pd.DataFrame):
    soyuncu_finaldat.to_csv(os.path.join(nest_folder_def, "soyuncu_finaldat.csv"))

In [740]:
praet_finaldat = compile_dat('86695068', 'Dennis-Praet', 'Praet', checkgames = True)
if isinstance(praet_finaldat, pd.DataFrame):
    praet_finaldat.to_csv(os.path.join(nest_folder, "praet_finaldat.csv"))

In [741]:
origi_finaldat = compile_dat('43a166b4', 'Divock-Origi', 'Origi', checkgames = True)
if isinstance(origi_finaldat, pd.DataFrame):
    origi_finaldat.to_csv(os.path.join(nest_folder, "origi_finaldat.csv"))

In [742]:
egarcia_finaldat = compile_dat('2bed3eab', 'Eric-Garcia', 'Eric_Garc', checkgames = True)
if isinstance(egarcia_finaldat, pd.DataFrame):
    egarcia_finaldat.to_csv(os.path.join(nest_folder_def, "egarcia_finaldat.csv"))

In [743]:
mgreenwood_finaldat = compile_dat('58eee997', 'Mason-Greenwood', 'Mason_Greenwood', checkgames = True)
if isinstance(mgreenwood_finaldat, pd.DataFrame):
    mgreenwood_finaldat.to_csv(os.path.join(nest_folder, "mgreenwood_finaldat.csv"))

In [744]:
djames_finaldat = compile_dat('c931d9f9', 'Daniel-James', 'Daniel_James', checkgames = True)
if isinstance(djames_finaldat, pd.DataFrame):
    djames_finaldat.to_csv(os.path.join(nest_folder, "djames_finaldat.csv"))

In [745]:
bwilliams_finaldat = compile_dat('5e74d6c8', 'Brandon-Williams', 'Brandon_Williams', checkgames = True)
if isinstance(bwilliams_finaldat, pd.DataFrame):
    bwilliams_finaldat.to_csv(os.path.join(nest_folder_def, "bwilliams_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [746]:
willems_finaldat = compile_dat('9050ddc4', 'Jetro-Willems', 'Jetro_Willems', checkgames = True)
if isinstance(willems_finaldat, pd.DataFrame):
    willems_finaldat.to_csv(os.path.join(nest_folder_def, "willems_finaldat.csv"))

In [747]:
saintmax_finaldat = compile_dat('2b16cb1a', 'Allan-Saint-Maximin', 'Saint-Maximin', checkgames = True)
if isinstance(saintmax_finaldat, pd.DataFrame):
    saintmax_finaldat.to_csv(os.path.join(nest_folder, "saintmax_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [748]:
bentaleb_finaldat = compile_dat('3189f61a', 'Nabil-Bentaleb', 'Bentaleb', checkgames = True)
if isinstance(bentaleb_finaldat, pd.DataFrame):
    bentaleb_finaldat.to_csv(os.path.join(nest_folder, "bentaleb_finaldat.csv"))

In [749]:
godfrey_finaldat = compile_dat('9f7c837d', 'Ben-Godfrey', 'Godfrey', checkgames = True)
if isinstance(godfrey_finaldat, pd.DataFrame):
    godfrey_finaldat.to_csv(os.path.join(nest_folder_def, "godfrey_finaldat.csv"))

In [750]:
zimmermann_finaldat = compile_dat('2d5f03bf', 'Christoph-Zimmermann', 'Christoph_Zimmermann', checkgames = True)
if isinstance(zimmermann_finaldat, pd.DataFrame):
    zimmermann_finaldat.to_csv(os.path.join(nest_folder_def, "zimmermann_finaldat.csv"))

In [751]:
hanley_finaldat = compile_dat('e9254eec', 'Grant-Hanley', 'Grant_Hanley', checkgames = True)
if isinstance(hanley_finaldat, pd.DataFrame):
    hanley_finaldat.to_csv(os.path.join(nest_folder_def, "hanley_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [752]:
pukki_finaldat = compile_dat('0745b37d', 'Teemu-Pukki', 'Pukki', checkgames = True)
if isinstance(pukki_finaldat, pd.DataFrame):
    pukki_finaldat.to_csv(os.path.join(nest_folder, "pukki_finaldat.csv"))

In [753]:
onel_finaldat = compile_dat('007b39a5', 'Onel-Hernandez', 'Onel_Hern', checkgames = True)
if isinstance(onel_finaldat, pd.DataFrame):
    onel_finaldat.to_csv(os.path.join(nest_folder, "onel_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [754]:
stiepermann_finaldat = compile_dat('b606638c', 'Marco-Stiepermann', 'Stiepermann', checkgames = True)
if isinstance(stiepermann_finaldat, pd.DataFrame):
    stiepermann_finaldat.to_csv(os.path.join(nest_folder, "stiepermann_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [755]:
mclean_finaldat = compile_dat('471f16b3', 'Kenny-McLean', 'Kenny_McLean', checkgames = True)
if isinstance(mclean_finaldat, pd.DataFrame):
    mclean_finaldat.to_csv(os.path.join(nest_folder, "mclean_finaldat.csv"))

In [756]:
cantwell_finaldat = compile_dat('f63d8a4b', 'Todd-Cantwell', 'Todd_Cantwell', checkgames = True)
if isinstance(cantwell_finaldat, pd.DataFrame):
    cantwell_finaldat.to_csv(os.path.join(nest_folder, "cantwell_finaldat.csv"))

In [757]:
leitner_finaldat = compile_dat('9460aaed', 'Moritz-Leitner', 'Moritz_Leitner', checkgames = True)
if isinstance(leitner_finaldat, pd.DataFrame):
    leitner_finaldat.to_csv(os.path.join(nest_folder, "leitner_finaldat.csv"))

In [758]:
tettey_finaldat = compile_dat('75566759', 'Alexander-Tettey', 'Tettey', checkgames = True)
if isinstance(tettey_finaldat, pd.DataFrame):
    tettey_finaldat.to_csv(os.path.join(nest_folder, "tettey_finaldat.csv"))

In [759]:
trybull_finaldat = compile_dat('552aa16c', 'Tom-Trybull', 'Trybull', checkgames = True)
if isinstance(trybull_finaldat, pd.DataFrame):
    trybull_finaldat.to_csv(os.path.join(nest_folder, "trybull_finaldat.csv"))

In [760]:
drmic_finaldat = compile_dat('15ea94de', 'Josip-Drmic', 'Josip_Drmi', checkgames = True)
if isinstance(drmic_finaldat, pd.DataFrame):
    drmic_finaldat.to_csv(os.path.join(nest_folder, "drmic_finaldat.csv"))

In [761]:
sbyram_finaldat = compile_dat('1bf811d4', 'Sam-Byram', 'Sam_Byram', checkgames = True)
if isinstance(sbyram_finaldat, pd.DataFrame):
    sbyram_finaldat.to_csv(os.path.join(nest_folder_def, "sbyram_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [762]:
amadou_finaldat = compile_dat('9260926b', 'Ibrahim-Amadou', 'Ibrahim_Amadou', checkgames = True)
if isinstance(amadou_finaldat, pd.DataFrame):
    amadou_finaldat.to_csv(os.path.join(nest_folder_def, "amadou_finaldat.csv"))

In [763]:
duda_finaldat = compile_dat('79ca3e23', 'Ondrej-Duda', 'Ondrej', checkgames = True)
if isinstance(duda_finaldat, pd.DataFrame):
    duda_finaldat.to_csv(os.path.join(nest_folder, "duda_finaldat.csv"))

In [764]:
rupp_finaldat = compile_dat('3faa1780', 'Lukas-Rupp', 'Lukas_Rupp', checkgames = True)
if isinstance(rupp_finaldat, pd.DataFrame):
    rupp_finaldat.to_csv(os.path.join(nest_folder, "rupp_finaldat.csv"))

In [765]:
mousset_finaldat = compile_dat('d9cfca3d', 'Lys-Mousset', 'Mousset', checkgames = True)
if isinstance(mousset_finaldat, pd.DataFrame):
    mousset_finaldat.to_csv(os.path.join(nest_folder, "mousset_finaldat.csv"))

In [766]:
estevens_finaldat = compile_dat('17d4507a', 'Enda-Stevens', 'Enda_Stevens', checkgames = True)
if isinstance(estevens_finaldat, pd.DataFrame):
    estevens_finaldat.to_csv(os.path.join(nest_folder_def, "estevens_finaldat.csv"))

In [767]:
oconnell_finaldat = compile_dat('312626c1', 'Jack-OConnell', 'Jack_O', checkgames = True)
if isinstance(oconnell_finaldat, pd.DataFrame):
    oconnell_finaldat.to_csv(os.path.join(nest_folder_def, "oconnell_finaldat.csv"))

In [768]:
baldock_finaldat = compile_dat('394854c4', 'George-Baldock', 'George_Baldock', checkgames = True)
if isinstance(baldock_finaldat, pd.DataFrame):
    baldock_finaldat.to_csv(os.path.join(nest_folder_def, "baldock_finaldat.csv"))

In [769]:
egan_finaldat = compile_dat('debe38a0', 'John-Egan', 'John_Egan', checkgames = True)
if isinstance(egan_finaldat, pd.DataFrame):
    egan_finaldat.to_csv(os.path.join(nest_folder_def, "egan_finaldat.csv"))

In [770]:
lundstram_finaldat = compile_dat('0df2b165', 'John-Lundstram', 'Lundstram', checkgames = True)
if isinstance(lundstram_finaldat, pd.DataFrame):
    lundstram_finaldat.to_csv(os.path.join(nest_folder_def, "lundstram_finaldat.csv"))

In [771]:
bsharp_finaldat = compile_dat('e8c9f4e9', 'Billy-Sharp', 'Billy_Sharp', checkgames = True)
if isinstance(bsharp_finaldat, pd.DataFrame):
    bsharp_finaldat.to_csv(os.path.join(nest_folder, "bsharp_finaldat.csv"))

In [772]:
fleck_finaldat = compile_dat('b63e789d', 'John-Fleck', 'John_Fleck', checkgames = True)
if isinstance(fleck_finaldat, pd.DataFrame):
    fleck_finaldat.to_csv(os.path.join(nest_folder, "fleck_finaldat.csv"))

In [773]:
norwood_finaldat = compile_dat('43439614', 'Oliver-Norwood', 'Oliver_Norwood', checkgames = True)
if isinstance(norwood_finaldat, pd.DataFrame):
    norwood_finaldat.to_csv(os.path.join(nest_folder, "norwood_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [774]:
mcgoldrick_finaldat = compile_dat('dbe56a33', 'David-McGoldrick', 'David_McGoldrick', checkgames = True)
if isinstance(mcgoldrick_finaldat, pd.DataFrame):
    mcgoldrick_finaldat.to_csv(os.path.join(nest_folder, "mcgoldrick_finaldat.csv"))

In [775]:
basham_finaldat = compile_dat('1aa84968', 'Chris-Basham', 'Chris_Basham', checkgames = True)
if isinstance(basham_finaldat, pd.DataFrame):
    basham_finaldat.to_csv(os.path.join(nest_folder_def, "basham_finaldat.csv"))

In [776]:
crobinson_finaldat = compile_dat('e75750a3', 'Callum-Robinson', 'Callum_Robinson', checkgames = True)
if isinstance(crobinson_finaldat, pd.DataFrame):
    crobinson_finaldat.to_csv(os.path.join(nest_folder, "crobinson_finaldat.csv"))

In [777]:
mcburnie_finaldat = compile_dat('78b8ac63', 'Oliver-McBurnie', 'McBurnie', checkgames = True)
if isinstance(mcburnie_finaldat, pd.DataFrame):
    mcburnie_finaldat.to_csv(os.path.join(nest_folder, "mcburnie_finaldat.csv"))

In [778]:
obafemi_finaldat = compile_dat('caa1a7f0', 'Michael-Obafemi', 'Michael_Obafemi', checkgames = True)
if isinstance(obafemi_finaldat, pd.DataFrame):
    obafemi_finaldat.to_csv(os.path.join(nest_folder, "obafemi_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [779]:
cadams_finaldat = compile_dat('f2bf1b0f', 'Che-Adams', 'Che_Adams', checkgames = True)
if isinstance(cadams_finaldat, pd.DataFrame):
    cadams_finaldat.to_csv(os.path.join(nest_folder, "cadams_finaldat.csv"))

In [780]:
djenepo_finaldat = compile_dat('254147d4', 'Moussa-Djenepo', 'Djenepo', checkgames = True)
if isinstance(djenepo_finaldat, pd.DataFrame):
    djenepo_finaldat.to_csv(os.path.join(nest_folder, "djenepo_finaldat.csv"))

In [781]:
ndombele_finaldat = compile_dat('5cdddffa', 'Tanguy-Ndombele', 'Ndombele', checkgames = True)
if isinstance(ndombele_finaldat, pd.DataFrame):
    ndombele_finaldat.to_csv(os.path.join(nest_folder, "ndombele_finaldat.csv"))

In [782]:
bergwijn_finaldat = compile_dat('a29b1131', 'Steven-Bergwijn', 'Bergwijn', checkgames = True)
if isinstance(bergwijn_finaldat, pd.DataFrame):
    bergwijn_finaldat.to_csv(os.path.join(nest_folder, "bergwijn_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [784]:
nchalobah_finaldat = compile_dat('422862ea', 'Nathaniel-Chalobah', 'Nathaniel_Chalobah', checkgames = True)
if isinstance(nchalobah_finaldat, pd.DataFrame):
    nchalobah_finaldat.to_csv(os.path.join(nest_folder_def, "nchalobah_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [783]:
yarmolenko_finaldat = compile_dat('849c826a', 'Andriy-Yarmolenko', 'Yarmolenko', checkgames = True)
if isinstance(yarmolenko_finaldat, pd.DataFrame):
    yarmolenko_finaldat.to_csv(os.path.join(nest_folder, "yarmolenko_finaldat.csv"))

In [785]:
fornals_finaldat = compile_dat('82927de4', 'Pablo-Fornals', 'Pablo_Fornals', checkgames = True)
if isinstance(fornals_finaldat, pd.DataFrame):
    fornals_finaldat.to_csv(os.path.join(nest_folder, "fornals_finaldat.csv"))

In [786]:
haller_finaldat = compile_dat('ba115a40', 'Sebastien-Haller', 'tien_Haller', checkgames = True)
if isinstance(haller_finaldat, pd.DataFrame):
    haller_finaldat.to_csv(os.path.join(nest_folder, "haller_finaldat.csv"))

In [787]:
mari_finaldat = compile_dat('aa54ec6f', 'Pablo-Mari', 'Pablo_Mar', checkgames = True)
if isinstance(mari_finaldat, pd.DataFrame):
    mari_finaldat.to_csv(os.path.join(nest_folder_def, "mari_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [788]:
btraore_finaldat = compile_dat('c47541e0', 'Bertrand-Traore', 'Bertrand_Traor', checkgames = True)
if isinstance(btraore_finaldat, pd.DataFrame):
    btraore_finaldat.to_csv(os.path.join(nest_folder, "btraore_finaldat.csv"))

In [789]:
ziyech_finaldat = compile_dat('6622454d', 'Hakim-Ziyech', 'Ziyech', checkgames = True)
if isinstance(ziyech_finaldat, pd.DataFrame):
    ziyech_finaldat.to_csv(os.path.join(nest_folder, "ziyech_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [790]:
tsilva_finaldat = compile_dat('86e7deaf', 'Thiago-Silva', 'Thiago_Emiliano da Silva', checkgames = True)
if isinstance(tsilva_finaldat, pd.DataFrame):
    tsilva_finaldat.to_csv(os.path.join(nest_folder_def, "tsilva_finaldat.csv"))

In [791]:
allan_finaldat = compile_dat('cc700722', 'Allan', 'Allan_Marques Loureiro', checkgames = True)
if isinstance(allan_finaldat, pd.DataFrame):
    allan_finaldat.to_csv(os.path.join(nest_folder, "allan_finaldat.csv"))

In [792]:
jamesrodriguez_finaldat = compile_dat('715bf047', 'James-Rodriguez', 'James_Rodr', checkgames = True)
if isinstance(jamesrodriguez_finaldat, pd.DataFrame):
    jamesrodriguez_finaldat.to_csv(os.path.join(nest_folder, "jamesrodriguez_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [793]:
lookman_finaldat = compile_dat('7c104bb7', 'Ademola-Lookman', 'Ademola_Lookman', checkgames = True)
if isinstance(lookman_finaldat, pd.DataFrame):
    lookman_finaldat.to_csv(os.path.join(nest_folder, "lookman_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [794]:
maja_finaldat = compile_dat('bc0bfc64', 'Josh-Maja', 'Josh_Maja', checkgames = True)
if isinstance(maja_finaldat, pd.DataFrame):
    maja_finaldat.to_csv(os.path.join(nest_folder, "maja_finaldat.csv"))

In [795]:
lcooper_finaldat = compile_dat('dc64b8b3', 'Liam-Cooper', 'Liam_Cooper', checkgames = True)
if isinstance(lcooper_finaldat, pd.DataFrame):
    lcooper_finaldat.to_csv(os.path.join(nest_folder_def, "lcooper_finaldat.csv"))

In [796]:
ayling_finaldat = compile_dat('420d7c7d', 'Luke-Ayling', 'Luke_Ayling', checkgames = True)
if isinstance(ayling_finaldat, pd.DataFrame):
    ayling_finaldat.to_csv(os.path.join(nest_folder_def, "ayling_finaldat.csv"))

In [797]:
dallas_finaldat = compile_dat('67ca8566', 'Stuart-Dallas', 'Stuart_Dallas', checkgames = True)
if isinstance(dallas_finaldat, pd.DataFrame):
    dallas_finaldat.to_csv(os.path.join(nest_folder_def, "dallas_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [798]:
alioski_finaldat = compile_dat('1a7fcf39', 'Ezgjan-Alioski', 'Alioski', checkgames = True)
if isinstance(alioski_finaldat, pd.DataFrame):
    alioski_finaldat.to_csv(os.path.join(nest_folder_def, "alioski_finaldat.csv"))

In [799]:
klich_finaldat = compile_dat('282679b4', 'Mateusz-Klich', 'Mateusz_Klich', checkgames = True)
if isinstance(klich_finaldat, pd.DataFrame):
    klich_finaldat.to_csv(os.path.join(nest_folder, "klich_finaldat.csv"))

In [800]:
bamford_finaldat = compile_dat('93feac6e', 'Patrick-Bamford', 'Patrick_Bamford', checkgames = True)
if isinstance(bamford_finaldat, pd.DataFrame):
    bamford_finaldat.to_csv(os.path.join(nest_folder, "bamford_finaldat.csv"))

In [801]:
troberts_finaldat = compile_dat('6c17e046', 'Tyler-Roberts', 'Tyler_Roberts', checkgames = True)
if isinstance(troberts_finaldat, pd.DataFrame):
    troberts_finaldat.to_csv(os.path.join(nest_folder, "troberts_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [802]:
struijk_finaldat = compile_dat('f4d296e4', 'Pascal-Struijk', 'Struijk', checkgames = True)
if isinstance(struijk_finaldat, pd.DataFrame):
    struijk_finaldat.to_csv(os.path.join(nest_folder_def, "struijk_finaldat.csv"))

In [803]:
koch_finaldat = compile_dat('ea6bc03a', 'Robin-Koch', 'Robin_Koch', checkgames = True)
if isinstance(koch_finaldat, pd.DataFrame):
    koch_finaldat.to_csv(os.path.join(nest_folder_def, "koch_finaldat.csv"))

In [804]:
rodrigom_finaldat = compile_dat('1fb1c435', 'Rodrigo-Moreno', 'Rodrigo_Moreno', checkgames = True)
if isinstance(rodrigom_finaldat, pd.DataFrame):
    rodrigom_finaldat.to_csv(os.path.join(nest_folder, "rodrigom_finaldat.csv"))

In [805]:
dllorente_finaldat = compile_dat('bba30585', 'Diego-Llorente', 'Diego_Llorente', checkgames = True)
if isinstance(dllorente_finaldat, pd.DataFrame):
    dllorente_finaldat.to_csv(os.path.join(nest_folder_def, "dllorente_finaldat.csv"))

In [806]:
raphinha_finaldat = compile_dat('3423f250', 'Raphinha', 'Raphael_Dias Belloli', checkgames = True)
if isinstance(raphinha_finaldat, pd.DataFrame):
    raphinha_finaldat.to_csv(os.path.join(nest_folder, "raphinha_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_58690/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [807]:
thiagoalcantara_finaldat = compile_dat('77e84962', 'Thiago-Alcantara', 'Thiago_Alc', checkgames = True)
if isinstance(thiagoalcantara_finaldat, pd.DataFrame):
    thiagoalcantara_finaldat.to_csv(os.path.join(nest_folder, "thiagoalcantara_finaldat.csv"))

In [10]:
kabak_finaldat = compile_dat('2d61d13a', 'Ozan-Kabak', 'Kabak', checkgames = True)
if isinstance(kabak_finaldat, pd.DataFrame):
    kabak_finaldat.to_csv(os.path.join(nest_folder_def, "kabak_finaldat.csv"))

In [12]:
ferran_finaldat = compile_dat('9e1035f8', 'Ferran-Torres', 'n_Torres', checkgames = True)
if isinstance(ferran_finaldat, pd.DataFrame):
    ferran_finaldat.to_csv(os.path.join(nest_folder, "ferran_finaldat.csv"))

In [13]:
telles_finaldat = compile_dat('e73c9bb2', 'Alex-Telles', 'Alex Nicolao_Telles', checkgames = True)
if isinstance(telles_finaldat, pd.DataFrame):
    telles_finaldat.to_csv(os.path.join(nest_folder_def, "telles_finaldat.csv"))

In [14]:
cavani_finaldat = compile_dat('527f063d', 'Edinson-Cavani', 'Cavani', checkgames = True)
if isinstance(cavani_finaldat, pd.DataFrame):
    cavani_finaldat.to_csv(os.path.join(nest_folder, "cavani_finaldat.csv"))

In [15]:
brewster_finaldat = compile_dat('30e81a88', 'Rhian-Brewster', 'Rhian_Brewster', checkgames = True)
if isinstance(brewster_finaldat, pd.DataFrame):
    brewster_finaldat.to_csv(os.path.join(nest_folder, "brewster_finaldat.csv"))

In [16]:
jrobinson_finaldat = compile_dat('7399c7a8', 'Jack-Robinson', 'Jack_Robinson', checkgames = True)
if isinstance(jrobinson_finaldat, pd.DataFrame):
    jrobinson_finaldat.to_csv(os.path.join(nest_folder_def, "jrobinson_finaldat.csv"))

In [17]:
osborn_finaldat = compile_dat('79bc4c45', 'Ben-Osborn', 'Ben_Osborn', checkgames = True)
if isinstance(osborn_finaldat, pd.DataFrame):
    osborn_finaldat.to_csv(os.path.join(nest_folder, "osborn_finaldat.csv"))

In [18]:
burke_finaldat = compile_dat('f631c17f', 'Oliver-Burke', 'Oliver_Burke', checkgames = True)
if isinstance(burke_finaldat, pd.DataFrame):
    burke_finaldat.to_csv(os.path.join(nest_folder, "burke_finaldat.csv"))

In [19]:
ampadu_finaldat = compile_dat('10b9fa99', 'Ethan-Ampadu', 'Ampadu', checkgames = True)
if isinstance(ampadu_finaldat, pd.DataFrame):
    ampadu_finaldat.to_csv(os.path.join(nest_folder_def, "ampadu_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [20]:
bogle_finaldat = compile_dat('ae5019e6', 'Jayden-Bogle', 'Bogle', checkgames = True)
if isinstance(bogle_finaldat, pd.DataFrame):
    bogle_finaldat.to_csv(os.path.join(nest_folder_def, "bogle_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [21]:
kbryan_finaldat = compile_dat('c684d3c9', 'Kean-Bryan', 'Kean_Bryan', checkgames = True)
if isinstance(kbryan_finaldat, pd.DataFrame):
    kbryan_finaldat.to_csv(os.path.join(nest_folder_def, "kbryan_finaldat.csv"))

In [22]:
minamino_finaldat = compile_dat('f833a830', 'Takumi-Minamino', 'Minamino', checkgames = True)
if isinstance(minamino_finaldat, pd.DataFrame):
    minamino_finaldat.to_csv(os.path.join(nest_folder, "minamino_finaldat.csv"))

In [23]:
salisu_finaldat = compile_dat('0b33f6ad', 'Mohammed-Salisu', 'Salisu', checkgames = True)
if isinstance(salisu_finaldat, pd.DataFrame):
    salisu_finaldat.to_csv(os.path.join(nest_folder_def, "salisu_finaldat.csv"))

In [24]:
tella_finaldat = compile_dat('f1e94cd6', 'Nathan-Tella', 'Nathan_Tella', checkgames = True)
if isinstance(tella_finaldat, pd.DataFrame):
    tella_finaldat.to_csv(os.path.join(nest_folder, "tella_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [25]:
idiallo_finaldat = compile_dat('11ff3d56', 'Ibrahima-Diallo', 'Ibrahima_Diallo', checkgames = True)
if isinstance(idiallo_finaldat, pd.DataFrame):
    idiallo_finaldat.to_csv(os.path.join(nest_folder, "idiallo_finaldat.csv"))

In [26]:
bale_finaldat = compile_dat('a58bb1e1', 'Gareth-Bale', 'Gareth_Bale', checkgames = True)
if isinstance(bale_finaldat, pd.DataFrame):
    bale_finaldat.to_csv(os.path.join(nest_folder, "bale_finaldat.csv"))

In [27]:
rodon_finaldat = compile_dat('8d62d31a', 'Joe-Rodon', 'Joe_Rodon', checkgames = True)
if isinstance(rodon_finaldat, pd.DataFrame):
    rodon_finaldat.to_csv(os.path.join(nest_folder_def, "rodon_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [28]:
bartley_finaldat = compile_dat('18e96f64', 'Kyle-Bartley', 'Kyle_Bartley', checkgames = True)
if isinstance(bartley_finaldat, pd.DataFrame):
    bartley_finaldat.to_csv(os.path.join(nest_folder_def, "bartley_finaldat.csv"))

In [29]:
ajayi_finaldat = compile_dat('d6192210', 'Semi-Ajayi', 'Ibidapo_Ajayi', checkgames = True)
if isinstance(ajayi_finaldat, pd.DataFrame):
    ajayi_finaldat.to_csv(os.path.join(nest_folder_def, "ajayi_finaldat.csv"))

In [30]:
furlong_finaldat = compile_dat('3ec9195b', 'Darnell-Furlong', 'Darnell_Furlong', checkgames = True)
if isinstance(furlong_finaldat, pd.DataFrame):
    furlong_finaldat.to_csv(os.path.join(nest_folder_def, "furlong_finaldat.csv"))

In [31]:
oshea_finaldat = compile_dat('1d042188', 'Dara-OShea', 'Dara_O', checkgames = True)
if isinstance(oshea_finaldat, pd.DataFrame):
    oshea_finaldat.to_csv(os.path.join(nest_folder_def, "oshea_finaldat.csv"))

In [32]:
diangana_finaldat = compile_dat('f522d7ec', 'Grady-Diangana', 'Diangana', checkgames = True)
if isinstance(diangana_finaldat, pd.DataFrame):
    diangana_finaldat.to_csv(os.path.join(nest_folder, "diangana_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [33]:
mpereira_finaldat = compile_dat('b04ee927', 'Matheus-Pereira', 'Matheus_Pereira', checkgames = True)
if isinstance(mpereira_finaldat, pd.DataFrame):
    mpereira_finaldat.to_csv(os.path.join(nest_folder, "mpereira_finaldat.csv"))

In [34]:
ivanovic_finaldat = compile_dat('85f8571a', 'Branislav-Ivanovic', 'Branislav_Ivanovi', checkgames = True)
if isinstance(ivanovic_finaldat, pd.DataFrame):
    ivanovic_finaldat.to_csv(os.path.join(nest_folder_def, "ivanovic_finaldat.csv"))

In [35]:
diagne_finaldat = compile_dat('dda3969e', 'Mbaye-Diagne', 'Mbaye_Diagne', checkgames = True)
if isinstance(diagne_finaldat, pd.DataFrame):
    diagne_finaldat.to_csv(os.path.join(nest_folder, "diagne_finaldat.csv"))

In [36]:
yokuslu_finaldat = compile_dat('e49ce451', 'Okay-Yokuslu', 'Okay_Yoku', checkgames = True)
if isinstance(yokuslu_finaldat, pd.DataFrame):
    yokuslu_finaldat.to_csv(os.path.join(nest_folder, "yokuslu_finaldat.csv"))

In [37]:
benrahma_finaldat = compile_dat('5a04fd92', 'Said-Benrahma', 'Benrahma', checkgames = True)
if isinstance(benrahma_finaldat, pd.DataFrame):
    benrahma_finaldat.to_csv(os.path.join(nest_folder, "benrahma_finaldat.csv"))

In [38]:
fsilva_finaldat = compile_dat('e7aa9d7c', 'Fabio-Silva', 'bio_Silva', checkgames = True)
if isinstance(fsilva_finaldat, pd.DataFrame):
    fsilva_finaldat.to_csv(os.path.join(nest_folder, "fsilva_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [39]:
willianjose_finaldat = compile_dat('d87e2cae', 'Willian-Jose', 'Willian Jo', checkgames = True)
if isinstance(willianjose_finaldat, pd.DataFrame):
    willianjose_finaldat.to_csv(os.path.join(nest_folder, "willianjose_finaldat.csv"))

In [40]:
tavares_finaldat = compile_dat('a65c844b', 'Nuno-Tavares', 'Nuno_Varela', checkgames = True)
if isinstance(tavares_finaldat, pd.DataFrame):
    tavares_finaldat.to_csv(os.path.join(nest_folder_def, "tavares_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [41]:
lokonga_finaldat = compile_dat('1b4f1169', 'Albert-Sambi-Lokonga', 'Lokonga', checkgames = True)
if isinstance(lokonga_finaldat, pd.DataFrame):
    lokonga_finaldat.to_csv(os.path.join(nest_folder, "lokonga_finaldat.csv"))

In [42]:
jansson_finaldat = compile_dat('172e52d8', 'Pontus-Jansson', 'Pontus_Jansson', checkgames = True)
if isinstance(jansson_finaldat, pd.DataFrame):
    jansson_finaldat.to_csv(os.path.join(nest_folder_def, "jansson_finaldat.csv"))

In [43]:
canos_finaldat = compile_dat('1137759d', 'Sergi-Canos', 'Sergi_Can', checkgames = True)
if isinstance(canos_finaldat, pd.DataFrame):
    canos_finaldat.to_csv(os.path.join(nest_folder, "canos_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [44]:
mbsorensen_finaldat = compile_dat('30db8574', 'Mads-Bech-Sorensen', 'Mads_Bech', checkgames = True)
if isinstance(mbsorensen_finaldat, pd.DataFrame):
    mbsorensen_finaldat.to_csv(os.path.join(nest_folder_def, "mbsorensen_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [45]:
baptiste_finaldat = compile_dat('2886c9b5', 'Shandon-Baptiste', 'Shandon_Baptiste', checkgames = True)
if isinstance(baptiste_finaldat, pd.DataFrame):
    baptiste_finaldat.to_csv(os.path.join(nest_folder, "baptiste_finaldat.csv"))

In [46]:
mwepu_finaldat = compile_dat('1d5cab82', 'Enock-Mwepu', 'Mwepu', checkgames = True)
if isinstance(mwepu_finaldat, pd.DataFrame):
    mwepu_finaldat.to_csv(os.path.join(nest_folder, "mwepu_finaldat.csv"))

In [47]:
croberts_finaldat = compile_dat('24fae784', 'Connor-Roberts', 'Connor_Roberts', checkgames = True)
if isinstance(croberts_finaldat, pd.DataFrame):
    croberts_finaldat.to_csv(os.path.join(nest_folder_def, "croberts_finaldat.csv"))

In [48]:
weghorst_finaldat = compile_dat('c4e87b8b', 'Wout-Weghorst', 'Weghorst', checkgames = True)
if isinstance(weghorst_finaldat, pd.DataFrame):
    weghorst_finaldat.to_csv(os.path.join(nest_folder, "weghorst_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [49]:
olise_finaldat = compile_dat('c4486bac', 'Michael-Olise', 'Olise', checkgames = True)
if isinstance(olise_finaldat, pd.DataFrame):
    olise_finaldat.to_csv(os.path.join(nest_folder, "olise_finaldat.csv"))

In [50]:
shackleton_finaldat = compile_dat('8d03ca5a', 'Jamie-Shackleton', 'Shackleton', checkgames = True)
if isinstance(shackleton_finaldat, pd.DataFrame):
    shackleton_finaldat.to_csv(os.path.join(nest_folder, "shackleton_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [51]:
firpo_finaldat = compile_dat('24f59bb2', 'Junior-Firpo', 'Junior_Firpo', checkgames = True)
if isinstance(firpo_finaldat, pd.DataFrame):
    firpo_finaldat.to_csv(os.path.join(nest_folder_def, "firpo_finaldat.csv"))

In [52]:
forshaw_finaldat = compile_dat('9f1938ab', 'Adam-Forshaw', 'Adam_Forshaw', checkgames = True)
if isinstance(forshaw_finaldat, pd.DataFrame):
    forshaw_finaldat.to_csv(os.path.join(nest_folder, "forshaw_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [53]:
gelhardt_finaldat = compile_dat('01962d0d', 'Joe-Gelhardt', 'Gelhardt', checkgames = True)
if isinstance(gelhardt_finaldat, pd.DataFrame):
    gelhardt_finaldat.to_csv(os.path.join(nest_folder, "gelhardt_finaldat.csv"))

In [54]:
varane_finaldat = compile_dat('9f8e9423', 'Raphael-Varane', 'l_Varane', checkgames = True)
if isinstance(varane_finaldat, pd.DataFrame):
    varane_finaldat.to_csv(os.path.join(nest_folder_def, "varane_finaldat.csv"))

In [55]:
cronaldo_finaldat = compile_dat('dea698d9', 'Cristiano-Ronaldo', 'Cristiano Ronaldo', checkgames = True)
if isinstance(cronaldo_finaldat, pd.DataFrame):
    cronaldo_finaldat.to_csv(os.path.join(nest_folder, "cronaldo_finaldat.csv"))

In [56]:
kdowell_finaldat = compile_dat('2f6ec02a', 'Kieran-Dowell', 'Kieran_Dowell', checkgames = True)
if isinstance(kdowell_finaldat, pd.DataFrame):
    kdowell_finaldat.to_csv(os.path.join(nest_folder, "kdowell_finaldat.csv"))

In [57]:
rashica_finaldat = compile_dat('6c6a3300', 'Milot-Rashica', 'Rashica', checkgames = True)
if isinstance(rashica_finaldat, pd.DataFrame):
    rashica_finaldat.to_csv(os.path.join(nest_folder, "rashica_finaldat.csv"))

In [58]:
bgibson_finaldat = compile_dat('bcfb0142', 'Ben-Gibson', 'Ben_Gibson', checkgames = True)
if isinstance(bgibson_finaldat, pd.DataFrame):
    bgibson_finaldat.to_csv(os.path.join(nest_folder_def, "bgibson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [59]:
giannoulis_finaldat = compile_dat('112c495e', 'Dimitris-Giannoulis', 'Giannoulis', checkgames = True)
if isinstance(giannoulis_finaldat, pd.DataFrame):
    giannoulis_finaldat.to_csv(os.path.join(nest_folder_def, "giannoulis_finaldat.csv"))

In [60]:
leesmelou_finaldat = compile_dat('f5044e54', 'Pierre-Lees-Melou', 'Lees-Melou', checkgames = True)
if isinstance(leesmelou_finaldat, pd.DataFrame):
    leesmelou_finaldat.to_csv(os.path.join(nest_folder, "leesmelou_finaldat.csv"))

In [62]:
sargent_finaldat = compile_dat('a8d4428b', 'Josh-Sargent', 'Joshua_Sargent', checkgames = True)
if isinstance(sargent_finaldat, pd.DataFrame):
    sargent_finaldat.to_csv(os.path.join(nest_folder, "sargent_finaldat.csv"))

In [63]:
normann_finaldat = compile_dat('8996350d', 'Mathias-Normann', 'Mathias_Normann', checkgames = True)
if isinstance(normann_finaldat, pd.DataFrame):
    normann_finaldat.to_csv(os.path.join(nest_folder, "normann_finaldat.csv"))

In [64]:
perraud_finaldat = compile_dat('6dfd8f05', 'Romain-Perraud', 'Perraud', checkgames = True)
if isinstance(perraud_finaldat, pd.DataFrame):
    perraud_finaldat.to_csv(os.path.join(nest_folder_def, "perraud_finaldat.csv"))

In [123]:
lyanco_finaldat = compile_dat('b5bf1770', 'Lyanco', 'Lyanco', checkgames = True)
if isinstance(lyanco_finaldat, pd.DataFrame):
    lyanco_finaldat.to_csv(os.path.join(nest_folder_def, "lyanco_finaldat.csv"))

In [66]:
tanganga_finaldat = compile_dat('e9971f2d', 'Japhet-Tanganga', 'Tanganga', checkgames = True)
if isinstance(tanganga_finaldat, pd.DataFrame):
    tanganga_finaldat.to_csv(os.path.join(nest_folder_def, "tanganga_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [124]:
emersonroyal_finaldat = compile_dat('df8b52a5', 'Emerson', 'Leite', checkgames = True)
if isinstance(emersonroyal_finaldat, pd.DataFrame):
    emersonroyal_finaldat.to_csv(os.path.join(nest_folder_def, "emersonroyal_finaldat.csv"))

In [68]:
troostekong_finaldat = compile_dat('7ed4921e', 'William-Troost-Ekong', 'Troost-Ekong', checkgames = True)
if isinstance(troostekong_finaldat, pd.DataFrame):
    troostekong_finaldat.to_csv(os.path.join(nest_folder_def, "troostekong_finaldat.csv"))

In [69]:
ngakia_finaldat = compile_dat('3631ad41', 'Jeremy-Ngakia', 'Ngakia', checkgames = True)
if isinstance(ngakia_finaldat, pd.DataFrame):
    ngakia_finaldat.to_csv(os.path.join(nest_folder_def, "ngakia_finaldat.csv"))

In [71]:
ilouza_finaldat = compile_dat('e1df6989', 'Imran-Louza', 'n_Louza', checkgames = True)
if isinstance(ilouza_finaldat, pd.DataFrame):
    ilouza_finaldat.to_csv(os.path.join(nest_folder, "ilouza_finaldat.csv"))

In [72]:
cucho_finaldat = compile_dat('71b47ab9', 'Cucho', 'Juan Camilo_Her', checkgames = True)
if isinstance(cucho_finaldat, pd.DataFrame):
    cucho_finaldat.to_csv(os.path.join(nest_folder, "cucho_finaldat.csv"))

In [73]:
kucka_finaldat = compile_dat('5aaef9ef', 'Juraj-Kucka', 'Juraj_Kucka', checkgames = True)
if isinstance(kucka_finaldat, pd.DataFrame):
    kucka_finaldat.to_csv(os.path.join(nest_folder, "kucka_finaldat.csv"))

In [74]:
hkamara_finaldat = compile_dat('19a12bed', 'Hassane-Kamara', 'Hassane_Kamara', checkgames = True)
if isinstance(hkamara_finaldat, pd.DataFrame):
    hkamara_finaldat.to_csv(os.path.join(nest_folder_def, "hkamara_finaldat.csv"))

In [75]:
samir_finaldat = compile_dat('92b4758e', 'Samir-Santos', 'Samir_Caetano', checkgames = True)
if isinstance(samir_finaldat, pd.DataFrame):
    samir_finaldat.to_csv(os.path.join(nest_folder_def, "samir_finaldat.csv"))

In [76]:
kayembe_finaldat = compile_dat('f99b71de', 'Edo-Kayembe', 'Edo_Kayembe', checkgames = True)
if isinstance(kayembe_finaldat, pd.DataFrame):
    kayembe_finaldat.to_csv(os.path.join(nest_folder, "kayembe_finaldat.csv"))

In [77]:
marcal_finaldat = compile_dat('9ae9eaea', 'Fernando-Marcal', 'Fernando_Mar', checkgames = True)
if isinstance(marcal_finaldat, pd.DataFrame):
    marcal_finaldat.to_csv(os.path.join(nest_folder_def, "marcal_finaldat.csv"))

In [78]:
trincao_finaldat = compile_dat('77e39b04', 'Francisco-Trincao', 'Francisco_Machado Mota', checkgames = True)
if isinstance(trincao_finaldat, pd.DataFrame):
    trincao_finaldat.to_csv(os.path.join(nest_folder, "trincao_finaldat.csv"))

In [79]:
kmoore_finaldat = compile_dat('01ce9e70', 'Kieffer-Moore', 'Kieffer_Moore', checkgames = True)
if isinstance(kmoore_finaldat, pd.DataFrame):
    kmoore_finaldat.to_csv(os.path.join(nest_folder, "kmoore_finaldat.csv"))

In [80]:
zemura_finaldat = compile_dat('e8271ad9', 'Jordan-Zemura', 'Zemura', checkgames = True)
if isinstance(zemura_finaldat, pd.DataFrame):
    zemura_finaldat.to_csv(os.path.join(nest_folder_def, "zemura_finaldat.csv"))

In [81]:
rothwell_finaldat = compile_dat('10a10454', 'Joe-Rothwell', 'Joe_Rothwell', checkgames = True)
if isinstance(rothwell_finaldat, pd.DataFrame):
    rothwell_finaldat.to_csv(os.path.join(nest_folder, "rothwell_finaldat.csv"))

In [82]:
koulibaly_finaldat = compile_dat('da974c7b', 'Kalidou-Koulibaly', 'Koulibaly', checkgames = True)
if isinstance(koulibaly_finaldat, pd.DataFrame):
    koulibaly_finaldat.to_csv(os.path.join(nest_folder_def, "koulibaly_finaldat.csv"))

In [83]:
kebano_finaldat = compile_dat('18b999f4', 'Neeskens-Kebano', 'Kebano', checkgames = True)
if isinstance(kebano_finaldat, pd.DataFrame):
    kebano_finaldat.to_csv(os.path.join(nest_folder, "kebano_finaldat.csv"))

In [84]:
palhinha_finaldat = compile_dat('a78ff07f', 'Joao-Palhinha', 'Palhinha', checkgames = True)
if isinstance(palhinha_finaldat, pd.DataFrame):
    palhinha_finaldat.to_csv(os.path.join(nest_folder, "palhinha_finaldat.csv"))

In [85]:
mmartins_finaldat = compile_dat('7120a04d', 'Tete', 'Mateus_Cardoso Lemos', checkgames = True)
if isinstance(mmartins_finaldat, pd.DataFrame):
    mmartins_finaldat.to_csv(os.path.join(nest_folder, "mmartins_finaldat.csv"))

In [86]:
kristensen_finaldat = compile_dat('39769cff', 'Rasmus-Kristensen', 'Rasmus_Kristensen', checkgames = True)
if isinstance(kristensen_finaldat, pd.DataFrame):
    kristensen_finaldat.to_csv(os.path.join(nest_folder_def, "kristensen_finaldat.csv"))

In [87]:
marcroca_finaldat = compile_dat('da27e268', 'Marc-Roca', 'Marc_Roca', checkgames = True)
if isinstance(marcroca_finaldat, pd.DataFrame):
    marcroca_finaldat.to_csv(os.path.join(nest_folder, "marcroca_finaldat.csv"))

In [88]:
aaronson_finaldat = compile_dat('5bc43860', 'Brenden-Aaronson', 'Brenden_Aaronson', checkgames = True)
if isinstance(aaronson_finaldat, pd.DataFrame):
    aaronson_finaldat.to_csv(os.path.join(nest_folder, "aaronson_finaldat.csv"))

In [89]:
gnonto_finaldat = compile_dat('75fdd638', 'Wilfried-Gnonto', 'Gnonto', checkgames = True)
if isinstance(gnonto_finaldat, pd.DataFrame):
    gnonto_finaldat.to_csv(os.path.join(nest_folder, "gnonto_finaldat.csv"))

In [90]:
wober_finaldat = compile_dat('4b803b3c', 'Maximilian-Wober', 'Maximilian_W', checkgames = True)
if isinstance(wober_finaldat, pd.DataFrame):
    wober_finaldat.to_csv(os.path.join(nest_folder_def, "wober_finaldat.csv"))

In [91]:
mckennie_finaldat = compile_dat('01c3aff5', 'Weston-McKennie', 'Weston_McKennie', checkgames = True)
if isinstance(mckennie_finaldat, pd.DataFrame):
    mckennie_finaldat.to_csv(os.path.join(nest_folder, "mckennie_finaldat.csv"))

In [92]:
mckenna_finaldat = compile_dat('4e5a4cbc', 'Scott-McKenna', 'Scott_McKenna', checkgames = True)
if isinstance(mckenna_finaldat, pd.DataFrame):
    mckenna_finaldat.to_csv(os.path.join(nest_folder_def, "mckenna_finaldat.csv"))

In [94]:
niakhate_finaldat = compile_dat('00242715', 'Moussa-Niakhate', 'Moussa_Niakhat', checkgames = True)
if isinstance(niakhate_finaldat, pd.DataFrame):
    niakhate_finaldat.to_csv(os.path.join(nest_folder_def, "niakhate_finaldat.csv"))

In [95]:
mangala_finaldat = compile_dat('a572e291', 'Orel-Mangala', 'Mangala', checkgames = True)
if isinstance(mangala_finaldat, pd.DataFrame):
    mangala_finaldat.to_csv(os.path.join(nest_folder, "mangala_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [125]:
freuler_finaldat = compile_dat('98d2c2c4', 'Remo-Freuler', 'Freuler', checkgames = True)
if isinstance(freuler_finaldat, pd.DataFrame):
    freuler_finaldat.to_csv(os.path.join(nest_folder, "freuler_finaldat.csv"))

In [97]:
renanlodi_finaldat = compile_dat('ed9f7cfe', 'Renan-Lodi', 'Renan Augusto_Lodi', checkgames = True)
if isinstance(renanlodi_finaldat, pd.DataFrame):
    renanlodi_finaldat.to_csv(os.path.join(nest_folder_def, "renanlodi_finaldat.csv"))

In [121]:
felipe_finaldat = compile_dat('51a3c4c8', 'Felipe', 'Felipe Augusto_de Almeida', checkgames = False)
if isinstance(felipe_finaldat, pd.DataFrame):
    felipe_finaldat.to_csv(os.path.join(nest_folder_def, "felipe_finaldat.csv"))

In [99]:
caletacar_finaldat = compile_dat('94f8d10c', 'Duje-Caleta-Car', 'Duje_Caleta', checkgames = True)
if isinstance(caletacar_finaldat, pd.DataFrame):
    caletacar_finaldat.to_csv(os.path.join(nest_folder_def, "caletacar_finaldat.csv"))

In [100]:
alcaraz_finaldat = compile_dat('4abac767', 'Carlos-Alcaraz', 'Carlos_Alcaraz', checkgames = True)
if isinstance(alcaraz_finaldat, pd.DataFrame):
    alcaraz_finaldat.to_csv(os.path.join(nest_folder, "alcaraz_finaldat.csv"))

In [101]:
perisic_finaldat = compile_dat('6fe90922', 'Ivan-Perisic', 'Ivan_Peri', checkgames = True)
if isinstance(perisic_finaldat, pd.DataFrame):
    perisic_finaldat.to_csv(os.path.join(nest_folder_def, "perisic_finaldat.csv"))

In [102]:
lenglet_finaldat = compile_dat('4f28a6ff', 'Clement-Lenglet', 'Lenglet', checkgames = True)
if isinstance(lenglet_finaldat, pd.DataFrame):
    lenglet_finaldat.to_csv(os.path.join(nest_folder_def, "lenglet_finaldat.csv"))

In [103]:
kehrer_finaldat = compile_dat('51dbeea9', 'Thilo-Kehrer', 'Thilo_Kehrer', checkgames = True)
if isinstance(kehrer_finaldat, pd.DataFrame):
    kehrer_finaldat.to_csv(os.path.join(nest_folder_def, "kehrer_finaldat.csv"))

In [104]:
scamacca_finaldat = compile_dat('8790f988', 'Gianluca-Scamacca', 'Scamacca', checkgames = True)
if isinstance(scamacca_finaldat, pd.DataFrame):
    scamacca_finaldat.to_csv(os.path.join(nest_folder, "scamacca_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [105]:
diegocosta_finaldat = compile_dat('a190d597', 'Diego-Costa', 'Diego_Da Silva Costa', checkgames = True)
if isinstance(diegocosta_finaldat, pd.DataFrame):
    diegocosta_finaldat.to_csv(os.path.join(nest_folder, "diegocosta_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [106]:
zaniolo_finaldat = compile_dat('9907b1c8', 'Nicolo-Zaniolo', 'Zaniolo', checkgames = True)
if isinstance(zaniolo_finaldat, pd.DataFrame):
    zaniolo_finaldat.to_csv(os.path.join(nest_folder, "zaniolo_finaldat.csv"))

In [107]:
yarmoliuk_finaldat = compile_dat('907a5d7c', 'Yehor-Yarmoliuk', 'Yarmoliuk', checkgames = True)
if isinstance(yarmoliuk_finaldat, pd.DataFrame):
    yarmoliuk_finaldat.to_csv(os.path.join(nest_folder, "yarmoliuk_finaldat.csv"))

In [108]:
ghoddos_finaldat = compile_dat('f7b1d4e0', 'Saman-Ghoddos', 'Ghoddos', checkgames = True)
if isinstance(ghoddos_finaldat, pd.DataFrame):
    ghoddos_finaldat.to_csv(os.path.join(nest_folder, "ghoddos_finaldat.csv"))

In [109]:
aldakhil_finaldat = compile_dat('93e08bce', 'Ameen-Al-Dakhil', 'Ameen_Al', checkgames = True)
if isinstance(aldakhil_finaldat, pd.DataFrame):
    aldakhil_finaldat.to_csv(os.path.join(nest_folder_def, "aldakhil_finaldat.csv"))

In [110]:
beyer_finaldat = compile_dat('9af1f9ff', 'Jordan-Beyer', 'Jordan_Beyer', checkgames = True)
if isinstance(beyer_finaldat, pd.DataFrame):
    beyer_finaldat.to_csv(os.path.join(nest_folder_def, "beyer_finaldat.csv"))

In [111]:
cullen_finaldat = compile_dat('f6d7d690', 'Josh-Cullen', 'Josh_Cullen', checkgames = True)
if isinstance(cullen_finaldat, pd.DataFrame):
    cullen_finaldat.to_csv(os.path.join(nest_folder, "cullen_finaldat.csv"))

In [112]:
foster_finaldat = compile_dat('f84a807c', 'Lyle-Foster', 'Lyle_Foster', checkgames = True)
if isinstance(foster_finaldat, pd.DataFrame):
    foster_finaldat.to_csv(os.path.join(nest_folder, "foster_finaldat.csv"))

In [113]:
vitinho_finaldat = compile_dat('9295dc1c', 'Vitinho', 'Victor_da Silva', checkgames = True)
if isinstance(vitinho_finaldat, pd.DataFrame):
    vitinho_finaldat.to_csv(os.path.join(nest_folder_def, "vitinho_finaldat.csv"))

In [114]:
amdouni_finaldat = compile_dat('2ee5b0c9', 'Zeki-Amdouni', 'Amdouni', checkgames = True)
if isinstance(amdouni_finaldat, pd.DataFrame):
    amdouni_finaldat.to_csv(os.path.join(nest_folder, "amdouni_finaldat.csv"))

In [115]:
koleosho_finaldat = compile_dat('fc076f78', 'Luca-Koleosho', 'Koleosho', checkgames = True)
if isinstance(koleosho_finaldat, pd.DataFrame):
    koleosho_finaldat.to_csv(os.path.join(nest_folder, "koleosho_finaldat.csv"))

In [116]:
bruunlarsen_finaldat = compile_dat('4e204552', 'Jacob-Bruun-Larsen', 'Jacob_Bruun', checkgames = True)
if isinstance(bruunlarsen_finaldat, pd.DataFrame):
    bruunlarsen_finaldat.to_csv(os.path.join(nest_folder, "bruunlarsen_finaldat.csv"))

In [117]:
assignon_finaldat = compile_dat('b026bbf6', 'Lorenz-Assignon', 'Assignon', checkgames = True)
if isinstance(assignon_finaldat, pd.DataFrame):
    assignon_finaldat.to_csv(os.path.join(nest_folder_def, "assignon_finaldat.csv"))

In [118]:
esteve_finaldat = compile_dat('1a066e69', 'Maxime-Esteve', 'Maxime_Es', checkgames = True)
if isinstance(esteve_finaldat, pd.DataFrame):
    esteve_finaldat.to_csv(os.path.join(nest_folder_def, "esteve_finaldat.csv"))

In [119]:
adebayo_finaldat = compile_dat('0480ebc7', 'Elijah-Adebayo', 'Elijah_Adebayo', checkgames = True)
if isinstance(adebayo_finaldat, pd.DataFrame):
    adebayo_finaldat.to_csv(os.path.join(nest_folder, "adebayo_finaldat.csv"))

In [126]:
abell_finaldat = compile_dat('399e9474', 'Amarii-Bell', 'Amarii', checkgames = True)
if isinstance(abell_finaldat, pd.DataFrame):
    abell_finaldat.to_csv(os.path.join(nest_folder_def, "abell_finaldat.csv"))

In [127]:
rburke_finaldat = compile_dat('a681b7b7', 'Reece-Burke', 'Reece_Burke', checkgames = True)
if isinstance(rburke_finaldat, pd.DataFrame):
    rburke_finaldat.to_csv(os.path.join(nest_folder_def, "rburke_finaldat.csv"))

In [128]:
doughty_finaldat = compile_dat('bf33b257', 'Alfie-Doughty', 'Alfie_Doughty', checkgames = True)
if isinstance(doughty_finaldat, pd.DataFrame):
    doughty_finaldat.to_csv(os.path.join(nest_folder_def, "doughty_finaldat.csv"))

In [129]:
jclark_finaldat = compile_dat('7c5fa6cf', 'Jordan-Clark', 'Jordan_Clark', checkgames = True)
if isinstance(jclark_finaldat, pd.DataFrame):
    jclark_finaldat.to_csv(os.path.join(nest_folder, "jclark_finaldat.csv"))

In [130]:
cmorris_finaldat = compile_dat('2da834f1', 'Carlton-Morris', 'Carlton_Morris', checkgames = True)
if isinstance(cmorris_finaldat, pd.DataFrame):
    cmorris_finaldat.to_csv(os.path.join(nest_folder, "cmorris_finaldat.csv"))

In [131]:
ogbene_finaldat = compile_dat('788c0277', 'Chiedozie-Ogbene', 'Ogbene', checkgames = True)
if isinstance(ogbene_finaldat, pd.DataFrame):
    ogbene_finaldat.to_csv(os.path.join(nest_folder, "ogbene_finaldat.csv"))

In [132]:
osho_finaldat = compile_dat('cd889153', 'Gabriel-Osho', 'Gabriel_Osho', checkgames = True)
if isinstance(osho_finaldat, pd.DataFrame):
    osho_finaldat.to_csv(os.path.join(nest_folder_def, "osho_finaldat.csv"))

In [135]:
mengi_finaldat = compile_dat('91748070', 'Teden-Mengi', 'Teden_Mengi', checkgames = True)
if isinstance(mengi_finaldat, pd.DataFrame):
    mengi_finaldat.to_csv(os.path.join(nest_folder_def, "mengi_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [136]:
lockyer_finaldat = compile_dat('37118440', 'Tom-Lockyer', 'Tom_Lockyer', checkgames = True)
if isinstance(lockyer_finaldat, pd.DataFrame):
    lockyer_finaldat.to_csv(os.path.join(nest_folder_def, "lockyer_finaldat.csv"))

In [137]:
mpanzu_finaldat = compile_dat('a9b1c319', 'Pelly-Ruddock-Mpanzu', 'Mpanzu', checkgames = True)
if isinstance(mpanzu_finaldat, pd.DataFrame):
    mpanzu_finaldat.to_csv(os.path.join(nest_folder, "mpanzu_finaldat.csv"))

In [138]:
chong_finaldat = compile_dat('2696b6a9', 'Tahith-Chong', 'Tahith_Chong', checkgames = True)
if isinstance(chong_finaldat, pd.DataFrame):
    chong_finaldat.to_csv(os.path.join(nest_folder, "chong_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_71171/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [139]:
jbrown_finaldat = compile_dat('ca18c8f1', 'Jacob-Brown', 'Jacob_Brown', checkgames = True)
if isinstance(jbrown_finaldat, pd.DataFrame):
    jbrown_finaldat.to_csv(os.path.join(nest_folder, "jbrown_finaldat.csv"))

In [140]:
amrabat_finaldat = compile_dat('5a2cb25d', 'Sofyan-Amrabat', 'Sofyan_Amrabat', checkgames = True)
if isinstance(amrabat_finaldat, pd.DataFrame):
    amrabat_finaldat.to_csv(os.path.join(nest_folder, "amrabat_finaldat.csv"))

In [141]:
montiel_finaldat = compile_dat('374d5158', 'Gonzalo-Montiel', 'Montiel', checkgames = True)
if isinstance(montiel_finaldat, pd.DataFrame):
    montiel_finaldat.to_csv(os.path.join(nest_folder_def, "montiel_finaldat.csv"))

In [142]:
trusty_finaldat = compile_dat('cddf767b', 'Auston-Trusty', 'Auston_Trusty', checkgames = True)
if isinstance(trusty_finaldat, pd.DataFrame):
    trusty_finaldat.to_csv(os.path.join(nest_folder_def, "trusty_finaldat.csv"))

In [134]:
ahmedhodzic_finaldat = compile_dat('eab957a3', 'Anel-Ahmedhodzic', 'Ahmedhod', checkgames = True)
if isinstance(ahmedhodzic_finaldat, pd.DataFrame):
    ahmedhodzic_finaldat.to_csv(os.path.join(nest_folder_def, "ahmedhodzic_finaldat.csv"))

In [143]:
vinisouza_finaldat = compile_dat('6a8a7af8', 'Vinicius-Souza', 'Vini_de Souza Costa', checkgames = True)
if isinstance(vinisouza_finaldat, pd.DataFrame):
    vinisouza_finaldat.to_csv(os.path.join(nest_folder, "vinisouza_finaldat.csv"))

In [144]:
abrooks_finaldat = compile_dat('5f346071', 'Andre-Brooks', 'Andre_Brooks', checkgames = True)
if isinstance(abrooks_finaldat, pd.DataFrame):
    abrooks_finaldat.to_csv(os.path.join(nest_folder, "abrooks_finaldat.csv"))

In [145]:
hamer_finaldat = compile_dat('eebacc07', 'Gustavo-Hamer', 'Gustavo_Hamer', checkgames = True)
if isinstance(hamer_finaldat, pd.DataFrame):
    hamer_finaldat.to_csv(os.path.join(nest_folder, "hamer_finaldat.csv"))

In [146]:
arblaster_finaldat = compile_dat('afe1a491', 'Oliver-Arblaster', 'Arblaster', checkgames = True)
if isinstance(arblaster_finaldat, pd.DataFrame):
    arblaster_finaldat.to_csv(os.path.join(nest_folder, "arblaster_finaldat.csv"))

# Other Data Grab for Data Exploration Purpose 

In [2]:
url = 'https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')
html_filtered = soup.find('tbody')

In [16]:
finalpos = []
for row in html_filtered.find_all('tr'):
    if not row.attrs:
        finalpos.append(row.find('th', {'data-stat': 'rank'}).text)

In [15]:
teamname = []
for row in html_filtered.find_all('tr'):
    if not row.attrs:
        teamname.append(row.find('td', attrs={'data-stat': 'team'}).find('a').text)

In [21]:
season = ['2023-2024']*20

In [23]:
leaguetable = pd.DataFrame({'Team': teamname, 'Final Position': finalpos, 'Season': season})

In [24]:
def get_league_table():
    years = ('2023-2024', '2022-2023', '2021-2022', '2020-2021', '2019-2020', '2018-2019', '2017-2018')
    finalpos = []
    teamname = []
    season = []
    
    for year in years:
        url = f'https://fbref.com/en/comps/9/{year}/{year}-Premier-League-Stats'
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'lxml')
        html_filtered = soup.find('tbody')
    
        for row in html_filtered.find_all('tr'):
            if not row.attrs:
                finalpos.append(row.find('th', {'data-stat': 'rank'}).text)
        
        for row in html_filtered.find_all('tr'):
            if not row.attrs:
                teamname.append(row.find('td', attrs={'data-stat': 'team'}).find('a').text)
                
        for row in html_filtered.find_all('tr'):
            season.append(year)
        
        time.sleep(60)
    
    leaguetable = pd.DataFrame({'Team': teamname, 'Position': finalpos, 'Season': season})
    return(leaguetable)

In [26]:
prem_table = get_league_table()

In [27]:
prem_table.to_csv(os.path.join("", "prem_table.csv"))

# Compile Defender Data

This section compiles all individual defender CSV files from the `Player_Data/Defenders/` folder into a single `def_finaldat.csv` file.

In [ ]:
# Compile all defender files into def_finaldat.csv
defender_path = 'Player_Data/Defenders'

defender_dataframes = []

if os.path.exists(defender_path):
    for file in sorted(os.listdir(defender_path)):
        if file.endswith('.csv'):
            file_path = os.path.join(defender_path, file)
            df = pd.read_csv(file_path)
            defender_dataframes.append(df)
    
    # Concatenate all defender dataframes
    def_finaldat = pd.concat(defender_dataframes, ignore_index=True)
    
    print(f"Compiled {len(defender_dataframes)} defender files")
    print(f"Total rows: {len(def_finaldat):,}")
    print(f"Unique players: {def_finaldat['Player ID'].nunique()}")

In [ ]:
# Save the compiled defender data
def_finaldat.to_csv('def_finaldat.csv', index=False)
print("✓ Saved to def_finaldat.csv")